In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE

In [2]:
cols = ['Currency','Account Type','Accounting Net Amount',
#'Accounting Net Amount Difference','Accounting Net Amount Difference Absolute ',
'Activity Code','Age','Age WK',
'Asset Type Category','Base Currency','Base Net Amount','Bloomberg_Yellow_Key',
'Cust Net Amount',
#'B-P Net Amount Difference','B-P Net Amount Difference Absolute',
'BreakID',
'Business Date','Cancel Amount','Cancel Flag','CUSIP','Custodian',
'Custodian Account',
'Derived Source','Description','Department','ExpiryDate','ExternalComment1','ExternalComment2',
'ExternalComment3','Fund','FX Rate','Interest Amount','InternalComment1','InternalComment2',
'InternalComment3','Investment Type','Is Combined Data','ISIN','Keys',
'Mapped Custodian Account','Net Amount Difference','Net Amount Difference Absolute','Non Trade Description',
'OTE Custodian Account',
#'Predicted Action','Predicted Status','Prediction Details',
'Price','Prime Broker',
'Quantity','SEDOL','Settle Date','SPM ID','Status','Strike Price',
'System Comments','Ticker','Trade Date','Trade Expenses','Transaction Category','Transaction ID','Transaction Type',
'Underlying Cusip','Underlying Investment ID','Underlying ISIN','Underlying Sedol','Underlying Ticker','Source Combination','_ID']
#'UnMapped']

add = ['ViewData.Side0_UniqueIds', 'ViewData.Side1_UniqueIds',
      # 'MetaData.0._RecordID','MetaData.1._RecordID',
       'ViewData.Task Business Date']


In [3]:
new_cols = ['ViewData.' + x for x in cols] + add

In [4]:
#df_170.shape

## Close Prediction OakTree

In [1074]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep  1 00:42:02 2020

@author: consultant138
"""

# -*- coding: utf-8 -*-
"""
Created on Thu Aug 13 19:12:48 2020

@author: consultant138
"""

import pandas as pd
import os
import sys
import numpy as np
from datetime import datetime, date
from difflib import SequenceMatcher
import pprint
import json



# In[2]:


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def dictionary_exclude_keys(fun_dict, fun_keys_to_exclude):
    return {x: fun_dict[x] for x in fun_dict if x not in fun_keys_to_exclude}

def write_dict_at_top(fun_filename, fun_dict_to_add):
    with open(fun_filename, 'r+') as f:
        fun_existing_content = f.read()
        f.seek(0, 0)
        f.write(json.dumps(fun_dict_to_add, indent = 4))
        f.write('\n')
        f.write(fun_existing_content)

def normalize_bp_acct_col_names(fun_df):
    bp_acct_col_names_mapping_dict = {
                                      'ViewData.Cust Net Amount' : 'ViewData.B-P Net Amount',
                                      'ViewData.Cust Net Amount Difference' : 'ViewData.B-P Net Amount Difference',
                                      'ViewData.Cust Net Amount Difference Absolute' : 'ViewData.B-P Net Amount Difference Absolute',
                                      'ViewData.CP Net Amount' : 'ViewData.B-P Net Amount',
                                      'ViewData.CP Net Amount Difference' : 'ViewData.B-P Net Amount Difference',
                                      'ViewData.CP Net Amount Difference Absolute' : 'ViewData.B-P Net Amount Difference Absolute',
                                      'ViewData.PMSVendor Net Amount' : 'ViewData.Accounting Net Amount'
                                        }
    fun_df.rename(columns = bp_acct_col_names_mapping_dict, inplace = True)
    return(fun_df)

# In[3]:


pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)


# In[4]:


pd.options.display.max_colwidth = 1000


# In[5]:


pd.options.display.float_format = '{:.5f}'.format


# M X M and N X N architecture for closed break prediction
def closed_cols():
    cols_for_closed_list = ['Status','Source Combination','Mapped Custodian Account',
                   'Accounting Currency','B-P Currency', 
                   'Transaction ID','Transaction Type','Description','Investment ID',
                   'Accounting Net Amount','B-P Net Amount', 
                   'InternalComment2','Custodian','Fund']
    cols_for_closed_list = ['ViewData.' + x for x in cols_for_closed_list]
    cols_for_closed_x_list = [x + '_x' for x in cols_for_closed_list] + ['ViewData.Side0_UniqueIds_x','ViewData.Side1_UniqueIds_x']
    cols_for_closed_y_list = [x + '_y' for x in cols_for_closed_list] + ['ViewData.Side0_UniqueIds_y','ViewData.Side1_UniqueIds_y']
    cols_for_closed_x_y_list = cols_for_closed_x_list + cols_for_closed_y_list
    return({
            'cols_for_closed' : cols_for_closed_list,
            'cols_for_closed_x' : cols_for_closed_x_list,
            'cols_for_closed_y' : cols_for_closed_y_list,
            'cols_for_closed_x_y' : cols_for_closed_x_y_list
            })

def cleaned_meo(fun_filepath_meo):
    meo = pd.read_csv(fun_filepath_meo) \
          .drop_duplicates() \
          .reset_index() \
          .drop('index',1)
    
    meo = normalize_bp_acct_col_names(fun_df = meo)
    
    meo = meo[~meo['ViewData.Status'].isin(['SMT','HST', 'OC', 'CT', 'Archive','SMR'])] 
    meo = meo[~meo['ViewData.Status'].isnull()] \
          .reset_index() \
          .drop('index',1)
    
    meo['Date'] = pd.to_datetime(meo['ViewData.Task Business Date'])
    meo = meo[~meo['Date'].isnull()] \
          .reset_index() \
          .drop('index',1)
    meo['Date'] = pd.to_datetime(meo['Date']).dt.date
    meo['Date'] = meo['Date'].astype(str)

    meo['ViewData.Side0_UniqueIds'] = meo['ViewData.Side0_UniqueIds'].astype(str)
    meo['ViewData.Side1_UniqueIds'] = meo['ViewData.Side1_UniqueIds'].astype(str)

    meo['flag_side0'] = meo.apply(lambda x: len(x['ViewData.Side0_UniqueIds'].split(',')), axis=1)
    meo['flag_side1'] = meo.apply(lambda x: len(x['ViewData.Side1_UniqueIds'].split(',')), axis=1)

    meo.loc[meo['ViewData.Side0_UniqueIds']=='nan','flag_side0'] = 0
    meo.loc[meo['ViewData.Side1_UniqueIds']=='nan','flag_side1'] = 0

    meo['ViewData.BreakID'] = meo['ViewData.BreakID'].astype(int)
    meo = meo[meo['ViewData.BreakID']!=-1] \
          .reset_index() \
          .drop('index',1)
          
    meo['Side_0_1_UniqueIds'] = meo['ViewData.Side0_UniqueIds'].astype(str) + \
                                meo['ViewData.Side1_UniqueIds'].astype(str)
                                
    meo = meo.sort_values(by=['ViewData.Transaction ID','ViewData.Transaction Type'],ascending = False)
    return(meo)
    
def cleaned_aua(fun_filepath_aua):
    aua = pd.read_csv(fun_filepath_aua) \
      .drop_duplicates() \
      .reset_index() \
      .drop('index',1) \
      .sort_values(by=['ViewData.Transaction ID','ViewData.Transaction Type'],ascending = False)

    aua = normalize_bp_acct_col_names(fun_df = aua)

    
    aua['Side_0_1_UniqueIds'] = aua['ViewData.Side0_UniqueIds'].astype(str) + \
                                aua['ViewData.Side1_UniqueIds'].astype(str)
    
    return(aua)

def Acct_MEO_combination_file(fun_side, fun_cleaned_meo_df):
    if(fun_side == 'PB' or fun_side == 'BP' or fun_side == 'B-P' or fun_side == 'Prime Broker'):
        side_meo = fun_cleaned_meo_df[(fun_cleaned_meo_df['flag_side1'] >= 1) & (fun_cleaned_meo_df['flag_side0'] == 0)]
#        Currency_col_name = 'ViewData.B-P Currency'
    elif(fun_side == 'Acct' or fun_side == 'Accounting'):
        side_meo = fun_cleaned_meo_df[(fun_cleaned_meo_df['flag_side1'] == 0) & (fun_cleaned_meo_df['flag_side0'] >= 1)]
#        Currency_col_name = 'ViewData.Accounting Currency'
    else:
        print('The only options for side are on of the following : ')
        print('For Prime Broker side, the options are PB or BP or B-P or Prime Broker')
        print('For Accounting side, the options are Acct or Accounting')
        raise ValueError('Exiting function because fun_side argument was not from the accepted set of parameter values')
    
    side_meo['filter_key'] = side_meo['ViewData.Source Combination'].astype(str) + \
                         side_meo['ViewData.Mapped Custodian Account'].astype(str) + \
                         side_meo['ViewData.Currency'].astype(str)
        
    side_meo_training_df =[]
    for key in (list(np.unique(np.array(list(side_meo['filter_key'].values))))):
        side_meo_filter_slice = side_meo[side_meo['filter_key']==key]
        if side_meo_filter_slice.empty == False:
    
            side_meo_filter_slice = side_meo_filter_slice.reset_index()
            side_meo_filter_slice = side_meo_filter_slice.drop('index', 1)
    
            side_meo_filter_joined = pd.merge(side_meo_filter_slice, side_meo_filter_slice, on='filter_key')
            side_meo_training_df.append(side_meo_filter_joined)
    return(pd.concat(side_meo_training_df))
    
def identifying_closed_breaks_from_Trans_type(fun_side, fun_transaction_type_list, fun_side_meo_combination_df, fun_setup_code_crucial):
    if(fun_side == 'PB' or fun_side == 'BP' or fun_side == 'B-P' or fun_side == 'Prime Broker'):
        Net_amount_col_name_list = ['ViewData.B-P Net Amount_' + x for x in ['x','y']]
        Side_0_1_UniqueIds_col_name_list = ['ViewData.Side1_UniqueIds_' + x for x in ['x','y']]
    elif(fun_side == 'Acct' or fun_side == 'Accounting'):
        Net_amount_col_name_list = ['ViewData.Accounting Net Amount_' + x for x in ['x','y']]
        Side_0_1_UniqueIds_col_name_list = ['ViewData.Side0_UniqueIds_' + x for x in ['x','y']]
    else:
        print('The only options for side are on of the following : ')
        print('For Prime Broker side, the options are PB or BP or B-P or Prime Broker')
        print('For Accounting side, the options are Acct or Accounting')
        raise ValueError('Exiting function because fun_side argument was not from the accepted set of parameter values')        
    
    if(fun_setup_code_crucial == '379'):
        Transaction_type_closed_break_df = \
            fun_side_meo_combination_df[ \
                    (fun_side_meo_combination_df['ViewData.Transaction Type_x'].astype(str).isin(fun_transaction_type_list)) & \
                    (fun_side_meo_combination_df['ViewData.Transaction Type_y'].astype(str).isin(fun_transaction_type_list)) & \
                    (abs(fun_side_meo_combination_df[Net_amount_col_name_list[0]]).astype(str) == abs(fun_side_meo_combination_df[Net_amount_col_name_list[1]]).astype(str)) & \
                    (fun_side_meo_combination_df[Side_0_1_UniqueIds_col_name_list[0]].astype(str) != fun_side_meo_combination_df[Side_0_1_UniqueIds_col_name_list[1]].astype(str)) \
                    ]
    return(set(
                Transaction_type_closed_break_df['ViewData.Side0_UniqueIds_x'].astype(str) + \
                Transaction_type_closed_break_df['ViewData.Side1_UniqueIds_x'].astype(str)
               ))

def closed_breaks_captured_mode(fun_aua_df, fun_transaction_type, fun_captured_closed_breaks_set, fun_mode):
    if(fun_transaction_type != 'All_Closed_Breaks'):
        aua_df = fun_aua_df[(fun_aua_df['ViewData.Status'] == 'UCB') & \
                            (fun_aua_df['ViewData.Transaction Type'] == fun_transaction_type)]
    else:
        aua_df = fun_aua_df[(fun_aua_df['ViewData.Status'] == 'UCB')]
        
    aua_side_0_1_UniqueIds_set = set(aua_df['ViewData.Side0_UniqueIds'].astype(str) + \
                                 aua_df['ViewData.Side1_UniqueIds'].astype(str))
    if(fun_mode == 'Correctly_Captured_In_AUA'):
        list_to_return = list(aua_side_0_1_UniqueIds_set & fun_captured_closed_breaks_set)
    elif(fun_mode == 'Not_Captured_In_AUA'):
        list_to_return = list(aua_side_0_1_UniqueIds_set - fun_captured_closed_breaks_set)
    elif(fun_mode == 'Over_Captured_In_AUA'):
        list_to_return = list(fun_captured_closed_breaks_set - aua_side_0_1_UniqueIds_set)
    return(list_to_return)

def update_dict_to_output_breakids_number_pct(fun_dict, fun_aua_df, fun_loop_transaction_type, fun_count, fun_Side_0_1_UniqueIds_list):
    mode_type_list = ['Correctly_Captured_In_AUA','Not_Captured_In_AUA','Over_Captured_In_AUA']
    for mode_type in mode_type_list:
#    if(fun_loop_transaction_type != 'All_Closed_Breaks'):
        fun_dict[fun_loop_transaction_type][mode_type + '_BreakIDs_in_AUA'] = list(set(\
            fun_aua_df[fun_aua_df['Side_0_1_UniqueIds'].isin(\
                     closed_breaks_captured_mode(fun_aua_df = fun_aua_df, \
                                        fun_transaction_type = fun_loop_transaction_type, \
                                        fun_captured_closed_breaks_set = set(fun_Side_0_1_UniqueIds_list), \
                                        fun_mode = mode_type))]\
                    ['ViewData.BreakID']))
    
        fun_total_number = len(\
                             fun_dict[fun_loop_transaction_type][mode_type + '_BreakIDs_in_AUA'])
        
        fun_dict[fun_loop_transaction_type][mode_type + '_Total_Number'] = len(\
                             fun_dict[fun_loop_transaction_type][mode_type + '_BreakIDs_in_AUA'])
        
        if(fun_count != 0):
            
            fun_dict[fun_loop_transaction_type][mode_type + '_Percentage'] = fun_total_number/fun_count#\
#                                 fun_dict[fun_loop_transaction_type][mode_type + '_Total_Number']/fun_count
        
        else:
            fun_dict[fun_loop_transaction_type][mode_type + '_Percentage'] = fun_loop_transaction_type + ' not found in Closed breaks of AUA'
    return(fun_dict)

def closed_daily_run(fun_setup_code, fun_date, fun_main_filepath_meo, fun_main_filepath_aua):
    setup_val = fun_setup_code
    main_meo = cleaned_meo(fun_filepath_meo = fun_main_filepath_meo)
    
    BP_meo_training_df = Acct_MEO_combination_file(fun_side = 'PB', \
                                                   fun_cleaned_meo_df = main_meo)
    
    Acct_meo_training_df = Acct_MEO_combination_file(fun_side = 'Acct', \
                                                   fun_cleaned_meo_df = main_meo)

    main_aua = cleaned_aua(fun_filepath_aua = fun_main_filepath_aua)
    
    if(fun_setup_code == '379'):
        Transaction_Type_dict = {
                                'Interest BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['Interest'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Interest Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Interest'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'STIF Interest BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['STIF Interest'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'STIF Interest Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['STIF Interest'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Buy BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['Buy'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Buy Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Buy'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Sell BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['Sell'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Sell Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Sell'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'ForwardFX BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['ForwardFX'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'ForwardFX Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['ForwardFX'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Internal Trans' : {'side' : 'PB',
                                           'Transaction_Type' : ['Internal Trans'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Withdraw' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Withdraw'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Deposit' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Deposit'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Redemption' : {'side' : 'PB',
                                           'Transaction_Type' : ['Redemption'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Subscription' : {'side' : 'PB',
                                           'Transaction_Type' : ['Redemption'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Incoming Wire' : {'side' : 'PB',
                                           'Transaction_Type' : ['Incoming Wire'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Transfer' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Transfer'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Withdrawal BP_side' : {'side' : 'PB',
                                           'Transaction_Type' : ['Withdrawal'],
                                           'Side_meo_training_df' : BP_meo_training_df},
                                'Withdrawal Acct_side' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Withdrawal'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Revenue' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Revenue'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Pay Down' : {'side' : 'Acct',
                                           'Transaction_Type' : ['Pay Down'],
                                           'Side_meo_training_df' : Acct_meo_training_df},
                                'Over & Short' : {'side' : 'PB',
                                           'Transaction_Type' : ['Over & Short'],
                                           'Side_meo_training_df' : BP_meo_training_df}
                                }
        
    
    print(os.getcwd())
    os.chdir('D:\\ViteosModel\\Closed')
    print(os.getcwd())
    
    filepath_stdout = fun_setup_code + '_closed_run_date_' + str(fun_date) + '_timestamp_' + str(datetime.now().strftime("%d_%m_%Y_%H_%M")) + '.txt'
    orig_stdout = sys.stdout
    f = open(filepath_stdout, 'w')
    sys.stdout = f
    
    Side_0_1_UniqueIds_closed_all_list = []
    for Transaction_type in Transaction_Type_dict:

        Side_0_1_UniqueIds_for_Transaction_type = identifying_closed_breaks_from_Trans_type(fun_side = Transaction_Type_dict.get(Transaction_type).get('side'), \
                                                                                  fun_transaction_type_list = Transaction_Type_dict.get(Transaction_type).get('Transaction_Type'), \
                                                                                  fun_side_meo_combination_df = Transaction_Type_dict.get(Transaction_type).get('Side_meo_training_df'),\
                                                                                  fun_setup_code_crucial = setup_val)

        count_closed_breaks_for_transaction_type = len(set(main_aua[(main_aua['ViewData.Status'] == 'UCB') & \
                    (main_aua['ViewData.Transaction Type'] == Transaction_type)]['Side_0_1_UniqueIds']))
        
        Transaction_Type_dict = update_dict_to_output_breakids_number_pct(fun_dict = Transaction_Type_dict, \
                                                                   fun_aua_df = main_aua, \
                                                                   fun_loop_transaction_type = Transaction_type, \
                                                                   fun_count = count_closed_breaks_for_transaction_type, \
                                                                   fun_Side_0_1_UniqueIds_list = Side_0_1_UniqueIds_for_Transaction_type)
            
        
        Side_0_1_UniqueIds_closed_all_list.extend(Side_0_1_UniqueIds_for_Transaction_type)
        print('\n' + Transaction_type + '\n')
        pprint.pprint(dictionary_exclude_keys(fun_dict = Transaction_Type_dict.get(Transaction_type),\
                                      fun_keys_to_exclude = {'side','Transaction_Type','Side_meo_training_df'}),\
                      width = 4)
    
    sys.stdout = orig_stdout
    f.close()
    
    count_all_closed_breaks = len(set(main_aua[(main_aua['ViewData.Status'] == 'UCB')]\
                                               ['Side_0_1_UniqueIds']))
    
    aua_closed_dict = {'All_Closed_Breaks' : {}}
    aua_closed_dict = update_dict_to_output_breakids_number_pct(fun_dict = aua_closed_dict, \
                                                                fun_aua_df = main_aua, \
                                                                fun_loop_transaction_type = 'All_Closed_Breaks', \
                                                                fun_count = count_all_closed_breaks, \
                                                                fun_Side_0_1_UniqueIds_list = Side_0_1_UniqueIds_closed_all_list)
    
    write_dict_at_top(fun_filename = filepath_stdout, \
                      fun_dict_to_add = aua_closed_dict)
    
    return(Side_0_1_UniqueIds_closed_all_list)
    
    

date_numbers_list = [16]
                     #2,3,4,
                    # 7,8,9,10,11,
                    # 14,15,16,17,18,
                    # 21,22,23,24,25,
                    # 28,29,30]

client = 'OakTree'

setup = '379'

filepaths_AUA = ['//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/' + client + '/JuneData/AUA/AUACollections.AUA_HST_RecData_' + setup + '_2020-06-' + str(date_numbers_list[i]) + '.csv' for i in range(0,len(date_numbers_list))]
filepaths_MEO = ['//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/' + client + '/JuneData/MEO/MeoCollections.MEO_HST_RecData_' + setup + '_2020-06-' + str(date_numbers_list[i]) + '.csv' for i in range(0,len(date_numbers_list))]


Side_0_1_UniqueIds_closed_all_dates_list = []

i = 0
for i in range(0,len(date_numbers_list)):

    Side_0_1_UniqueIds_closed_all_dates_list.append(
            closed_daily_run(fun_setup_code=setup,\
                             fun_date = i,\
                             fun_main_filepath_meo= filepaths_MEO[i],\
                             fun_main_filepath_aua = filepaths_AUA[i])
            )

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\consultant136\ML1.0
D:\ViteosModel\Closed


In [1100]:
new_closed_keys = [i.replace('nan','') for i in Side_0_1_UniqueIds_closed_all_dates_list[0]]

## Read testing data 

In [1077]:
#MeoCollections.MEO_HST_RecData_379_2020-06-18
meo = pd.read_csv("//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/OakTree/JuneData/MEO/MeoCollections.MEO_HST_RecData_379_2020-06-16.csv",usecols=new_cols)

In [1078]:
#df['ViewData.Task Business Date']

In [1079]:
meo['ViewData.Status'].value_counts()

OB     758
UMB    242
SMR    201
SDB     28
SMB     20
UOB     15
SMT     11
UDB     10
SPM      3
CNF      1
Name: ViewData.Status, dtype: int64

In [1080]:
df1 = meo[~meo['ViewData.Status'].isin(['SMT','HST', 'OC', 'CT', 'Archive','SMR'])]
#df = df[df['MatchStatus'] != 21]
df1 = df1[~df1['ViewData.Status'].isnull()]
df1 = df1.reset_index()
df1 = df1.drop('index',1)

In [1081]:
#df1[(df1['Date']=='2020-04-10') & (df1['ViewData.Side1_UniqueIds']=='996_125813417_Goldman Sachs')]
df1.shape

(1077, 65)

In [1082]:
df1['close_key'] = df1['ViewData.Side0_UniqueIds'].astype(str) + df1['ViewData.Side1_UniqueIds'].astype(str)


In [1093]:

## Output for Closed breaks

closed_df = df1[df1['close_key'].isin(list(Side_0_1_UniqueIds_closed_all_dates_list[0]))]

In [1107]:
closed_df.shape

(76, 66)

## Machine generated output

In [1105]:
#df2 = df1[~df1['close_key'].isin(list(all_closed))]
df2 = df1[~((df1['ViewData.Side1_UniqueIds'].isin(new_closed_keys)) | (df1['ViewData.Side0_UniqueIds'].isin(new_closed_keys)))]

In [1108]:
df = df2.copy()

In [1109]:
df = df.reset_index()
df = df.drop('index',1)

In [1110]:
df.shape

(1001, 66)

In [1113]:
#pd.set_option('display.max_columns', 500)

In [1114]:
df['Date'] = pd.to_datetime(df['ViewData.Task Business Date'])

In [1115]:
#df['Date'] = pd.to_datetime(df['ViewData.Task Business Date'])

In [1116]:
df = df[~df['Date'].isnull()]
df = df.reset_index()
df = df.drop('index',1)

In [1117]:
pd.to_datetime(df['Date'])

0      2020-06-16 00:00:00+00:00
1      2020-06-17 00:00:00+00:00
2      2020-06-17 00:00:00+00:00
3      2020-06-17 00:00:00+00:00
4      2020-06-17 00:00:00+00:00
                  ...           
996    2020-06-17 00:00:00+00:00
997    2020-06-17 00:00:00+00:00
998    2020-06-17 00:00:00+00:00
999    2020-06-17 00:00:00+00:00
1000   2020-06-17 00:00:00+00:00
Name: Date, Length: 1001, dtype: datetime64[ns, UTC]

In [1118]:
df['Date'] = pd.to_datetime(df['Date']).dt.date

In [1119]:
df['Date'] = df['Date'].astype(str)

In [1121]:
#df['ViewData.Status'].value_counts()

In [1122]:
df = df[df['ViewData.Status'].isin(['OB','SDB','UOB','UDB','CMF','CNF','SMB'])]
df = df.reset_index()
df = df.drop('index',1)

In [1123]:
#df1[df1['ViewData.Status']=='SMB']
df['ViewData.Status'].value_counts()

OB     687
SDB     25
SMB     20
UOB     15
UDB     10
CNF      1
Name: ViewData.Status, dtype: int64

In [1124]:
df['ViewData.Side0_UniqueIds'] = df['ViewData.Side0_UniqueIds'].astype(str)
df['ViewData.Side1_UniqueIds'] = df['ViewData.Side1_UniqueIds'].astype(str)
df['flag_side0'] = df.apply(lambda x: len(x['ViewData.Side0_UniqueIds'].split(',')), axis=1)
df['flag_side1'] = df.apply(lambda x: len(x['ViewData.Side1_UniqueIds'].split(',')), axis=1)

In [1125]:
#df_170[(df_170['ViewData.Status']=='UMR')]

In [1127]:
#df['Date'].value_counts()

## Sample data on one date

In [1128]:
df = df.rename(columns= {'ViewData.Cust Net Amount':'ViewData.B-P Net Amount'})

In [1129]:
sample = df[df['Date'] =='2020-06-17']
sample = sample.reset_index()
sample = sample.drop('index',1)

In [1130]:
smb = sample[sample['ViewData.Status']=='SMB'].reset_index()
smb = smb.drop('index',1)

In [1134]:
smb_pb = smb.copy()
smb_acc = smb.copy()

In [1135]:
smb_pb['ViewData.Accounting Net Amount'] = np.nan
smb_pb['ViewData.Side0_UniqueIds'] = np.nan
smb_pb['ViewData.Status'] ='SMB-OB'

smb_acc['ViewData.B-P Net Amount'] = np.nan
smb_acc['ViewData.Side1_UniqueIds'] = np.nan
smb_acc['ViewData.Status'] ='SMB-OB'

In [1136]:
sample = sample[sample['ViewData.Status']!='SMB']
sample = sample.reset_index()
sample = sample.drop('index',1)

In [1137]:
sample.shape

(706, 69)

In [1138]:
sample = pd.concat([sample,smb_pb,smb_acc],axis=0)
sample = sample.reset_index()
sample = sample.drop('index',1)

In [1140]:
#sample['ViewData.Status'].value_counts()

In [1141]:
sample['ViewData.Side0_UniqueIds'] = sample['ViewData.Side0_UniqueIds'].astype(str)
sample['ViewData.Side1_UniqueIds'] = sample['ViewData.Side1_UniqueIds'].astype(str)

In [1142]:
sample.loc[sample['ViewData.Side0_UniqueIds']=='nan','flag_side0'] = 0
sample.loc[sample['ViewData.Side1_UniqueIds']=='nan','flag_side1'] = 0

In [1146]:
#sample['ViewData.Status'].value_counts()

In [1145]:
#sample['flag_side1'].value_counts()

In [1147]:
sample.loc[sample['ViewData.Side1_UniqueIds']=='nan','Trans_side'] = 'B_side'
sample.loc[sample['ViewData.Side0_UniqueIds']=='nan','Trans_side'] = 'A_side'

sample.loc[sample['Trans_side']=='A_side','ViewData.B-P Currency'] = sample.loc[sample['Trans_side']=='A_side','ViewData.Currency']
sample.loc[sample['Trans_side']=='B_side','ViewData.Accounting Currency'] = sample.loc[sample['Trans_side']=='B_side','ViewData.Currency'] 

sample['ViewData.B-P Currency'] = sample['ViewData.B-P Currency'].astype(str)
sample['ViewData.Accounting Currency'] = sample['ViewData.Accounting Currency'].astype(str)
sample['ViewData.Mapped Custodian Account'] = sample['ViewData.Mapped Custodian Account'].astype(str)
#sample['ViewData.Mapped Custodian Account'] = sample['ViewData.Mapped Custodian Account'].astype(str)
sample['filter_key'] = sample.apply(lambda x: x['ViewData.Mapped Custodian Account'] + x['ViewData.B-P Currency'] if x['Trans_side']=='A_side' else x['ViewData.Mapped Custodian Account'] + x['ViewData.Accounting Currency'], axis=1)


sample1 = sample[(sample['flag_side0']<=1) & (sample['flag_side1']<=1) & (sample['ViewData.Status'].isin(['OB','SPM','SDB','UDB','UOB','SMB-OB','CNF','CMF']))]

sample1 = sample1.reset_index()
sample1 = sample1.drop('index', 1)

In [1148]:
sample1['ViewData.BreakID'] = sample1['ViewData.BreakID'].astype(int)

In [1149]:
sample1 = sample1[sample1['ViewData.BreakID']!=-1]
sample1 = sample1.reset_index()
sample1 = sample1.drop('index',1)

In [1150]:
sample1 = sample1.sort_values(['ViewData.BreakID','Date'], ascending =[True, False])
sample1 = sample1.reset_index()
sample1 = sample1.drop('index',1)

In [1152]:
#sample1['ViewData.Status'].value_counts()

In [1154]:
aa = sample1[sample1['Trans_side']=='A_side']
bb = sample1[sample1['Trans_side']=='B_side']

In [1156]:
#bb['ViewData.Source Combination'].value_counts()

In [1157]:
aa['filter_key'] = aa['ViewData.Source Combination'].astype(str) + aa['ViewData.Mapped Custodian Account'].astype(str) + aa['ViewData.B-P Currency'].astype(str)

bb['filter_key'] = bb['ViewData.Source Combination'].astype(str) + bb['ViewData.Mapped Custodian Account'].astype(str) + bb['ViewData.Accounting Currency'].astype(str)

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1158]:
aa = aa.reset_index()
aa = aa.drop('index', 1)
bb = bb.reset_index()
bb = bb.drop('index', 1)

In [1159]:
#'ViewData.Side0_UniqueIds', 'ViewData.Side1_UniqueIds'
common_cols = ['ViewData.Accounting Net Amount', 'ViewData.Age',
'ViewData.Age WK', 'ViewData.Asset Type Category',
'ViewData.B-P Net Amount', 'ViewData.Base Net Amount','ViewData.CUSIP', 
 'ViewData.Cancel Amount',
       'ViewData.Cancel Flag',
#'ViewData.Commission',
        'ViewData.Currency', 'ViewData.Custodian',
       'ViewData.Custodian Account',
       'ViewData.Description','ViewData.Department', 'ViewData.ExpiryDate', 'ViewData.Fund',
       'ViewData.ISIN',
       'ViewData.Investment Type',
      # 'ViewData.Keys',
       'ViewData.Mapped Custodian Account',
       'ViewData.Net Amount Difference',
       'ViewData.Net Amount Difference Absolute',
        #'ViewData.OTE Ticker',
        'ViewData.Price',
       'ViewData.Prime Broker', 'ViewData.Quantity',
       'ViewData.SEDOL', 'ViewData.SPM ID', 'ViewData.Settle Date',
       
  #  'ViewData.Strike Price',
               'Date',
       'ViewData.Ticker', 'ViewData.Trade Date',
       'ViewData.Transaction Category',
       'ViewData.Transaction Type', 'ViewData.Underlying Cusip',
       'ViewData.Underlying ISIN',
       'ViewData.Underlying Sedol','filter_key','ViewData.Status','ViewData.BreakID',
              'ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','ViewData._ID']

In [1160]:
bb = bb[~bb['ViewData.Accounting Net Amount'].isnull()]
bb = bb.reset_index()
bb = bb.drop('index',1)

In [1161]:
bb['ViewData.Status'].value_counts()

OB        337
SMB-OB     20
SDB        10
UOB        10
UDB         2
CNF         1
Name: ViewData.Status, dtype: int64

In [1163]:
bb.shape

(380, 73)

In [1165]:
###################### loop m*n ###############################
from pandas import merge
from tqdm import tqdm

pool =[]
key_index =[]
training_df =[]

no_pair_ids = []
#max_rows = 5

for d in tqdm(aa['Date'].unique()):
    aa1 = aa.loc[aa['Date']==d,:][common_cols]
    bb1 = bb.loc[bb['Date']==d,:][common_cols]
    
    aa1 = aa1.reset_index()
    aa1 = aa1.drop('index',1)
    bb1 = bb1.reset_index()
    bb1 = bb1.drop('index', 1)
    
    bb1 = bb1.sort_values(by='filter_key',ascending =True)
    
    for key in (list(np.unique(np.array(list(aa1['filter_key'].values) + list(bb1['filter_key'].values))))):
        
        df1 = aa1[aa1['filter_key']==key]
        df2 = bb1[bb1['filter_key']==key]

        if df1.empty == False and df2.empty == False:
            #aa_df = pd.concat([aa1[aa1.index==i]]*repeat_num, ignore_index=True)
            #bb_df = bb1.loc[pool[len(pool)-1],:][common_cols].reset_index()
            #bb_df = bb_df.drop('index', 1)

            df1 = df1.rename(columns={'ViewData.BreakID':'ViewData.BreakID_A_side'})
            df2 = df2.rename(columns={'ViewData.BreakID':'ViewData.BreakID_B_side'})

            #dff  = pd.concat([aa[aa.index==i],bb.loc[pool[i],:][accounting_vars]],axis=1)

            df1 = df1.reset_index()
            df2 = df2.reset_index()
            df1 = df1.drop('index', 1)
            df2 = df2.drop('index', 1)

            df1.columns = ['SideA.' + x  for x in df1.columns] 
            df2.columns = ['SideB.' + x  for x in df2.columns]

            df1 = df1.rename(columns={'SideA.filter_key':'filter_key'})
            df2 = df2.rename(columns={'SideB.filter_key':'filter_key'})

            #dff = pd.concat([aa_df,bb_df],axis=1)
            dff = merge(df1, df2, on='filter_key')
            training_df.append(dff)
                #key_index.append(i)
            #else:
            #no_pair_ids.append([aa1[(aa1['filter_key']=='key') & (aa1['ViewData.Status'].isin(['OB','SDB']))]['ViewData.Side1_UniqueIds'].values[0]])
               # no_pair_ids.append(aa1[(aa1['filter_key']== key) & (aa1['ViewData.Status'].isin(['OB','SDB']))]['ViewData.Side1_UniqueIds'].values[0])
    
        else:
            no_pair_ids.append([aa1[(aa1['filter_key']==key) & (aa1['ViewData.Status'].isin(['OB','SDB']))]['ViewData.Side1_UniqueIds'].values])
            no_pair_ids.append([bb1[(bb1['filter_key']==key) & (bb1['ViewData.Status'].isin(['OB','SDB']))]['ViewData.Side0_UniqueIds'].values])
            

no_pair_ids = np.unique(np.concatenate(no_pair_ids,axis=1)[0])


100%|████████████████████████████████████████████| 1/1 [00:03<00:00,  3.22s/it]


In [1166]:
#no_pair_ids = np.unique(np.concatenate(no_pair_ids,axis=1)[0])

In [1167]:
#pd.DataFrame(no_pair_ids).rename

In [1168]:
len(no_pair_ids)

171

In [1169]:
#test_file['SideA.ViewData.Status'].value_counts()

In [1170]:
test_file = pd.concat(training_df)

In [1171]:
test_file = test_file.reset_index()
test_file = test_file.drop('index',1)

In [1172]:
test_file['SideB.ViewData.BreakID_B_side'] = test_file['SideB.ViewData.BreakID_B_side'].astype('int64')
test_file['SideA.ViewData.BreakID_A_side'] = test_file['SideA.ViewData.BreakID_A_side'].astype('int64')

In [1173]:
test_file['SideB.ViewData.CUSIP'] = test_file['SideB.ViewData.CUSIP'].str.split(".",expand=True)[0]
test_file['SideA.ViewData.CUSIP'] = test_file['SideA.ViewData.CUSIP'].str.split(".",expand=True)[0]

In [1174]:
test_file['SideA.ViewData.ISIN'] = test_file['SideA.ViewData.ISIN'].astype(str)
test_file['SideB.ViewData.ISIN'] = test_file['SideB.ViewData.ISIN'].astype(str)
test_file['SideA.ViewData.CUSIP'] = test_file['SideA.ViewData.CUSIP'].astype(str)
test_file['SideB.ViewData.CUSIP'] = test_file['SideB.ViewData.CUSIP'].astype(str)
test_file['SideA.ViewData.Currency'] = test_file['SideA.ViewData.Currency'].astype(str)
test_file['SideB.ViewData.Currency'] = test_file['SideB.ViewData.Currency'].astype(str)


test_file['SideA.ViewData.Trade Date'] = test_file['SideA.ViewData.Trade Date'].astype(str)
test_file['SideB.ViewData.Trade Date'] = test_file['SideB.ViewData.Trade Date'].astype(str)
test_file['SideA.ViewData.Settle Date'] = test_file['SideA.ViewData.Settle Date'].astype(str)
test_file['SideB.ViewData.Settle Date'] = test_file['SideB.ViewData.Settle Date'].astype(str)
test_file['SideA.ViewData.Fund'] = test_file['SideA.ViewData.Fund'].astype(str)
test_file['SideB.ViewData.Fund'] = test_file['SideB.ViewData.Fund'].astype(str)

In [1175]:
#test_file[['SideA.ViewData.ISIN','SideB.ViewData.ISIN']]

def equals_fun(a,b):
    if a == b:
        return 1
    else:
        return 0

vec_equals_fun = np.vectorize(equals_fun)
values_ISIN_A_Side = test_file['SideA.ViewData.ISIN'].values
values_ISIN_B_Side = test_file['SideB.ViewData.ISIN'].values
#test_file['ISIN_match'] = vec_equals_fun(values_ISIN_A_Side,values_ISIN_B_Side)

values_CUSIP_A_Side = test_file['SideA.ViewData.CUSIP'].values
values_CUSIP_B_Side = test_file['SideB.ViewData.CUSIP'].values
#
# values_CUSIP_A_Side = test_file['SideA.ViewData.Currency'].values
# values_CUSIP_B_Side = test_file['SideB.ViewData.Currency'].values

values_Currency_match_A_Side = test_file['SideA.ViewData.Currency'].values
values_Currency_match_B_Side = test_file['SideA.ViewData.Currency'].values

values_Trade_Date_match_A_Side = test_file['SideA.ViewData.Trade Date'].values
values_Trade_Date_match_B_Side = test_file['SideB.ViewData.Trade Date'].values

values_Settle_Date_match_A_Side = test_file['SideA.ViewData.Settle Date'].values
values_Settle_Date_match_B_Side = test_file['SideB.ViewData.Settle Date'].values

values_Fund_match_A_Side = test_file['SideA.ViewData.Fund'].values
values_Fund_match_B_Side = test_file['SideB.ViewData.Fund'].values

test_file['ISIN_match'] = vec_equals_fun(values_ISIN_A_Side,values_ISIN_B_Side)
test_file['CUSIP_match'] = vec_equals_fun(values_CUSIP_A_Side,values_CUSIP_B_Side)
test_file['Currency_match'] = vec_equals_fun(values_Currency_match_A_Side,values_Currency_match_B_Side)
test_file['Trade_Date_match'] = vec_equals_fun(values_Trade_Date_match_A_Side,values_Trade_Date_match_B_Side)
test_file['Settle_Date_match'] = vec_equals_fun(values_Settle_Date_match_A_Side,values_Settle_Date_match_B_Side)
test_file['Fund_match'] = vec_equals_fun(values_Fund_match_A_Side,values_Fund_match_B_Side)


In [1176]:
#test_file['ISIN_match'] = test_file.apply(lambda x: 1 if x['SideA.ViewData.ISIN']==x['SideB.ViewData.ISIN'] else 0, axis=1)
#test_file['CUSIP_match'] = test_file.apply(lambda x: 1 if x['SideA.ViewData.CUSIP']==x['SideB.ViewData.CUSIP'] else 0, axis=1)
#test_file['Currency_match'] = test_file.apply(lambda x: 1 if x['SideA.ViewData.Currency']==x['SideB.ViewData.Currency'] else 0, axis=1)

In [1177]:
#test_file['Trade_Date_match'] = test_file.apply(lambda x: 1 if x['SideA.ViewData.Trade Date']==x['SideB.ViewData.Trade Date'] else 0, axis=1)
#test_file['Settle_Date_match'] = test_file.apply(lambda x: 1 if x['SideA.ViewData.Settle Date']==x['SideB.ViewData.Settle Date'] else 0, axis=1)
#test_file['Fund_match'] = test_file.apply(lambda x: 1 if x['SideA.ViewData.Fund']==x['SideB.ViewData.Fund'] else 0, axis=1)

In [1178]:
test_file['Amount_diff_1'] = test_file['SideA.ViewData.Accounting Net Amount'] - test_file['SideB.ViewData.B-P Net Amount']
test_file['Amount_diff_2'] = test_file['SideB.ViewData.Accounting Net Amount'] - test_file['SideA.ViewData.B-P Net Amount']

## Description code

In [1184]:
os.chdir('C:\\Users\\consultant136\\ML1.0')

In [1185]:
print(os.getcwd())

C:\Users\consultant136\ML1.0


In [1187]:
## TODO - Import a csv file for description category mapping

com = pd.read_csv('desc cat with naveen oaktree.csv')
#com

In [1188]:
cat_list = list(set(com['Pairing']))

In [1189]:
import re

def descclean(com,cat_list):
    cat_all1 = []
    list1 = cat_list
    m = 0
    if (type(com) == str):
        com = com.lower()
        com1 =  re.split("[,/. \-!?:]+", com)
        
        
        
        for item in list1:
            if (type(item) == str):
                item = item.lower()
                item1 = item.split(' ')
                lst3 = [value for value in item1 if value in com1] 
                if len(lst3) == len(item1):
                    cat_all1.append(item)
                    m = m+1
            
                else:
                    m = m
            else:
                    m = 0
    else:
        m = 0
    

            
    if m >0 :
        return list(set(cat_all1))
    else:
        if ((type(com)==str)):
            if (len(com1)<4):
                if ((len(com1)==1) & com1[0].startswith('20')== True):
                    return 'swap id'
                else:
                    return com
            else:
                return 'NA'
        else:
            return 'NA'

In [1190]:
#df3['desc_cat'] = df3['ViewData.Description'].apply(lambda x : descclean(x,cat_list))

test_file['SideA.desc_cat'] = test_file['SideA.ViewData.Description'].apply(lambda x : descclean(x,cat_list))
test_file['SideB.desc_cat'] = test_file['SideB.ViewData.Description'].apply(lambda x : descclean(x,cat_list))

In [1191]:
def currcln(x):
    if (type(x)==list):
        return x
      
    else:
       
        
        if x == 'NA':
            return "NA"
        elif (('dollar' in x) | ('dollars' in x )):
            return 'dollar'
        elif (('pound' in x) | ('pounds' in x)):
            return 'pound'
        elif ('yen' in x):
            return 'yen'
        elif ('euro' in x) :
            return 'euro'
        else:
            return x
        

In [1192]:

#df3['desc_cat'] = df3['desc_cat'].apply(lambda x : currcln(x))

test_file['SideA.desc_cat'] = test_file['SideA.desc_cat'].apply(lambda x : currcln(x))
test_file['SideB.desc_cat'] = test_file['SideB.desc_cat'].apply(lambda x : currcln(x))


In [1193]:
com = com.drop(['var','Catogery'], axis = 1)

com = com.drop_duplicates()

com['Pairing'] = com['Pairing'].apply(lambda x : x.lower())
com['replace'] = com['replace'].apply(lambda x : x.lower())

In [1194]:
def catcln1(cat,df):
    ret = []
    if (type(cat)==list):
        
        if 'equity swap settlement' in cat:
            ret.append('equity swap settlement')
        #return 'equity swap settlement'
        elif 'equity swap' in cat:
            ret.append('equity swap settlement')
        #return 'equity swap settlement'
        elif 'swap settlement' in cat:
            ret.append('equity swap settlement')
        #return 'equity swap settlement'
        elif 'swap unwind' in cat:
            ret.append('swap unwind')
        #return 'swap unwind'
   
    
    
    
        else:
        
       
            for item in cat:
            
                a = df[df['Pairing']==item]['replace'].values[0]
                if a not in ret:
                    ret.append(a)
        return list(set(ret))
      
    else:
        return cat



In [1195]:
test_file['SideA.new_desc_cat'] = test_file['SideA.desc_cat'].apply(lambda x : catcln1(x,com))
test_file['SideB.new_desc_cat'] = test_file['SideB.desc_cat'].apply(lambda x : catcln1(x,com))

In [1196]:
comp = ['inc','stk','corp ','llc','pvt','plc']
#df3['new_desc_cat'] = df3['new_desc_cat'].apply(lambda x : 'Company' if x in comp else x)

test_file['SideA.new_desc_cat'] = test_file['SideA.new_desc_cat'].apply(lambda x : 'Company' if x in comp else x)

test_file['SideB.new_desc_cat'] = test_file['SideB.new_desc_cat'].apply(lambda x : 'Company' if x in comp else x)


In [1197]:
#df3['new_desc_cat'] = df3['desc_cat'].apply(lambda x : catcln1(x,com))

def desccat(x):
    if isinstance(x, list):
        
        if 'equity swap settlement' in x:
            return 'swap settlement'
        elif 'collateral transfer' in x:
            return 'collateral transfer'
        elif 'dividend' in x:
            return 'dividend'
        elif (('loan' in x) & ('option' in x)):
            return 'option loan'
        
        elif (('interest' in x) & ('corp' in x) ):
            return 'corp loan'
        elif (('interest' in x) & ('loan' in x) ):
            return 'interest'
        else:
            return x[0]
    else:
        return x



In [1198]:
#df3['new_desc_cat'] = df3['new_desc_cat'].apply(lambda x : desccat(x))

test_file['SideA.new_desc_cat'] = test_file['SideA.new_desc_cat'].apply(lambda x : desccat(x))
test_file['SideB.new_desc_cat'] = test_file['SideB.new_desc_cat'].apply(lambda x : desccat(x))


In [1200]:
#test_file['SideB.new_desc_cat'].value_counts()

## Prime Broker

In [1201]:
test_file['new_pb'] = test_file['SideA.ViewData.Mapped Custodian Account'].apply(lambda x : x.split('_')[0] if type(x)==str else x)

In [1202]:
new_pb_mapping = {'GSIL':'GS','CITIGM':'CITI','JPMNA':'JPM'}

In [1203]:
def new_pf_mapping(x):
    if x=='GSIL':
        return 'GS'
    elif x == 'CITIGM':
        return 'CITI'
    elif x == 'JPMNA':
        return 'JPM'
    else:
        return x

In [1204]:
test_file['SideA.ViewData.Prime Broker'] = test_file['SideA.ViewData.Prime Broker'].fillna('kkk')

In [1205]:
test_file['new_pb1'] = test_file.apply(lambda x : x['new_pb'] if x['SideA.ViewData.Prime Broker']=='kkk' else x['SideA.ViewData.Prime Broker'],axis = 1)

In [1206]:
#test_file = pd.read_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/OakTree/X_test_files_after_loop/meo_testing_HST_RecData_379_06_19_2020_test_file_with_ID.csv')

In [1207]:
#test_file = test_file.drop('Unnamed: 0',1)

In [1208]:
test_file['Trade_date_diff'] = (pd.to_datetime(test_file['SideA.ViewData.Trade Date']) - pd.to_datetime(test_file['SideB.ViewData.Trade Date'])).dt.days

test_file['Settle_date_diff'] = (pd.to_datetime(test_file['SideA.ViewData.Settle Date']) - pd.to_datetime(test_file['SideB.ViewData.Settle Date'])).dt.days

In [1209]:
def mhreplaced(item):
    word1 = []
    word2 = []
    if (type(item) == str):
    
        for items in item.split(' '):
            if (type(items) == str):
                items = items.lower()
                if items.isdigit() == False:
                    word1.append(items)
        
            
                for c in word1:
                    if c.endswith('MH')==False:
                        word2.append(c)
    
                words = ' '.join(word2)
                return words
    else:
        return item
    

def fundmatch(item):
    items = item.lower()
    items = item.replace(' ','') 
    return items

In [1210]:

############ Fund match new ########

values_Fund_match_A_Side = test_file['SideA.ViewData.Fund'].values
values_Fund_match_B_Side = test_file['SideB.ViewData.Fund'].values

vec_fund_match = np.vectorize(fundmatch)

#test_file['ISIN_match'] = vec_(values_ISIN_A_Side,values_ISIN_B_Side)
#test_file['SideA.ViewData.Fund'] = test_file.apply(lambda x : fundmatch(x['SideA.ViewData.Fund']), axis=1)
test_file['SideA.ViewData.Fund'] = vec_fund_match(values_Fund_match_A_Side)
#test_file['SideB.ViewData.Fund'] = test_file.apply(lambda x : fundmatch(x['SideB.ViewData.Fund']), axis=1)
test_file['SideB.ViewData.Fund'] = vec_fund_match(values_Fund_match_B_Side)

In [1211]:
### New code for cleaning text variables 

import pandas as pd
import dask.dataframe as dd
import glob
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from dateutil.parser import parse
import operator
import itertools
from sklearn.feature_extraction.text import CountVectorizer
import re
import os
import numpy as np


#column_names = ['SideA.ViewData.Transaction Type', 'ViewData.Investment Type', 'ViewData.Asset Type Category', 'ViewData.Prime Broker', 'ViewData.Description']

trans_type_A_side = test_file['SideA.ViewData.Transaction Type']
trans_type_B_side = test_file['SideB.ViewData.Transaction Type']

asset_type_cat_A_side = test_file['SideA.ViewData.Asset Type Category']
asset_type_cat_B_side = test_file['SideB.ViewData.Asset Type Category']

invest_type_A_side = test_file['SideA.ViewData.Investment Type']
invest_type_B_side = test_file['SideB.ViewData.Investment Type']

prime_broker_A_side = test_file['SideA.ViewData.Prime Broker']
prime_broker_B_side = test_file['SideB.ViewData.Prime Broker']

In [1212]:
# LOWER CASE
trans_type_A_side = [str(item).lower() for item in trans_type_A_side]
trans_type_B_side = [str(item).lower() for item in trans_type_B_side]

asset_type_cat_A_side = [str(item).lower() for item in asset_type_cat_A_side]
asset_type_cat_B_side = [str(item).lower() for item in asset_type_cat_B_side]

invest_type_A_side = [str(item).lower() for item in invest_type_A_side]
invest_type_B_side = [str(item).lower() for item in invest_type_B_side]

prime_broker_A_side = [str(item).lower() for item in prime_broker_A_side]
prime_broker_B_side = [str(item).lower() for item in prime_broker_B_side]



In [1213]:
split_trans_A_side = [item.split() for item in trans_type_A_side]
split_trans_B_side = [item.split() for item in trans_type_B_side]


split_asset_A_side = [item.split() for item in asset_type_cat_A_side]
split_asset_B_side = [item.split() for item in asset_type_cat_B_side]


split_invest_A_side = [item.split() for item in invest_type_A_side]
split_invest_B_side = [item.split() for item in invest_type_B_side]

split_prime_A_side = [item.split() for item in prime_broker_A_side]
split_prime_b_side = [item.split() for item in prime_broker_B_side]

# In[310]:


def is_num(item):
    try:
        float(item)
        return True
    except ValueError:
        return False

def is_date_format(item):
    try:
        parse(item, fuzzy=False)
        return True
    
    except ValueError:
        return False
    
def date_edge_cases(item):
    if len(item) == 5 and item[2] =='/' and is_num(item[:2]) and is_num(item[3:]):
        return True
    return False

In [1214]:
## Transacion type

remove_nums_A_side = [[item for item in sublist if not is_num(item)] for sublist in split_trans_A_side]
remove_nums_B_side = [[item for item in sublist if not is_num(item)] for sublist in split_trans_B_side]

#remove_dates_A_side = [[item for item in sublist if not is_date(item)] for sublist in remove_nums_A_side]

#remove_dates_B_side = [[item for item in sublist if not is_date(item)] for sublist in remove_nums_B_side]

remove_dates_A_side = [[item for item in sublist if not (is_date_format(item) or date_edge_cases(item))] for sublist in remove_nums_A_side]
remove_dates_B_side = [[item for item in sublist if not (is_date_format(item) or date_edge_cases(item))] for sublist in remove_nums_B_side]


# Specific to clients already used on, will have to be edited for other edge cases
remove_amts_A_side = [[item for item in sublist if item[0] != '$'] for sublist in remove_dates_A_side]
remove_amts_B_side = [[item for item in sublist if item[0] != '$'] for sublist in remove_dates_B_side]


clean_adr_A_side = [(['ADR'] if 'adr' in item else item) for item in remove_amts_A_side]
clean_adr_B_side = [(['ADR'] if 'adr' in item else item) for item in remove_amts_B_side]

clean_tax_A_side = [(item[:2] if '30%' in item else item) for item in clean_adr_A_side]
clean_tax_B_side = [(item[:2] if '30%' in item else item) for item in clean_adr_B_side]

remove_ons_A_side = [(item[:item.index('on')] if 'on' in item else item) for item in clean_tax_A_side]
remove_ons_B_side = [(item[:item.index('on')] if 'on' in item else item) for item in clean_tax_B_side]

clean_eqswap_A_side = [(item[1:] if 'eqswap' in item else item) for item in remove_ons_A_side]
clean_eqswap_B_side = [(item[1:] if 'eqswap' in item else item) for item in remove_ons_B_side]

remove_mh_A_side = [[item for item in sublist if 'mh' not in item] for sublist in clean_eqswap_A_side]
remove_mh_B_side = [[item for item in sublist if 'mh' not in item] for sublist in clean_eqswap_B_side]

remove_ats_A_side = [(item[:item.index('@')] if '@' in item else item) for item in remove_mh_A_side]
remove_ats_B_side = [(item[:item.index('@')] if '@' in item else item) for item in remove_mh_B_side]

#remove_blanks_A_side = [item for item in remove_ats_A_side if item]
#remove_blanks_B_side = [item for item in remove_ats_B_side if item]


# In[319]:

cleaned_trans_types_A_side = [' '.join(item) for item in remove_ats_A_side]
cleaned_trans_types_B_side = [' '.join(item) for item in remove_ats_B_side]

In [1215]:
# # INVESTMENT TYPE

# # In[322]:


remove_nums_i_A_side = [[item for item in sublist if not is_num(item)] for sublist in split_invest_A_side]
remove_nums_i_B_side = [[item for item in sublist if not is_num(item)] for sublist in split_invest_B_side]

remove_dates_i_A_side = [[item for item in sublist if not is_date_format(item)] for sublist in remove_nums_i_A_side]
remove_dates_i_B_side = [[item for item in sublist if not is_date_format(item)] for sublist in remove_nums_i_B_side]

#remove_blanks_i_A_side = [item for item in remove_dates_i_A_side if item]
#remove_blanks_i_B_side = [item for item in remove_dates_i_B_side if item]
#remove_blanks_i[:10]


# # In[323]:

cleaned_invest_A_side = [' '.join(item) for item in remove_dates_i_A_side]
cleaned_invest_B_side = [' '.join(item) for item in remove_dates_i_B_side]

In [1216]:

remove_nums_a_A_side = [[item for item in sublist if not is_num(item)] for sublist in split_asset_A_side]
remove_nums_a_B_side = [[item for item in sublist if not is_num(item)] for sublist in split_asset_B_side]

remove_dates_a_A_side = [[item for item in sublist if not is_date_format(item)] for sublist in remove_nums_a_A_side]
remove_dates_a_B_side = [[item for item in sublist if not is_date_format(item)] for sublist in remove_nums_a_B_side]
# remove_blanks_a = [item for item in remove_dates_a if item]
# # remove_blanks_a[:10]


# # In[321]:

cleaned_asset_A_side = [' '.join(item) for item in remove_dates_a_A_side]
cleaned_asset_B_side = [' '.join(item) for item in remove_dates_a_B_side]

In [1217]:
test_file['SideA.ViewData.Transaction Type'] = cleaned_trans_types_A_side
test_file['SideB.ViewData.Transaction Type'] = cleaned_trans_types_B_side

test_file['SideA.ViewData.Investment Type'] = cleaned_invest_A_side
test_file['SideB.ViewData.Investment Type'] = cleaned_invest_B_side

test_file['SideA.ViewData.Asset Category Type'] = cleaned_asset_A_side
test_file['SideB.ViewData.Asset Category Type'] = cleaned_asset_B_side

In [1218]:
#test_file['SideB.ViewData.Transaction Type'] = test_file['SideB.ViewData.Transaction Type'].apply(lambda x : mhreplaced(x))

In [1219]:
#test_file['SideA.ViewData.Transaction Type'] = test_file['SideA.ViewData.Transaction Type'].apply(lambda x : mhreplaced(x))

In [1220]:
##############

values_transaction_type_match_A_Side = test_file['SideA.ViewData.Transaction Type'].values
values_transaction_type_match_B_Side = test_file['SideB.ViewData.Transaction Type'].values

vec_tt_match = np.vectorize(mhreplaced)

#test_file['ISIN_match'] = vec_(values_ISIN_A_Side,values_ISIN_B_Side)
#test_file['SideA.ViewData.Fund'] = test_file.apply(lambda x : fundmatch(x['SideA.ViewData.Fund']), axis=1)
test_file['SideA.ViewData.Transaction Type'] = vec_tt_match(values_transaction_type_match_A_Side)
#test_file['SideB.ViewData.Fund'] = test_file.apply(lambda x : fundmatch(x['SideB.ViewData.Fund']), axis=1)
test_file['SideB.ViewData.Transaction Type'] = vec_tt_match(values_transaction_type_match_B_Side)

In [1221]:
test_file.loc[test_file['SideA.ViewData.Transaction Type']=='int','SideA.ViewData.Transaction Type'] = 'interest'
test_file.loc[test_file['SideA.ViewData.Transaction Type']=='wires','SideA.ViewData.Transaction Type'] = 'wire'
test_file.loc[test_file['SideA.ViewData.Transaction Type']=='dividends','SideA.ViewData.Transaction Type'] = 'dividend'
test_file.loc[test_file['SideA.ViewData.Transaction Type']=='miscellaneous','SideA.ViewData.Transaction Type'] = 'misc'
test_file.loc[test_file['SideA.ViewData.Transaction Type']=='div','SideA.ViewData.Transaction Type'] = 'dividend'

In [1222]:
test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: x.replace('eqty','equity'))
test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: x.replace('options','option'))
test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: x.replace('eqt','equity'))
test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: x.replace('eqty','equity'))

In [1223]:
test_file['ViewData.Combined Transaction Type'] = test_file['SideA.ViewData.Transaction Type'].astype(str) +  test_file['SideB.ViewData.Transaction Type'].astype(str)

In [1224]:
#train_full_new1['ViewData.Combined Transaction Type'] = train_full_new1['SideA.ViewData.Transaction Type'].astype(str) + train_full_new1['SideB.ViewData.Transaction Type'].astype(str)
test_file['ViewData.Combined Fund'] = test_file['SideA.ViewData.Fund'].astype(str) + test_file['SideB.ViewData.Fund'].astype(str)

In [1225]:
test_file['Combined_Investment_Type'] = test_file['SideA.ViewData.Investment Type'].astype(str) + test_file['SideB.ViewData.Investment Type'].astype(str)

In [1226]:
test_file['Combined_Asset_Type_Category'] = test_file['SideA.ViewData.Asset Category Type'].astype(str) + test_file['SideB.ViewData.Asset Category Type'].astype(str)

In [1227]:
def nan_fun(x):
    if x=='nan':
        return 1
    else:
        return 0
    
vec_nan_fun = np.vectorize(nan_fun)
values_ISIN_A_Side = test_file['SideA.ViewData.ISIN'].values
values_ISIN_B_Side = test_file['SideB.ViewData.ISIN'].values
test_file['SideA.ISIN_NA'] = vec_nan_fun(values_ISIN_A_Side)
test_file['SideB.ISIN_NA'] = vec_nan_fun(values_ISIN_A_Side)

#test_file['SideA.ISIN_NA'] =  test_file.apply(lambda x: 1 if x['SideA.ViewData.ISIN']=='nan' else 0, axis=1)
#test_file['SideB.ISIN_NA'] =  test_file.apply(lambda x: 1 if x['SideB.ViewData.ISIN']=='nan' else 0, axis=1)

In [1228]:
len(test_file['SideB.ViewData.CUSIP'].values)

1477

In [1229]:
def a_keymatch(a_cusip, a_isin):
    
    pb_nan = 0
    a_common_key = 'NA' 
    if a_cusip=='nan' and a_isin =='nan':
        pb_nan =1
    elif(a_cusip!='nan' and a_isin == 'nan'):
        a_common_key = a_cusip
    elif(a_cusip =='nan' and a_isin !='nan'):
        a_common_key = a_isin
    else:
        a_common_key = a_isin
        
    return (pb_nan, a_common_key)

def b_keymatch(b_cusip, b_isin):
    accounting_nan = 0
    b_common_key = 'NA'
    if b_cusip =='nan' and b_isin =='nan':
        accounting_nan =1
    elif (b_cusip!='nan' and b_isin == 'nan'):
        b_common_key = b_cusip
    elif(b_cusip =='nan' and b_isin !='nan'):
        b_common_key = b_isin
    else:
        b_common_key = b_isin
    return (accounting_nan, b_common_key)

    
vec_a_key_match_fun = np.vectorize(a_keymatch)
vec_b_key_match_fun = np.vectorize(b_keymatch)

values_ISIN_A_Side = test_file['SideA.ViewData.ISIN'].values
values_ISIN_B_Side = test_file['SideB.ViewData.ISIN'].values

values_CUSIP_A_Side = test_file['SideA.ViewData.CUSIP'].values
values_CUSIP_B_Side = test_file['SideB.ViewData.CUSIP'].values

test_file['SideB.ViewData.key_NAN']= vec_a_key_match_fun(values_CUSIP_B_Side,values_ISIN_B_Side)[0]
test_file['SideB.ViewData.Common_key'] = vec_a_key_match_fun(values_CUSIP_B_Side,values_ISIN_B_Side)[1]
test_file['SideA.ViewData.key_NAN'] = vec_b_key_match_fun(values_CUSIP_A_Side,values_ISIN_A_Side)[0]
test_file['SideA.ViewData.Common_key'] = vec_b_key_match_fun(values_CUSIP_A_Side,values_ISIN_A_Side)[1]


In [1230]:
#test_file[['SideB.ViewData.key_NAN','SideB.ViewData.Common_key']] = test_file.apply(lambda x: b_keymatch(x['SideB.ViewData.CUSIP'], x['SideB.ViewData.ISIN']), axis=1)
#test_file[['SideA.ViewData.key_NAN','SideA.ViewData.Common_key']] = test_file.apply(lambda x: a_keymatch(x['SideA.ViewData.CUSIP'],x['SideA.ViewData.ISIN']), axis=1)

In [1231]:
def nan_equals_fun(a,b):
    if a==1 and b==1:
        return 1
    else:
        return 0
    
vec_nan_equal_fun = np.vectorize(nan_equals_fun)
values_key_NAN_B_Side = test_file['SideB.ViewData.key_NAN'].values
values_key_NAN_A_Side = test_file['SideA.ViewData.key_NAN'].values
test_file['All_key_nan'] = vec_nan_equal_fun(values_key_NAN_B_Side,values_key_NAN_A_Side )

#test_file['All_key_nan'] = test_file.apply(lambda x: 1 if x['SideB.ViewData.key_NAN']==1 and x['SideA.ViewData.key_NAN']==1 else 0, axis=1)


In [1232]:
test_file['SideB.ViewData.Common_key'] = test_file['SideB.ViewData.Common_key'].astype(str)
test_file['SideA.ViewData.Common_key'] = test_file['SideA.ViewData.Common_key'].astype(str)


def new_key_match_fun(a,b,c):
    if a==b and c==0:
        return 1
    else:
        return 0
    
vec_new_key_match_fun = np.vectorize(new_key_match_fun)
values_Common_key_B_Side = test_file['SideB.ViewData.Common_key'].values
values_Common_key_A_Side = test_file['SideA.ViewData.Common_key'].values
values_All_key_NAN = test_file['All_key_nan'].values

test_file['new_key_match']= vec_new_key_match_fun(values_Common_key_B_Side,values_Common_key_A_Side,values_All_key_NAN)

In [1233]:
test_file['amount_percent'] = (test_file['SideA.ViewData.B-P Net Amount']/test_file['SideB.ViewData.Accounting Net Amount']*100)

In [1234]:
test_file['SideB.ViewData.Investment Type'] = test_file['SideB.ViewData.Investment Type'].apply(lambda x: str(x).lower())
test_file['SideA.ViewData.Investment Type'] = test_file['SideA.ViewData.Investment Type'].apply(lambda x: str(x).lower())

In [1235]:
test_file['SideB.ViewData.Prime Broker'] = test_file['SideB.ViewData.Prime Broker'].apply(lambda x: str(x).lower())
test_file['SideA.ViewData.Prime Broker'] = test_file['SideA.ViewData.Prime Broker'].apply(lambda x: str(x).lower())

In [1236]:
test_file['SideB.ViewData.Asset Type Category'] = test_file['SideB.ViewData.Asset Type Category'].apply(lambda x: str(x).lower())
test_file['SideA.ViewData.Asset Type Category'] = test_file['SideA.ViewData.Asset Type Category'].apply(lambda x: str(x).lower())

In [1237]:
#test_file

test_file['ViewData.Combined Transaction Type'] = test_file['ViewData.Combined Transaction Type'].apply(lambda x: x.replace('jnl','journal'))

In [1238]:

#test_file['new_key_match'] = test_file.apply(lambda x: 1 if x['SideB.ViewData.Common_key']==x['SideA.ViewData.Common_key'] and x['All_key_nan']==0 else 0, axis=1)

In [1239]:
#test_file.to_csv("//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/X_test_files_after_loop/meo_testing_HST_RecData_170_06-18-2020_test_file.csv")

In [1240]:
test_file['SideA.ViewData.Transaction Type'] = test_file['SideA.ViewData.Transaction Type'].apply(lambda x: x.replace('cover short','covershort'))

In [1241]:
trade_types_A = ['buy', 'sell', 'covershort','sellshort',
       'fx', 'fx settlement', 'sell short',
       'trade not to be reported_buy', 'covershort','ptbl','ptss', 'ptcs', 'ptcl']
trade_types_B = ['trade not to be reported_buy','buy', 'sellshort', 'sell', 'covershort',
       'spotfx', 'forwardfx',
       'trade not to be reported_sell',
       'trade not to be reported_sellshort',
       'trade not to be reported_covershort']

In [1242]:
test_file['SideA.TType'] = test_file.apply(lambda x: "Trade" if x['SideA.ViewData.Transaction Type'] in trade_types_A else "Non-Trade", axis=1)
test_file['SideB.TType'] = test_file.apply(lambda x: "Trade" if x['SideB.ViewData.Transaction Type'] in trade_types_B else "Non-Trade", axis=1)

In [1243]:
test_file['Combined_Desc'] = test_file['SideA.new_desc_cat'] + test_file['SideB.new_desc_cat']

In [1244]:
test_file['Combined_TType'] = test_file['SideA.TType'].astype(str) + test_file['SideB.TType'].astype(str)

In [1245]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np

#le = LabelEncoder()
for feature in ['SideA.Date','SideB.Date','SideA.ViewData.Settle Date','SideB.ViewData.Settle Date']:
    #train_full_new12[feature] = le.fit_transform(train_full_new12[feature])
    test_file[feature] = pd.to_datetime(test_file[feature],errors = 'coerce').dt.weekday

In [1247]:
model_cols = [
            'SideA.ViewData.B-P Net Amount', 
              #'SideA.ViewData.Cancel Flag', 
              #'SideA.new_desc_cat',
             # 'SideA.ViewData.Description',
             # 'SideA.ViewData.Department',
   
    
              
             # 'SideA.ViewData.Price',
             # 'SideA.ViewData.Quantity',
             #'SideA.ViewData.Investment Type', 
              #'SideA.ViewData.Asset Type Category', 
              'SideB.ViewData.Accounting Net Amount', 
              #'SideB.ViewData.Cancel Flag', 
             # 'SideB.ViewData.Description',
              # 'SideB.ViewData.Department',
              
             # 'SideB.ViewData.Price',
             # 'SideB.ViewData.Quantity',
             # 'SideB.new_desc_cat',
             # 'SideB.ViewData.Investment Type', 
              #'SideB.ViewData.Asset Type Category', 
              'Trade_Date_match', 'Settle_Date_match', 
                'Amount_diff_2', 
              'Trade_date_diff', 'Settle_date_diff', 'SideA.ISIN_NA', 'SideB.ISIN_NA', 
             # 'ViewData.Combined Fund',
              'ViewData.Combined Transaction Type', 'Combined_Desc','Combined_TType',
             # 'SideA.TType', 'SideB.TType', 
              'abs_amount_flag',
    'tt_map_flag', 
              'All_key_nan','new_key_match', 'new_pb1',
              'SideB.Date','SideA.ViewData.Settle Date','SideB.ViewData.Settle Date',
            'SideA.ViewData._ID', 'SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds',
              'SideB.ViewData.Status', 'SideB.ViewData.BreakID_B_side',
              'SideA.ViewData.Status', 'SideA.ViewData.BreakID_A_side'] 
              #'label']

## UMR Mapping

In [1248]:

## TODO Import HIstorical UMR FILE for Transaction Type mapping

oaktree_umr = pd.read_csv('OakTree_UMR.csv')

In [1249]:
#soros_umr['ViewData.Combined Transaction Type'].unique()

In [1250]:
test_file['tt_map_flag'] = test_file.apply(lambda x: 1 if x['ViewData.Combined Transaction Type'] in oaktree_umr['ViewData.Combined Transaction Type'].unique() else 0, axis=1)

In [1251]:
test_file['abs_amount_flag'] = test_file.apply(lambda x: 1 if x['SideB.ViewData.Accounting Net Amount'] == x['SideA.ViewData.B-P Net Amount']*(-1) else 0, axis=1)

In [1252]:
test_file = test_file[~test_file['SideB.ViewData.Settle Date'].isnull()]
test_file = test_file[~test_file['SideA.ViewData.Settle Date'].isnull()]

test_file = test_file.reset_index().drop('index',1)
test_file['SideA.ViewData.Settle Date'] = test_file['SideA.ViewData.Settle Date'].astype(int)
test_file['SideB.ViewData.Settle Date'] = test_file['SideB.ViewData.Settle Date'].astype(int)

In [765]:
#test_file2 = test_file[((test_file['SideA.TType']=="Trade") & (test_file['SideB.TType']=="Trade")) | ((test_file['SideA.TType']!="Trade") & (test_file['SideB.TType']!="Trade")) ]

In [766]:
#test_file2 = test_file[(test_file['SideA.TType']=="Trade") & (test_file['SideB.TType']=="Trade")]

In [767]:
#test_file[(test_file['SideA.TType']==test_file['SideB.TType'])]['SideB.TType']

In [768]:
#test_file2 = test_file2.reset_index()
#test_file2 = test_file2.drop('index',1)

In [1256]:
#test_file['SideA.ViewData.BreakID_A_side'].value_counts()
#test_file[model_cols]

## Test file served into the model

In [1257]:
test_file2 = test_file.copy()

In [1258]:
X_test = test_file2[model_cols]

In [1259]:
X_test = X_test.reset_index()
X_test = X_test.drop('index',1)
X_test = X_test.fillna(0)

In [1260]:
X_test = X_test.fillna(0)

In [1261]:
X_test.shape

(1477, 28)

In [1262]:
X_test = X_test.drop_duplicates()
X_test = X_test.reset_index()
X_test = X_test.drop('index',1)

In [1263]:
X_test.shape

(1334, 28)

## Model Pickle file import

In [1266]:

## TODO Import Pickle file for 1st Model

import pickle

In [1265]:
#filename = 'Oak_W125_model_with_umb.sav'
#filename = '125_with_umb_without_des_and_many_to_many.sav'
#filename = '125_with_umb_and_price_without_des_and_many_to_many_tdsd2.sav'
#filename = 'Weiss_new_model_V1.sav'
#filename = 'Soros_new_model_V1_with_close.sav'
#filename = 'Soros_full_model_smote.sav'

#filename = 'Soros_full_model_best_cleaned_tt_without_date.sav'
#filename = 'Soros_full_model_version2.sav'
filename = 'OakTree_final_model2.sav'

#filename = 'Soros_full_model_umr_umt.sav'

clf = pickle.load(open(filename, 'rb'))

In [1267]:
X_test

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side
0,-6089038.67000,-6089038.67000,0,1,0.00000,2.00000,0,1,1,s304spotfx,NAdollar,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499b,5eeaf3841554582ac01b4973,1_379902880_Advent Geneva,5_379902880_BNP Paribas,OB,1340602390,OB,1340602429
1,6866000.00000,6866000.00000,0,1,0.00000,2.00000,0,1,1,s210spotfx,NAdollar,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b49a3,5eeaf3841554582ac01b4976,5_379902880_Advent Geneva,4_379902880_BNP Paribas,OB,1340602393,OB,1340602439
2,-2314.39000,-396193.87000,1,0,-393879.48000,0.00000,-2,1,1,ntrfspotfx,NApound,Non-TradeTrade,0,1,1,0,BNP,2,0,2,5eeaf3841554582ac01b49a2,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,78_379899248_BNP Paribas,OB,1340602394,OB,1336048325
3,-2314.39000,1326013.74000,1,0,1328328.13000,0.00000,-2,1,1,ntrfspotfx,NAeuro,Non-TradeTrade,0,1,1,0,BNP,2,0,2,5eeaf3841554582ac01b49a2,5eeaf3841554582ac01b4978,92_379902880_Advent Geneva,78_379899248_BNP Paribas,OB,1340602395,OB,1336048325
4,1326013.74000,-396193.87000,0,1,-1722207.61000,2.00000,0,1,1,s210spotfx,corp pound,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499d,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,20_379902880_BNP Paribas,OB,1340602394,OB,1340602430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,100.62000,496.04000,0,0,395.42000,0.00000,-57,1,1,interestinterest,interestinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,3,4,5eeaf37b15545830ac1ccabd,5eeaf37b15545830ac1ccabc,1_379879036_Advent Geneva,7_379805934_US BANK,OB,1308694502,OB,1214171472
1330,3706.27000,626.41000,0,0,-3079.86000,0.00000,-20,1,1,interestinterest,interestinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,4,3,5eeaf37b15545830ac1ccac4,5eeaf37b15545830ac1ccabb,1_379841544_Advent Geneva,1_379816267_US BANK,OB,1259461223,OB,1228154166
1331,3706.27000,496.04000,0,0,-3210.23000,0.00000,-49,1,1,interestinterest,interestinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,4,4,5eeaf37b15545830ac1ccac4,5eeaf37b15545830ac1ccabc,1_379879036_Advent Geneva,1_379816267_US BANK,OB,1308694502,OB,1228154166
1332,-591.94000,626.41000,0,0,1218.35000,0.00000,7,1,1,interest,NAinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,3,3,5eeaf37b15545830ac1ccac0,5eeaf37b15545830ac1ccabb,1_379841544_Advent Geneva,2_379850769_US BANK,OB,1259461223,OB,1271871929


## Predictions

In [1269]:
# Actual class predictions
rf_predictions = clf.predict(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))
# Probabilities for each class
rf_probs = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 1]

In [1270]:
probability_class_0 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side','SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 0]
probability_class_1 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 1]

probability_class_2 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side','SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 2]
#probability_class_3 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 3]

#probability_class_4 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 4]



In [1271]:
X_test['Predicted_action'] = rf_predictions
#X_test['Predicted_action_probabilty'] = rf_probs

#X_test['probability_Close'] = probability_class_0
X_test['probability_No_pair'] = probability_class_0
#X_test['probability_Partial_match'] = probability_class_0
#X_test['probability_UMB'] = probability_class_1
X_test['probability_UMB'] = probability_class_1
X_test['probability_UMR'] = probability_class_2
#X_test['probability_UMT'] = probability_class_3

In [1272]:
X_test['Predicted_action'].value_counts()

No-Pair           1161
UMB_One_to_One     139
UMR_One_to_One      34
Name: Predicted_action, dtype: int64

In [1273]:
X_test

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR
0,-6089038.67000,-6089038.67000,0,1,0.00000,2.00000,0,1,1,s304spotfx,NAdollar,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499b,5eeaf3841554582ac01b4973,1_379902880_Advent Geneva,5_379902880_BNP Paribas,OB,1340602390,OB,1340602429,UMR_One_to_One,0.00000,0.00009,0.99991
1,6866000.00000,6866000.00000,0,1,0.00000,2.00000,0,1,1,s210spotfx,NAdollar,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b49a3,5eeaf3841554582ac01b4976,5_379902880_Advent Geneva,4_379902880_BNP Paribas,OB,1340602393,OB,1340602439,UMR_One_to_One,0.00000,0.00012,0.99988
2,-2314.39000,-396193.87000,1,0,-393879.48000,0.00000,-2,1,1,ntrfspotfx,NApound,Non-TradeTrade,0,1,1,0,BNP,2,0,2,5eeaf3841554582ac01b49a2,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,78_379899248_BNP Paribas,OB,1340602394,OB,1336048325,No-Pair,0.52782,0.44409,0.02809
3,-2314.39000,1326013.74000,1,0,1328328.13000,0.00000,-2,1,1,ntrfspotfx,NAeuro,Non-TradeTrade,0,1,1,0,BNP,2,0,2,5eeaf3841554582ac01b49a2,5eeaf3841554582ac01b4978,92_379902880_Advent Geneva,78_379899248_BNP Paribas,OB,1340602395,OB,1336048325,UMB_One_to_One,0.15745,0.81983,0.02272
4,1326013.74000,-396193.87000,0,1,-1722207.61000,2.00000,0,1,1,s210spotfx,corp pound,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499d,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,20_379902880_BNP Paribas,OB,1340602394,OB,1340602430,UMB_One_to_One,0.02955,0.96129,0.00915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,100.62000,496.04000,0,0,395.42000,0.00000,-57,1,1,interestinterest,interestinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,3,4,5eeaf37b15545830ac1ccabd,5eeaf37b15545830ac1ccabc,1_379879036_Advent Geneva,7_379805934_US BANK,OB,1308694502,OB,1214171472,No-Pair,0.99999,0.00000,0.00000
1330,3706.27000,626.41000,0,0,-3079.86000,0.00000,-20,1,1,interestinterest,interestinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,4,3,5eeaf37b15545830ac1ccac4,5eeaf37b15545830ac1ccabb,1_379841544_Advent Geneva,1_379816267_US BANK,OB,1259461223,OB,1228154166,No-Pair,0.99996,0.00004,0.00000
1331,3706.27000,496.04000,0,0,-3210.23000,0.00000,-49,1,1,interestinterest,interestinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,4,4,5eeaf37b15545830ac1ccac4,5eeaf37b15545830ac1ccabc,1_379879036_Advent Geneva,1_379816267_US BANK,OB,1308694502,OB,1228154166,No-Pair,0.99999,0.00001,0.00000
1332,-591.94000,626.41000,0,0,1218.35000,0.00000,7,1,1,interest,NAinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,3,3,5eeaf37b15545830ac1ccac0,5eeaf37b15545830ac1ccabb,1_379841544_Advent Geneva,2_379850769_US BANK,OB,1259461223,OB,1271871929,No-Pair,0.99849,0.00133,0.00018


## Two Step Modeling

In [1274]:
model_cols_2 =[#'SideA.ViewData.B-P Net Amount', 
              #'SideA.ViewData.Cancel Flag', 
              #'SideA.new_desc_cat',
             # 'SideA.ViewData.Description',
             # 'SideA.ViewData.Department',
   
    
              
             # 'SideA.ViewData.Price',
             # 'SideA.ViewData.Quantity',
             #'SideA.ViewData.Investment Type', 
              #'SideA.ViewData.Asset Type Category', 
              #'SideB.ViewData.Accounting Net Amount', 
              #'SideB.ViewData.Cancel Flag', 
             # 'SideB.ViewData.Description',
              # 'SideB.ViewData.Department',
              
             # 'SideB.ViewData.Price',
             # 'SideB.ViewData.Quantity',
             # 'SideB.new_desc_cat',
             # 'SideB.ViewData.Investment Type', 
              #'SideB.ViewData.Asset Type Category', 
              'Trade_Date_match', 'Settle_Date_match', 
              #  'Amount_diff_2', 
              'Trade_date_diff', 'Settle_date_diff', 'SideA.ISIN_NA', 'SideB.ISIN_NA', 
             # 'ViewData.Combined Fund',
              'ViewData.Combined Transaction Type', 'Combined_Desc','Combined_TType',
             # 'SideA.TType', 'SideB.TType', 
              'abs_amount_flag',
    'tt_map_flag', 
              'All_key_nan','new_key_match', 'new_pb1',
              'SideB.Date','SideA.ViewData.Settle Date','SideB.ViewData.Settle Date',
            'SideA.ViewData._ID', 'SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds',
              'SideB.ViewData.Status', 'SideB.ViewData.BreakID_B_side',
              'SideA.ViewData.Status', 'SideA.ViewData.BreakID_A_side'] 
              #'label']

In [1275]:
X_test2 = test_file[model_cols_2]

In [1276]:
X_test2 = X_test2.reset_index()
X_test2 = X_test2.drop('index',1)
X_test2 = X_test2.fillna(0)

In [1277]:
X_test2.shape

(1477, 25)

In [1278]:
X_test2 = X_test2.drop_duplicates()
X_test2 = X_test2.reset_index()
X_test2 = X_test2.drop('index',1)

In [1279]:
X_test2.shape

(1334, 25)

In [1281]:
#filename2 = 'Soros_full_model_all_two_step.sav'

## TODO Import MOdel2 as per the two step modelling process

filename2 = 'OakTree_final_model2_step_two.sav'
clf2 = pickle.load(open(filename2, 'rb'))

In [1282]:
# Actual class predictions
rf_predictions2 = clf2.predict(X_test2.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))
# Probabilities for each class
rf_probs2 = clf2.predict_proba(X_test2.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 1]

In [1283]:
probability_class_0_two = clf2.predict_proba(X_test2.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side','SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 0]
probability_class_1_two = clf2.predict_proba(X_test2.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 1]

#probability_class_2 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side','SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 2]
#probability_class_3 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 3]

#probability_class_4 = clf.predict_proba(X_test.drop(['SideB.ViewData.Status','SideB.ViewData.BreakID_B_side', 'SideA.ViewData.Status','SideA.ViewData.BreakID_A_side','SideA.ViewData._ID','SideB.ViewData._ID','SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'],1))[:, 4]


In [1284]:
X_test2['Predicted_action_2'] = rf_predictions2
#X_test['Predicted_action_probabilty'] = rf_probs

#X_test['probability_Close'] = probability_class_0
X_test2['probability_No_pair_2'] = probability_class_0_two
#X_test['probability_Partial_match'] = probability_class_0
#X_test['probability_UMB'] = probability_class_1
X_test2['probability_UMB_2'] = probability_class_1_two
#X_test['probability_UMR'] = probability_class_2
#X_test['probability_UMT'] = probability_class_3

In [1285]:
X_test2['Predicted_action_2'].value_counts()

No-Pair           1127
UMB_One_to_One     207
Name: Predicted_action_2, dtype: int64

In [1286]:
X_test.shape

(1334, 32)

In [1287]:
X_test = pd.concat([X_test, X_test2[['Predicted_action_2','probability_No_pair_2','probability_UMB_2']]],axis=1)

In [1289]:
X_test

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2
0,-6089038.67000,-6089038.67000,0,1,0.00000,2.00000,0,1,1,s304spotfx,NAdollar,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499b,5eeaf3841554582ac01b4973,1_379902880_Advent Geneva,5_379902880_BNP Paribas,OB,1340602390,OB,1340602429,UMR_One_to_One,0.00000,0.00009,0.99991,UMB_One_to_One,0.15884,0.84116
1,6866000.00000,6866000.00000,0,1,0.00000,2.00000,0,1,1,s210spotfx,NAdollar,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b49a3,5eeaf3841554582ac01b4976,5_379902880_Advent Geneva,4_379902880_BNP Paribas,OB,1340602393,OB,1340602439,UMR_One_to_One,0.00000,0.00012,0.99988,UMB_One_to_One,0.15884,0.84116
2,-2314.39000,-396193.87000,1,0,-393879.48000,0.00000,-2,1,1,ntrfspotfx,NApound,Non-TradeTrade,0,1,1,0,BNP,2,0,2,5eeaf3841554582ac01b49a2,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,78_379899248_BNP Paribas,OB,1340602394,OB,1336048325,No-Pair,0.52782,0.44409,0.02809,UMB_One_to_One,0.36879,0.63121
3,-2314.39000,1326013.74000,1,0,1328328.13000,0.00000,-2,1,1,ntrfspotfx,NAeuro,Non-TradeTrade,0,1,1,0,BNP,2,0,2,5eeaf3841554582ac01b49a2,5eeaf3841554582ac01b4978,92_379902880_Advent Geneva,78_379899248_BNP Paribas,OB,1340602395,OB,1336048325,UMB_One_to_One,0.15745,0.81983,0.02272,UMB_One_to_One,0.33694,0.66306
4,1326013.74000,-396193.87000,0,1,-1722207.61000,2.00000,0,1,1,s210spotfx,corp pound,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499d,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,20_379902880_BNP Paribas,OB,1340602394,OB,1340602430,UMB_One_to_One,0.02955,0.96129,0.00915,UMB_One_to_One,0.44013,0.55987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,100.62000,496.04000,0,0,395.42000,0.00000,-57,1,1,interestinterest,interestinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,3,4,5eeaf37b15545830ac1ccabd,5eeaf37b15545830ac1ccabc,1_379879036_Advent Geneva,7_379805934_US BANK,OB,1308694502,OB,1214171472,No-Pair,0.99999,0.00000,0.00000,No-Pair,0.99984,0.00016
1330,3706.27000,626.41000,0,0,-3079.86000,0.00000,-20,1,1,interestinterest,interestinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,4,3,5eeaf37b15545830ac1ccac4,5eeaf37b15545830ac1ccabb,1_379841544_Advent Geneva,1_379816267_US BANK,OB,1259461223,OB,1228154166,No-Pair,0.99996,0.00004,0.00000,No-Pair,0.99986,0.00014
1331,3706.27000,496.04000,0,0,-3210.23000,0.00000,-49,1,1,interestinterest,interestinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,4,4,5eeaf37b15545830ac1ccac4,5eeaf37b15545830ac1ccabc,1_379879036_Advent Geneva,1_379816267_US BANK,OB,1308694502,OB,1228154166,No-Pair,0.99999,0.00001,0.00000,No-Pair,0.99993,0.00007
1332,-591.94000,626.41000,0,0,1218.35000,0.00000,7,1,1,interest,NAinterest,Non-TradeNon-Trade,0,1,0,0,USBK,2,3,3,5eeaf37b15545830ac1ccac0,5eeaf37b15545830ac1ccabb,1_379841544_Advent Geneva,2_379850769_US BANK,OB,1259461223,OB,1271871929,No-Pair,0.99849,0.00133,0.00018,No-Pair,0.96087,0.03913


## New Aggregation

In [1292]:
X_test['Tolerance_level'] = np.abs(X_test['probability_UMB_2'] - X_test['probability_No_pair_2'])

In [1293]:
#X_test[X_test['Tolerance_level']<0.1]['Predicted_action'].value_counts()

In [1294]:
b_side_agg = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action_2'].unique().reset_index()
a_side_agg = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action_2'].unique().reset_index()

## UMR segregation

In [1295]:
def umr_seg(X_test):
    b_count = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].value_counts().reset_index(name='count')
    b_unique = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].unique().reset_index()
    
    b_unique['len'] = b_unique['Predicted_action'].str.len()
    b_count2 = pd.merge(b_count, b_unique.drop('Predicted_action',1), on='SideB.ViewData.Side0_UniqueIds', how='left')
    umr_table = b_count2[(b_count2['Predicted_action']=='UMR_One_to_One') & (b_count2['count']==1) & (b_count2['len']<=2)]
    return umr_table['SideB.ViewData.Side0_UniqueIds'].values
    

In [1296]:
umr_ids_0 = umr_seg(X_test)

## 1st Prediction Table for One to One UMR

In [1297]:
final_umr_table = X_test[X_test['SideB.ViewData.Side0_UniqueIds'].isin(umr_ids_0) & (X_test['Predicted_action']=='UMR_One_to_One')]

In [1298]:
final_umr_table = final_umr_table[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds','SideB.ViewData.BreakID_B_side','SideA.ViewData.BreakID_A_side','Predicted_action','probability_No_pair','probability_UMB','probability_UMR']]

In [1299]:
final_umr_table.shape

(30, 8)

## No-Pair segregation

In [1307]:
#b_side_agg = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action_2'].unique().reset_index()
#a_side_agg = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action_2'].unique().reset_index()

In [1308]:
def no_pair_seg(X_test):
    
    b_side_agg = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action_2'].unique().reset_index()
    a_side_agg = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action_2'].unique().reset_index()
    
    b_side_agg['len'] = b_side_agg['Predicted_action_2'].str.len()
    b_side_agg['No_Pair_flag'] = b_side_agg['Predicted_action_2'].apply(lambda x: 1 if 'No-Pair' in x else 0)

    a_side_agg['len'] = a_side_agg['Predicted_action_2'].str.len()
    a_side_agg['No_Pair_flag'] = a_side_agg['Predicted_action_2'].apply(lambda x: 1 if 'No-Pair' in x else 0)
    
    no_pair_ids_b_side = b_side_agg[(b_side_agg['len']==1) & (b_side_agg['No_Pair_flag']==1)]['SideB.ViewData.Side0_UniqueIds'].values

    no_pair_ids_a_side = a_side_agg[(a_side_agg['len']==1) & (a_side_agg['No_Pair_flag']==1)]['SideA.ViewData.Side1_UniqueIds'].values
    
    return no_pair_ids_b_side, no_pair_ids_a_side

In [1309]:
no_pair_ids_b_side, no_pair_ids_a_side = no_pair_seg(X_test)

In [1310]:
X_test[(X_test['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side))]['Predicted_action_2'].value_counts()

No-Pair    922
Name: Predicted_action_2, dtype: int64

In [1311]:
X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action_2'].unique().reset_index()

,SideA.ViewData.Side1_UniqueIds,Predicted_action_2
0,100_379903081_State Street,"[No-Pair, UMB_One_to_One]"
1,102_379901293_State Street,[No-Pair]
2,104_379868950_The Bank of New York Mellon,[No-Pair]
3,106_379862216_State Street,[No-Pair]
4,108_379903081_State Street,[No-Pair]
5,10_379856281_Northern Trust,[No-Pair]
6,10_379903081_State Street,[No-Pair]
7,10_379903799_Credit Suisse,"[UMB_One_to_One, No-Pair]"
8,115_379834500_The Bank of New York Mellon,[No-Pair]
9,115_379903081_State Street,[No-Pair]


In [1312]:
X_test[X_test['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side)]['Predicted_action_2'].value_counts()

No-Pair    922
Name: Predicted_action_2, dtype: int64

In [1313]:
final_open_table = X_test[(X_test['SideB.ViewData.Side0_UniqueIds'].isin(no_pair_ids_b_side)) | (X_test['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side))]

In [1314]:
final_open_table = final_open_table[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds','SideB.ViewData.BreakID_B_side','SideA.ViewData.BreakID_A_side','Predicted_action_2','probability_No_pair_2','probability_UMB_2','probability_UMR']]

In [1315]:
final_open_table['probability_UMR'] = 0.00010
final_open_table = final_open_table.rename(columns = {'Predicted_action_2':'Predicted_action','probability_No_pair_2':'probability_No_pair','probability_UMB_2':'probability_UMB'})

In [1316]:
final_open_table.shape

(1085, 8)

In [1321]:
#final_open_table.head()

len(no_pair_ids_b_side)

171

In [1327]:
b_side_open_table = final_open_table.groupby('SideB.ViewData.Side0_UniqueIds')[['probability_No_pair','probability_UMB','probability_UMR']].mean().reset_index()
a_side_open_table = final_open_table.groupby('SideA.ViewData.Side1_UniqueIds')[['probability_No_pair','probability_UMB','probability_UMR']].mean().reset_index()

In [1330]:
a_side_open_table = a_side_open_table[a_side_open_table['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side)]
b_side_open_table = b_side_open_table[b_side_open_table['SideB.ViewData.Side0_UniqueIds'].isin(no_pair_ids_b_side)]

In [1332]:
b_side_open_table = b_side_open_table.reset_index().drop('index',1)
a_side_open_table = a_side_open_table.reset_index().drop('index',1)

In [1336]:
final_no_pair_table = pd.concat([a_side_open_table,b_side_open_table], axis=0)
final_no_pair_table = final_no_pair_table.reset_index().drop('index',1)

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [1337]:
#final_no_pair_table

,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Side0_UniqueIds,probability_No_pair,probability_UMB,probability_UMR
0,102_379901293_State Street,NaN,0.99018,0.00982,0.00010
1,104_379868950_The Bank of New York Mellon,NaN,0.98812,0.01188,0.00010
2,106_379862216_State Street,NaN,0.80836,0.19164,0.00010
3,108_379903081_State Street,NaN,0.87813,0.12187,0.00010
4,10_379856281_Northern Trust,NaN,0.99999,0.00001,0.00010
5,10_379903081_State Street,NaN,0.99956,0.00044,0.00010
6,115_379834500_The Bank of New York Mellon,NaN,0.99949,0.00051,0.00010
7,115_379903081_State Street,NaN,0.97735,0.02265,0.00010
8,116_379834500_The Bank of New York Mellon,NaN,0.99949,0.00051,0.00010
9,116_379903081_State Street,NaN,0.97735,0.02265,0.00010


## One to One UMB segregation

In [829]:
### IDs left after removing open ids from 0 and 1 side


X_test_left = X_test[~(X_test['SideB.ViewData.Side0_UniqueIds'].isin(no_pair_ids_b_side))]
X_test_left = X_test_left[~(X_test_left['SideA.ViewData.Side1_UniqueIds'].isin(no_pair_ids_a_side))]

X_test_left = X_test_left.reset_index().drop('index',1)

In [830]:
X_test_left['Predicted_action_2'].value_counts()

UMB_One_to_One    237
No-Pair            83
Name: Predicted_action_2, dtype: int64

In [831]:
### IDs left after removing UMR ids from 0 and 1 side

X_test_left = X_test_left[~(X_test_left['SideA.ViewData.Side1_UniqueIds'].isin(final_umr_table['SideA.ViewData.Side1_UniqueIds']))]

X_test_left = X_test_left[~(X_test_left['SideB.ViewData.Side0_UniqueIds'].isin(final_umr_table['SideB.ViewData.Side0_UniqueIds']))]

In [832]:
X_test_left.shape

(289, 36)

In [833]:
X_test_left['Predicted_action'].value_counts()

No-Pair           142
UMB_One_to_One    136
UMR_One_to_One     11
Name: Predicted_action, dtype: int64

In [834]:
X_test_left.groupby('SideB.ViewData.Side0_UniqueIds')['SideA.ViewData.Side1_UniqueIds'].unique().reset_index()

,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds
0,10_379901013_Advent Geneva,[123_379899499_Northern Trust]
1,112_379899569_Advent Geneva,"[22_379779032_State Street, 23_379810019_State..."
2,124_379903081_Advent Geneva,"[140_379903081_State Street, 146_379903081_Sta..."
3,131_379903081_Advent Geneva,"[127_379903081_State Street, 128_379903081_Sta..."
4,136_379903081_Advent Geneva,"[177_379903081_State Street, 178_379903081_Sta..."
5,137_379903081_Advent Geneva,"[177_379903081_State Street, 178_379903081_Sta..."
6,13_379903081_Advent Geneva,"[22_379901293_State Street, 21_379901293_State..."
7,141_379903081_Advent Geneva,"[177_379903081_State Street, 178_379903081_Sta..."
8,142_379903081_Advent Geneva,"[177_379903081_State Street, 178_379903081_Sta..."
9,143_379903081_Advent Geneva,"[177_379903081_State Street, 178_379903081_Sta..."


In [835]:
X_test_left = X_test_left.drop(['SideB.ViewData._ID','SideA.ViewData._ID'],1).drop_duplicates()
X_test_left = X_test_left.reset_index().drop('index',1)

In [836]:
X_test_umb = X_test_left[X_test_left['Predicted_action_2']=='UMB_One_to_One']
X_test_umb = X_test_umb.reset_index().drop('index',1)

In [837]:
#X_test_umb['UMB_key_OTO'] = X_test_umb['SideA.ViewData.Side1_UniqueIds'] + X_test_umb['SideB.ViewData.Side0_UniqueIds']

In [838]:
def one_to_one_umb(data):
    
    count = data['SideB.ViewData.Side0_UniqueIds'].value_counts().reset_index(name='count0')
    id0s = count[count['count0']==1]['index'].unique()
    id1s = data[data['SideB.ViewData.Side0_UniqueIds'].isin(id0s)]['SideA.ViewData.Side1_UniqueIds']
    
    count1 = data['SideA.ViewData.Side1_UniqueIds'].value_counts().reset_index(name='count1')
    final_ids = count1[(count1['count1']==1) & (count1['index'].isin(id1s))]['index'].unique()
    return final_ids
    
    

In [839]:
one_side_unique_umb_ids = one_to_one_umb(X_test_umb)

In [840]:
final_oto_umb_table = X_test_umb[X_test_umb['SideA.ViewData.Side1_UniqueIds'].isin(one_side_unique_umb_ids)]

In [841]:
final_oto_umb_table = final_oto_umb_table[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds','SideB.ViewData.BreakID_B_side','SideA.ViewData.BreakID_A_side','Predicted_action_2','probability_No_pair_2','probability_UMB_2','probability_UMR']]

In [842]:
final_oto_umb_table['probability_UMR'] = 0.00010
final_oto_umb_table = final_oto_umb_table.rename(columns = {'Predicted_action_2':'Predicted_action','probability_No_pair_2':'probability_No_pair','probability_UMB_2':'probability_UMB'})

In [843]:
X_test_umb2 = X_test_umb[~X_test_umb['SideA.ViewData.Side1_UniqueIds'].isin(one_side_unique_umb_ids)]
X_test_umb2 = X_test_umb2.reset_index().drop('index',1)

In [845]:
X_test_umb2.shape

(196, 34)

In [844]:
X_test_umb2['SideB.ViewData.Side0_UniqueIds'].value_counts()

5_379903799_Advent Geneva      15
6_379903799_Advent Geneva      15
58_379903081_Advent Geneva     13
67_379903081_Advent Geneva     13
64_379903081_Advent Geneva     11
56_379903081_Advent Geneva      8
66_379903081_Advent Geneva      8
57_379903081_Advent Geneva      8
65_379903081_Advent Geneva      8
43_379903081_Advent Geneva      6
51_379903081_Advent Geneva      6
141_379903081_Advent Geneva     6
50_379903081_Advent Geneva      6
44_379903081_Advent Geneva      6
142_379903081_Advent Geneva     6
143_379903081_Advent Geneva     6
41_379903081_Advent Geneva      5
48_379903081_Advent Geneva      5
49_379903081_Advent Geneva      5
42_379903081_Advent Geneva      5
69_379899240_Advent Geneva      3
68_379899240_Advent Geneva      3
6_379903081_Advent Geneva       2
124_379903081_Advent Geneva     2
13_379903081_Advent Geneva      2
136_379903081_Advent Geneva     2
137_379903081_Advent Geneva     2
131_379903081_Advent Geneva     2
15_379903081_Advent Geneva      2
93_379902880_A

In [846]:
X_test_umb2[X_test_umb2['SideB.ViewData.Side0_UniqueIds']=='432_379899569_Advent Geneva']['SideA.ViewData.Side1_UniqueIds'].values

array([], dtype=object)

In [847]:
df = pd.DataFrame({
'Column1': [1, 1, 1, 2, 2, 3, 3, 3, 3,4,5,6,7],
'Column2': [14, 13, 16, 18, 19, 14, 11, 24, 15,22,33,33,33]})

In [848]:
df

,Column1,Column2
0,1,14
1,1,13
2,1,16
3,2,18
4,2,19
5,3,14
6,3,11
7,3,24
8,3,15
9,4,22


In [849]:
#X_test_umb2[X_test_umb2['SideA.ViewData.Side1_UniqueIds']=='1175_379879573_State Street']['SideB.ViewData.Side0_UniqueIds'].unique()

In [850]:
def sample_otm(data, col1, col2):
    ss =[]
    count_table = data[col2].value_counts().reset_index()
    count_table.columns = [col2, 'count']
    for i in data[col1].unique():
        ids1 = data[data[col1]==i][col2].unique()
        if count_table[count_table[col2].isin(ids1)]['count'].max() ==1:
            ss.append(i)
        else:
            pass
    return ss


def sample_mto(data, col1, col2):
    ss =[]
    count_table = data[col1].value_counts().reset_index()
    count_table.columns = [col1, 'count']
    for i in data[col2].unique():
        ids0 = data[data[col2]==i][col1].unique()
        if count_table[count_table[col1].isin(ids0)]['count'].max() ==1:
            ss.append(i)
        else:
            pass
    return ss

In [851]:
sample_otm(df, 'Column1', 'Column2')

[2, 4]

In [852]:
sample_mto(df, 'Column1', 'Column2')

[22, 33]

In [853]:
sample_mto(X_test_umb2, 'SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds')

['123_379899499_Northern Trust',
 '5_379888504_State Street',
 '18_379901293_State Street',
 '121_379899240_The Bank of New York Mellon']

In [854]:
sample_otm(X_test_umb2, 'SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds')

['93_379902880_Advent Geneva',
 '6_379903081_Advent Geneva',
 '124_379903081_Advent Geneva',
 '131_379903081_Advent Geneva']

In [857]:
X_test_umb2[X_test_umb2['SideB.ViewData.Side0_UniqueIds'].isin(sample_otm(X_test_umb2, 'SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds'))]

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
0,-2314.39,-396193.87,1,0,-393879.48,0.0,-2,1,1,ntrfspotfx,NApound,Non-TradeTrade,0,1,1,0,BNP,2,0,2,93_379902880_Advent Geneva,78_379899248_BNP Paribas,OB,1340602394,OB,1336048325,No-Pair,0.527818,0.444094,0.028088,UMB_One_to_One,0.368785,0.631215,0.262429
1,-396193.87,-396193.87,0,1,0.00,2.0,0,1,1,s304spotfx,NApound,Non-TradeTrade,0,1,1,0,BNP,2,2,2,93_379902880_Advent Geneva,19_379902880_BNP Paribas,OB,1340602394,OB,1340602433,UMR_One_to_One,0.000003,0.000074,0.999923,UMB_One_to_One,0.347588,0.652412,0.304823
138,15861.69,945451.08,0,1,929589.39,-14.0,0,0,0,interestcorp,corp option loan,Non-TradeNon-Trade,0,0,0,1,SS,2,2,2,6_379903081_Advent Geneva,63_379903081_State Street,SMB-OB,1341054193,OB,1341054190,UMB_One_to_One,0.390472,0.601494,0.008034,UMB_One_to_One,0.191531,0.808469,0.616938
139,844023.42,945451.08,1,1,101427.66,0.0,0,0,0,corpcorp,option loanoption loan,Non-TradeNon-Trade,0,0,0,1,SS,2,2,2,6_379903081_Advent Geneva,64_379903081_State Street,SMB-OB,1341054193,SMB-OB,1341054193,UMB_One_to_One,0.048820,0.938496,0.012684,UMB_One_to_One,0.072652,0.927348,0.854696
184,-179.21,179.21,0,0,358.42,1.0,1,1,1,corp,loanloan,Non-TradeNon-Trade,1,0,0,0,SS,2,2,1,124_379903081_Advent Geneva,140_379903081_State Street,OB,1341054305,OB,1341054320,No-Pair,0.981551,0.015563,0.002886,UMB_One_to_One,0.452612,0.547388,0.094776
185,-179.21,179.21,0,0,358.42,1.0,1,1,1,corp,loanloan,Non-TradeNon-Trade,1,0,0,0,SS,2,2,1,124_379903081_Advent Geneva,146_379903081_State Street,OB,1341054305,OB,1341054326,No-Pair,0.981551,0.015563,0.002886,UMB_One_to_One,0.452612,0.547388,0.094776
186,-321.08,321.08,0,0,642.16,1.0,1,1,1,corp,loanloan,Non-TradeNon-Trade,1,0,0,0,SS,2,2,1,131_379903081_Advent Geneva,127_379903081_State Street,OB,1341054289,OB,1341054316,No-Pair,0.982306,0.015199,0.002495,UMB_One_to_One,0.452612,0.547388,0.094776
187,-321.08,321.08,0,0,642.16,1.0,1,1,1,corp,loanloan,Non-TradeNon-Trade,1,0,0,0,SS,2,2,1,131_379903081_Advent Geneva,128_379903081_State Street,OB,1341054289,OB,1341054333,No-Pair,0.982306,0.015199,0.002495,UMB_One_to_One,0.452612,0.547388,0.094776


In [855]:
#for i in sample_mto(X_test_umb2, 'SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds'):
X_test_umb2[X_test_umb2['SideA.ViewData.Side1_UniqueIds'].isin(sample_mto(X_test_umb2, 'SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds'))]

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
32,-274407.00,82860.45,1,1,357267.45,0.0,0,1,1,pay,feesloan,Non-TradeNon-Trade,0,0,0,0,NT,2,0,0,9_379901013_Advent Geneva,123_379899499_Northern Trust,OB,1338199239,OB,1336335342,No-Pair,0.988525,0.011270,0.000205,UMB_One_to_One,0.051404,0.948596,0.897192
33,-274407.00,319162.49,1,1,593569.49,0.0,0,1,1,pay,feesloan,Non-TradeNon-Trade,0,0,0,0,NT,2,0,0,10_379901013_Advent Geneva,123_379899499_Northern Trust,OB,1338199237,OB,1336335342,No-Pair,0.991217,0.008622,0.000161,UMB_One_to_One,0.051404,0.948596,0.897192
34,1365.00,-735.00,1,1,-2100.00,0.0,0,0,0,interesttax,loanloan,Non-TradeNon-Trade,0,0,0,1,SS,2,4,4,32_379903081_Advent Geneva,5_379888504_State Street,OB,1341054241,SMB-OB,1322256293,UMB_One_to_One,0.319971,0.677677,0.002352,UMB_One_to_One,0.138202,0.861798,0.723597
35,1365.00,2100.00,1,1,735.00,0.0,0,0,0,interestinterest,loanloan,Non-TradeNon-Trade,0,1,0,1,SS,2,4,4,25_379888504_Advent Geneva,5_379888504_State Street,SMB-OB,1322256293,SMB-OB,1322256293,No-Pair,0.589928,0.403143,0.006929,UMB_One_to_One,0.481978,0.518022,0.036043
36,-2348.36,2448.44,0,0,4796.80,20.0,20,1,1,internalinterest,/rec/em7857NA,Non-TradeNon-Trade,0,0,1,0,SS,2,1,2,675_379875583_Advent Geneva,18_379901293_State Street,SDB,1303827250,OB,1338687387,No-Pair,0.998286,0.001575,0.000140,UMB_One_to_One,0.478573,0.521427,0.042855
37,-2348.36,3293.70,1,1,5642.06,0.0,0,1,1,internalcorp,/rec/em7857loan,Non-TradeNon-Trade,0,0,0,0,SS,2,1,1,22_379903081_Advent Geneva,18_379901293_State Street,OB,1341054269,OB,1338687387,No-Pair,0.956709,0.042575,0.000717,UMB_One_to_One,0.180974,0.819026,0.638053
38,-2348.36,12884.69,0,0,15233.05,12.0,12,1,1,internalinterest,/rec/em7857NA,Non-TradeNon-Trade,0,0,1,0,SS,2,1,3,21_379886740_Advent Geneva,18_379901293_State Street,SDB,1319881332,OB,1338687387,No-Pair,0.995237,0.004650,0.000113,UMB_One_to_One,0.463360,0.536640,0.073281
39,-2348.36,2456.33,0,0,4804.69,12.0,12,1,1,internalinterest,/rec/em7857NA,Non-TradeNon-Trade,0,0,1,0,SS,2,1,3,19_379886740_Advent Geneva,18_379901293_State Street,SDB,1319881366,OB,1338687387,No-Pair,0.996116,0.003714,0.000169,UMB_One_to_One,0.463360,0.536640,0.073281
40,-2348.36,849.23,0,0,3197.59,12.0,12,1,1,internalinterest,/rec/em7857NA,Non-TradeNon-Trade,0,0,1,0,SS,2,1,3,18_379886740_Advent Geneva,18_379901293_State Street,SDB,1319881354,OB,1338687387,No-Pair,0.995619,0.004056,0.000325,UMB_One_to_One,0.463360,0.536640,0.073281
41,-2348.36,37268.09,0,0,39616.45,12.0,12,1,1,internalinterest,/rec/em7857interest,Non-TradeNon-Trade,0,0,1,0,SS,2,1,3,20_379886740_Advent Geneva,18_379901293_State Street,SDB,1319881318,OB,1338687387,No-Pair,0.995417,0.004494,0.000089,UMB_One_to_One,0.463360,0.536640,0.073281


In [861]:
X_test[X_test['SideA.ViewData.Side1_UniqueIds']=='20_379902880_BNP Paribas']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
4,1326013.74,-396193.87,0,1,-1722207.61,2.0,0,1,1,s210spotfx,corp pound,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499d,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,20_379902880_BNP Paribas,OB,1340602394,OB,1340602430,UMB_One_to_One,2.955194e-02,0.961295,0.009153,UMB_One_to_One,0.440126,0.559874,0.119748
5,1326013.74,1326013.74,0,1,0.00,2.0,0,1,1,s210spotfx,corp euro,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499d,5eeaf3841554582ac01b4978,92_379902880_Advent Geneva,20_379902880_BNP Paribas,OB,1340602395,OB,1340602430,UMR_One_to_One,9.197328e-07,0.000108,0.999891,UMB_One_to_One,0.440126,0.559874,0.119748


In [874]:
X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='93_379902880_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
2,-2314.39,-396193.87,1,0,-393879.48,0.0,-2,1,1,ntrfspotfx,NApound,Non-TradeTrade,0,1,1,0,BNP,2,0,2,5eeaf3841554582ac01b49a2,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,78_379899248_BNP Paribas,OB,1340602394,OB,1336048325,No-Pair,0.527818,0.444094,0.028088,UMB_One_to_One,0.368785,0.631215,0.262429
4,1326013.74,-396193.87,0,1,-1722207.61,2.0,0,1,1,s210spotfx,corp pound,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499d,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,20_379902880_BNP Paribas,OB,1340602394,OB,1340602430,UMB_One_to_One,0.029552,0.961295,0.009153,UMB_One_to_One,0.440126,0.559874,0.119748
6,-396193.87,-396193.87,0,1,0.00,2.0,0,1,1,s304spotfx,NApound,Non-TradeTrade,0,1,1,0,BNP,2,2,2,5eeaf3841554582ac01b499e,5eeaf3841554582ac01b4977,93_379902880_Advent Geneva,19_379902880_BNP Paribas,OB,1340602394,OB,1340602433,UMR_One_to_One,0.000003,0.000074,0.999923,UMB_One_to_One,0.347588,0.652412,0.304823


In [929]:
X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='68_379899240_Advent Geneva']['SideA.ViewData.B-P Net Amount'].values

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
1490,-8824.05,626481.67,0,0,635305.72,-50.0,-48,0,0,buycorp,NAoption loan,TradeNon-Trade,0,0,0,0,BNY,2,4,3,5eeaf39815545825a47180d4,5eeaf39815545825a4718156,68_379899240_Advent Geneva,44_379834500_The Bank of New York Mellon,SMB-OB,1336040167,SDB,1252653771,No-Pair,0.999569,0.000410,0.000022,No-Pair,0.999661,0.000339,0.999323
1492,8838.97,626481.67,0,0,617642.70,-48.0,-48,0,0,buycorp,NAoption loan,TradeNon-Trade,0,0,0,0,BNY,2,4,3,5eeaf39815545825a47180bb,5eeaf39815545825a4718156,68_379899240_Advent Geneva,23_379848989_The Bank of New York Mellon,SMB-OB,1336040167,OB,1269339337,No-Pair,0.999503,0.000447,0.000051,No-Pair,0.999661,0.000339,0.999323
1494,598850.00,626481.67,1,1,27631.67,0.0,0,0,0,corpcorp,option loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,5eeaf39815545825a4718156,5eeaf39815545825a4718156,68_379899240_Advent Geneva,38_379899240_The Bank of New York Mellon,SMB-OB,1336040167,SMB-OB,1336040167,UMB_One_to_One,0.072893,0.918218,0.008889,UMB_One_to_One,0.098889,0.901111,0.802222
1496,11406.67,626481.67,1,1,615075.00,0.0,0,0,0,corpcorp,option loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,5eeaf39815545825a4718154,5eeaf39815545825a4718156,68_379899240_Advent Geneva,221_379899240_The Bank of New York Mellon,SMB-OB,1336040167,SMB-OB,1336040170,UMB_One_to_One,0.134558,0.860440,0.005002,UMB_One_to_One,0.098889,0.901111,0.802222
1498,16225.00,626481.67,0,1,610256.67,12.0,0,0,0,cashcorp,loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,5eeaf39815545825a471813c,5eeaf39815545825a4718156,68_379899240_Advent Geneva,39_379899240_The Bank of New York Mellon,SMB-OB,1336040167,OB,1336040173,No-Pair,0.600873,0.390824,0.008303,UMB_One_to_One,0.347105,0.652895,0.305790
1500,940840.56,626481.67,0,0,-314358.89,-14.0,-10,0,0,sellcorp,corp option loan,TradeNon-Trade,0,0,0,0,BNY,2,0,3,5eeaf39815545825a47180d6,5eeaf39815545825a4718156,68_379899240_Advent Geneva,20_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602521,No-Pair,0.998754,0.001156,0.000090,No-Pair,0.999607,0.000393,0.999213
1502,-940840.56,626481.67,0,0,1567322.23,-10.0,-10,0,0,sellcorp,corp option loan,TradeNon-Trade,0,0,0,0,BNY,2,0,3,5eeaf39815545825a47180d5,5eeaf39815545825a4718156,68_379899240_Advent Geneva,18_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602528,No-Pair,0.998709,0.001251,0.000040,No-Pair,0.999607,0.000393,0.999213
1504,-89987.40,626481.67,0,0,716469.07,-1.0,1,0,0,buycorp,corp option loan,TradeNon-Trade,0,0,0,0,BNY,2,4,3,5eeaf39815545825a47180f0,5eeaf39815545825a4718156,68_379899240_Advent Geneva,19_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602542,No-Pair,0.994351,0.005590,0.000059,No-Pair,0.998756,0.001244,0.997512
1506,-332968.16,626481.67,0,0,959449.83,6.0,6,0,0,stifcorp,NAoption loan,Non-TradeNon-Trade,0,0,0,0,BNY,2,2,3,5eeaf39815545825a4718121,5eeaf39815545825a4718156,68_379899240_Advent Geneva,23_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602556,No-Pair,0.999557,0.000416,0.000028,No-Pair,0.985600,0.014400,0.971200
1508,332968.16,626481.67,0,0,293513.51,6.0,6,0,0,stifcorp,NAoption loan,Non-TradeNon-Trade,0,0,0,0,BNY,2,2,3,5eeaf39815545825a471814a,5eeaf39815545825a4718156,68_379899240_Advent Geneva,24_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602

In [941]:
import numpy as np 
import itertools
  
def find_closese_sum(numbers, targets):
    numbers = numbers[:]
    for t in targets:
        if not numbers:
            break
    combs = sum([list(itertools.combinations(numbers, r)) for r in range(1, len(numbers)+1)], [])
    sums = np.asarray(list(map(sum, combs)))
    bestcomb = combs[np.argmin(np.abs(np.asarray(sums) - t))]
    numbers = list(set(numbers).difference(bestcomb))
    print("Target: {},  combination: {}".format(t, bestcomb))
 


In [953]:
X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='68_379899240_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
1490,-8824.05,626481.67,0,0,635305.72,-50.0,-48,0,0,buycorp,NAoption loan,TradeNon-Trade,0,0,0,0,BNY,2,4,3,5eeaf39815545825a47180d4,5eeaf39815545825a4718156,68_379899240_Advent Geneva,44_379834500_The Bank of New York Mellon,SMB-OB,1336040167,SDB,1252653771,No-Pair,0.999569,0.000410,0.000022,No-Pair,0.999661,0.000339,0.999323
1492,8838.97,626481.67,0,0,617642.70,-48.0,-48,0,0,buycorp,NAoption loan,TradeNon-Trade,0,0,0,0,BNY,2,4,3,5eeaf39815545825a47180bb,5eeaf39815545825a4718156,68_379899240_Advent Geneva,23_379848989_The Bank of New York Mellon,SMB-OB,1336040167,OB,1269339337,No-Pair,0.999503,0.000447,0.000051,No-Pair,0.999661,0.000339,0.999323
1494,598850.00,626481.67,1,1,27631.67,0.0,0,0,0,corpcorp,option loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,5eeaf39815545825a4718156,5eeaf39815545825a4718156,68_379899240_Advent Geneva,38_379899240_The Bank of New York Mellon,SMB-OB,1336040167,SMB-OB,1336040167,UMB_One_to_One,0.072893,0.918218,0.008889,UMB_One_to_One,0.098889,0.901111,0.802222
1496,11406.67,626481.67,1,1,615075.00,0.0,0,0,0,corpcorp,option loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,5eeaf39815545825a4718154,5eeaf39815545825a4718156,68_379899240_Advent Geneva,221_379899240_The Bank of New York Mellon,SMB-OB,1336040167,SMB-OB,1336040170,UMB_One_to_One,0.134558,0.860440,0.005002,UMB_One_to_One,0.098889,0.901111,0.802222
1498,16225.00,626481.67,0,1,610256.67,12.0,0,0,0,cashcorp,loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,5eeaf39815545825a471813c,5eeaf39815545825a4718156,68_379899240_Advent Geneva,39_379899240_The Bank of New York Mellon,SMB-OB,1336040167,OB,1336040173,No-Pair,0.600873,0.390824,0.008303,UMB_One_to_One,0.347105,0.652895,0.305790
1500,940840.56,626481.67,0,0,-314358.89,-14.0,-10,0,0,sellcorp,corp option loan,TradeNon-Trade,0,0,0,0,BNY,2,0,3,5eeaf39815545825a47180d6,5eeaf39815545825a4718156,68_379899240_Advent Geneva,20_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602521,No-Pair,0.998754,0.001156,0.000090,No-Pair,0.999607,0.000393,0.999213
1502,-940840.56,626481.67,0,0,1567322.23,-10.0,-10,0,0,sellcorp,corp option loan,TradeNon-Trade,0,0,0,0,BNY,2,0,3,5eeaf39815545825a47180d5,5eeaf39815545825a4718156,68_379899240_Advent Geneva,18_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602528,No-Pair,0.998709,0.001251,0.000040,No-Pair,0.999607,0.000393,0.999213
1504,-89987.40,626481.67,0,0,716469.07,-1.0,1,0,0,buycorp,corp option loan,TradeNon-Trade,0,0,0,0,BNY,2,4,3,5eeaf39815545825a47180f0,5eeaf39815545825a4718156,68_379899240_Advent Geneva,19_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602542,No-Pair,0.994351,0.005590,0.000059,No-Pair,0.998756,0.001244,0.997512
1506,-332968.16,626481.67,0,0,959449.83,6.0,6,0,0,stifcorp,NAoption loan,Non-TradeNon-Trade,0,0,0,0,BNY,2,2,3,5eeaf39815545825a4718121,5eeaf39815545825a4718156,68_379899240_Advent Geneva,23_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602556,No-Pair,0.999557,0.000416,0.000028,No-Pair,0.985600,0.014400,0.971200
1508,332968.16,626481.67,0,0,293513.51,6.0,6,0,0,stifcorp,NAoption loan,Non-TradeNon-Trade,0,0,0,0,BNY,2,2,3,5eeaf39815545825a471814a,5eeaf39815545825a4718156,68_379899240_Advent Geneva,24_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602

In [943]:


find_closese_sum(numbers, targets)

TypeError: 'float' object is not iterable

In [950]:
import itertools
def finding_closet(ls,target,depth):
    closest = []
    for i in itertools.combinations(ls, depth):

        if sum(i) == target:
            return i
        else:
            closest.append((abs(sum(i) - target), i))
    return min(closest)[1]

In [955]:
finding_closet(numbers, targets,5)

TypeError: unsupported operand type(s) for -: 'float' and 'list'

In [971]:
X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='68_379899240_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
1490,-8824.05,626481.67,0,0,635305.72,-50.0,-48,0,0,buycorp,NAoption loan,TradeNon-Trade,0,0,0,0,BNY,2,4,3,5eeaf39815545825a47180d4,5eeaf39815545825a4718156,68_379899240_Advent Geneva,44_379834500_The Bank of New York Mellon,SMB-OB,1336040167,SDB,1252653771,No-Pair,0.999569,0.000410,0.000022,No-Pair,0.999661,0.000339,0.999323
1492,8838.97,626481.67,0,0,617642.70,-48.0,-48,0,0,buycorp,NAoption loan,TradeNon-Trade,0,0,0,0,BNY,2,4,3,5eeaf39815545825a47180bb,5eeaf39815545825a4718156,68_379899240_Advent Geneva,23_379848989_The Bank of New York Mellon,SMB-OB,1336040167,OB,1269339337,No-Pair,0.999503,0.000447,0.000051,No-Pair,0.999661,0.000339,0.999323
1494,598850.00,626481.67,1,1,27631.67,0.0,0,0,0,corpcorp,option loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,5eeaf39815545825a4718156,5eeaf39815545825a4718156,68_379899240_Advent Geneva,38_379899240_The Bank of New York Mellon,SMB-OB,1336040167,SMB-OB,1336040167,UMB_One_to_One,0.072893,0.918218,0.008889,UMB_One_to_One,0.098889,0.901111,0.802222
1496,11406.67,626481.67,1,1,615075.00,0.0,0,0,0,corpcorp,option loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,5eeaf39815545825a4718154,5eeaf39815545825a4718156,68_379899240_Advent Geneva,221_379899240_The Bank of New York Mellon,SMB-OB,1336040167,SMB-OB,1336040170,UMB_One_to_One,0.134558,0.860440,0.005002,UMB_One_to_One,0.098889,0.901111,0.802222
1498,16225.00,626481.67,0,1,610256.67,12.0,0,0,0,cashcorp,loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,5eeaf39815545825a471813c,5eeaf39815545825a4718156,68_379899240_Advent Geneva,39_379899240_The Bank of New York Mellon,SMB-OB,1336040167,OB,1336040173,No-Pair,0.600873,0.390824,0.008303,UMB_One_to_One,0.347105,0.652895,0.305790
1500,940840.56,626481.67,0,0,-314358.89,-14.0,-10,0,0,sellcorp,corp option loan,TradeNon-Trade,0,0,0,0,BNY,2,0,3,5eeaf39815545825a47180d6,5eeaf39815545825a4718156,68_379899240_Advent Geneva,20_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602521,No-Pair,0.998754,0.001156,0.000090,No-Pair,0.999607,0.000393,0.999213
1502,-940840.56,626481.67,0,0,1567322.23,-10.0,-10,0,0,sellcorp,corp option loan,TradeNon-Trade,0,0,0,0,BNY,2,0,3,5eeaf39815545825a47180d5,5eeaf39815545825a4718156,68_379899240_Advent Geneva,18_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602528,No-Pair,0.998709,0.001251,0.000040,No-Pair,0.999607,0.000393,0.999213
1504,-89987.40,626481.67,0,0,716469.07,-1.0,1,0,0,buycorp,corp option loan,TradeNon-Trade,0,0,0,0,BNY,2,4,3,5eeaf39815545825a47180f0,5eeaf39815545825a4718156,68_379899240_Advent Geneva,19_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602542,No-Pair,0.994351,0.005590,0.000059,No-Pair,0.998756,0.001244,0.997512
1506,-332968.16,626481.67,0,0,959449.83,6.0,6,0,0,stifcorp,NAoption loan,Non-TradeNon-Trade,0,0,0,0,BNY,2,2,3,5eeaf39815545825a4718121,5eeaf39815545825a4718156,68_379899240_Advent Geneva,23_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602556,No-Pair,0.999557,0.000416,0.000028,No-Pair,0.985600,0.014400,0.971200
1508,332968.16,626481.67,0,0,293513.51,6.0,6,0,0,stifcorp,NAoption loan,Non-TradeNon-Trade,0,0,0,0,BNY,2,2,3,5eeaf39815545825a471814a,5eeaf39815545825a4718156,68_379899240_Advent Geneva,24_379902881_The Bank of New York Mellon,SMB-OB,1336040167,OB,1340602

In [1073]:
#targets = list(X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='68_379899240_Advent Geneva']['SideB.ViewData.Accounting Net Amount'].max())
#numbers = list(X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='68_379899240_Advent Geneva']['SideA.ViewData.B-P Net Amount'].values)


In [ ]:
X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='68_379899240_Advent Geneva']['SideA.ViewData.B-P Net Amount'].values

## UMR One to Many Summation Code

In [1012]:
def f(v, i, S, memo):
    if i >= len(v): return 1 if S == 0 else 0
    if (i, S) not in memo:  # <-- Check if value has not been calculated.
        count = f(v, i + 1, S, memo)
        count += f(v, i + 1, S - v[i], memo)
        memo[(i, S)] = count  # <-- Memoize calculated result.
    return memo[(i, S)] # <-- Return memoized value.


def g(v, S, memo):
    subset = []
    for i, x in enumerate(v):
        # Check if there is still a solution if we include v[i]
        if f(v, i + 1, S - x, memo) > 0:
            subset.append(x)
            S -= x
    return subset


many_ids_1 = []
one_id_0 = []
for key in X_test_left['SideB.ViewData.Side0_UniqueIds'].unique():

    values =  X_test_left[X_test_left['SideB.ViewData.Side0_UniqueIds']==key]['SideA.ViewData.B-P Net Amount'].values
    net_sum = X_test_left[X_test_left['SideB.ViewData.Side0_UniqueIds']==key]['SideB.ViewData.Accounting Net Amount'].max()

    memo = dict()
    if f(v, 0, sum, memo) == 0: 
        print("There are no valid subsets.")
    else:
        amount_array = np.array((g(values, net_sum, memo)))

    id1_aggregation = X_test_left[(X_test_left['SideA.ViewData.B-P Net Amount'].isin(amount_array)) & (X_test_left['SideB.ViewData.Side0_UniqueIds']==key)]['SideA.ViewData.Side1_UniqueIds'].values
    id0_unique = key
    
    if key =='68_379899240_Advent Geneva':
        print(amount_array)
    
    if len(id1_aggregation)>0: 
        many_ids_1.append(id1_aggregation)
        one_id_0.append(id0_unique)
    else:
        pass
    #print(id1_aggregation)

[]


In [1010]:
#X_test_left

In [1025]:
def f(v, i, S, memo):
    if i >= len(v): return 1 if S == 0 else 0
    if (i, S) not in memo:  # <-- Check if value has not been calculated.
        count = f(v, i + 1, S, memo)
        count += f(v, i + 1, S - v[i], memo)
        memo[(i, S)] = count  # <-- Memoize calculated result.
    return memo[(i, S)] # <-- Return memoized value.


def g(v, S, memo):
    subset = []
    for i, x in enumerate(v):
        # Check if there is still a solution if we include v[i]
        if f(v, i + 1, S - x, memo) > 0:
            subset.append(x)
            S = S-x
    return subset

def f_equal(a, b):
    return abs(a - b) < epsilon


many_ids_1 = []
one_id_0 = []
for key in X_test_left['SideB.ViewData.Side0_UniqueIds'].unique():

    values =  X_test_left[X_test_left['SideB.ViewData.Side0_UniqueIds']==key]['SideA.ViewData.B-P Net Amount'].values
    net_sum = X_test_left[X_test_left['SideB.ViewData.Side0_UniqueIds']==key]['SideB.ViewData.Accounting Net Amount'].max()

    memo = dict()
    if f(v, 0, sum, memo) == 0: 
        print("There are no valid subsets.")
    else:
        amount_array = np.array((g(values, net_sum, memo)))

    id1_aggregation = X_test_left[(X_test_left['SideA.ViewData.B-P Net Amount'].isin(amount_array)) & (X_test_left['SideB.ViewData.Side0_UniqueIds']==key)]['SideA.ViewData.Side1_UniqueIds'].values
    id0_unique = key       
    
    if len(id1_aggregation)>0: 
        many_ids_1.append(id1_aggregation)
        one_id_0.append(id0_unique)
    else:
        pass

[]
626481.67
626481.67
True


In [1065]:
import numpy as np
array = np.array([11,22.22,3,4])
num = 33.33

def subsetsum(array,num):

    if np.isclose(num,0) or num < 1:
        return None
    elif len(array) == 0:
        return None
    else:
        if np.isclose(array[0], num):
            return [array[0]]
        else:
            with_v = subsetsum(array[1:],(num - array[0])) 
            if with_v:
                return [array[0]] + with_v
            else:
                return subsetsum(array[1:],num)

print('\nList of Values : ',array)
print('\nSum Desired : ',num)
print('\nValues that add up to sum : ',subsetsum(array,num))


List of Values :  [11.   22.22  3.    4.  ]

Sum Desired :  33.33

Values that add up to sum :  None


In [1071]:
import numpy as np
#array = np.array([11.11,22.22,3,4])
#num = 33.33

def subsetsum(array,num):

    if np.isclose(num,0) or num < 1:
        return None
    elif len(array) == 0:
        return None
    else:
        if np.isclose(array[0], num):
            return [array[0]]
        else:
            with_v = subsetsum(array[1:],(num - array[0])) 
            if with_v:
                return [array[0]] + with_v
            else:
                return subsetsum(array[1:],num)

#print('\nList of Values : ',array)
#print('\nSum Desired : ',num)
#print('\nValues that add up to sum : ',subsetsum(array,num))


many_ids_1 = []
one_id_0 = []
for key in X_test_left['SideB.ViewData.Side0_UniqueIds'].unique():

    values =  X_test[X_test['SideB.ViewData.Side0_UniqueIds']==key]['SideA.ViewData.B-P Net Amount'].values
    net_sum = X_test[X_test['SideB.ViewData.Side0_UniqueIds']==key]['SideB.ViewData.Accounting Net Amount'].max()

    #memo = dict()
    if subsetsum(values,net_sum) == None: 
        #print("There are no valid subsets.")
        pass
    else:
        amount_array = np.array(subsetsum(values,net_sum))

    id1_aggregation = X_test[(X_test['SideA.ViewData.B-P Net Amount'].isin(amount_array)) & (X_test['SideB.ViewData.Side0_UniqueIds']==key)]['SideA.ViewData.Side1_UniqueIds'].values
    id0_unique = key       
    
    if len(id1_aggregation)>1: 
        many_ids_1.append(id1_aggregation)
        one_id_0.append(id0_unique)
    else:
        pass

In [1072]:
many_ids_1

[array(['126_379901293_State Street', '133_379856502_State Street',
        '140_379903081_State Street', '146_379903081_State Street',
        '138_379903081_State Street', '139_379903081_State Street'],
       dtype=object),
 array(['117_379901293_State Street', '124_379856502_State Street',
        '129_379903081_State Street', '130_379903081_State Street'],
       dtype=object),
 array(['38_379899240_The Bank of New York Mellon',
        '221_379899240_The Bank of New York Mellon',
        '39_379899240_The Bank of New York Mellon'], dtype=object)]

In [1044]:
def f_equal(a, b):
    return abs(a - b) < epsilon
def find_sum(numbers, target):
    numbers = set(numbers)
    half = math.ceil(target / 2)
    print(half)
    partial_results = set(range(1, half))
    print(partial_results)
    for i in numbers.intersection(partial_results):
        print(i)
        #if f_equal(target, i) in numbers:
            
        if target - i in numbers:
            return True



In [1059]:
approximate_subset_sum([11,22.22,3,4], 33.33,0)

IndexError: list index out of range

In [1060]:
#X_test_left['SideB.ViewData.Side0_UniqueIds'].unique()

def trim(data, delta):
    """Trims elements within `delta` of other elements in the list."""

    output = []
    last = 0

    for element in data:
        if element['value'] > last * (1 + delta):
            output.append(element)
            last = element['value']

    return output
import itertools
import operator

def merge_lists(m, n):
    """
    Merges two lists into one.

    We do *not* remove duplicates, since we'd like to see all possible
    item combinations for the given approximate subset sum instead of simply
    confirming that there exists a subset that satisfies the given conditions.

    """
    merged = itertools.chain(m, n)
    return sorted(merged, key=operator.itemgetter('value'))

def approximate_subset_sum(data, target, epsilon):
    """
    Calculates the approximate subset sum total in addition
    to the items that were used to construct the subset sum.

    Modified to track the elements that make up the partial
    sums to then identify which subset items were chosen
    for the solution.

    """

    # Intialize our accumulator with the trivial solution
    acc = [{'value': 0, 'partials': [0]}]

    count = len(data)

    # Prep data by turning it into a list of hashes
    data = [{'value': d, 'partials': [d]} for d in data]

    for key, element in enumerate(data, start=1):
        augmented_list = [{
            'value': element['value'] + a['value'],
            'partials': a['partials'] + [element['value']]
        } for a in acc]

        acc = merge_lists(acc, augmented_list)
        acc = trim(acc, delta=float(epsilon) / (2 * count))
        acc = [val for val in acc if val['value'] <= target]

    # The resulting list is in ascending order of partial sums; the
    # best subset will be the last one in the list.
    return acc[-1]


List of Values :  [11.11 22.22  3.    4.  ]

Sum Desired :  33.33

Values that add up to sum :  [11.11, 22.22]


In [1002]:
many_ids_1

[array(['22_379901293_State Street', '21_379901293_State Street'],
       dtype=object),
 array(['22_379901293_State Street', '21_379901293_State Street'],
       dtype=object),
 array(['178_379903081_State Street', '171_379903081_State Street',
        '174_379903081_State Street'], dtype=object),
 array(['177_379903081_State Street', '171_379903081_State Street',
        '174_379903081_State Street'], dtype=object)]

In [999]:
X_test_left[X_test_left['SideB.ViewData.Side0_UniqueIds']=='68_379899240_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
264,598850.00,626481.67,1,1,27631.67,0.0,0,0,0,corpcorp,option loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,68_379899240_Advent Geneva,38_379899240_The Bank of New York Mellon,SMB-OB,1336040167,SMB-OB,1336040167,UMB_One_to_One,0.072893,0.918218,0.008889,UMB_One_to_One,0.098889,0.901111,0.802222
266,11406.67,626481.67,1,1,615075.00,0.0,0,0,0,corpcorp,option loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,68_379899240_Advent Geneva,221_379899240_The Bank of New York Mellon,SMB-OB,1336040167,SMB-OB,1336040170,UMB_One_to_One,0.134558,0.860440,0.005002,UMB_One_to_One,0.098889,0.901111,0.802222
268,16225.00,626481.67,0,1,610256.67,12.0,0,0,0,cashcorp,loanoption loan,Non-TradeNon-Trade,0,0,0,1,BNY,2,3,3,68_379899240_Advent Geneva,39_379899240_The Bank of New York Mellon,SMB-OB,1336040167,OB,1336040173,No-Pair,0.600873,0.390824,0.008303,UMB_One_to_One,0.347105,0.652895,0.305790


In [ ]:
for key in X_test['SideB.ViewData.Side0_UniqueIds'].unique()

In [939]:
def binary_search(a, x, lo=0, hi=None): 
    if hi is None: 
        hi = len(a) 
    while lo < hi: 
        mid = (lo+hi)//2
        midval = a[mid] 
        if midval < x: 
            lo = mid+1
        elif midval > x:  
            hi = mid 
        else: 
            return mid 
    return -1
  
def printPairs(arr, n):  
  
    pair_exists = False
    # Sort the array  
    arr.sort()  

    # Traverse the array  
    for i in range(n): 
        # For every arr[i] < 0 element,  
        # do a binary search for arr[i] > 0.  
        if (arr[i] < 0):  
            # If found, print the pair.  
            if (binary_search(arr,-arr[i])):  
                print(arr[i] , ", " , -arr[i])  
  
                pair_exists = True
        else: 
            break
  
    if (pair_exists == False):  
        print("No such pair exists") 

In [940]:
printPairs(X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='68_379899240_Advent Geneva']['SideA.ViewData.B-P Net Amount'].values,2)

-940840.56 ,  940840.56
-332968.16 ,  332968.16


In [949]:
X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='166_379903081_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
1055,6000000.00,-62271.85,0,0,-6062271.85,22.0,12,1,1,internalbuy,internal transferloan,Non-TradeTrade,0,0,0,0,SS,2,2,4,5eeb0c4915545825a479af22,5eeb0c4915545825a479aeb4,166_379903081_Advent Geneva,94_379903081_State Street,OB,1341054239,OB,1341054247,No-Pair,0.999209,0.000762,2.966968e-05,No-Pair,0.992563,0.007437,0.985126
1057,3646.66,-62271.85,0,0,-65918.51,2.0,-7,0,0,interestbuy,frontier communicationsloan,Non-TradeTrade,0,0,0,0,SS,2,4,4,5eeb0c4915545825a479af4a,5eeb0c4915545825a479aeb4,166_379903081_Advent Geneva,95_379903081_State Street,OB,1341054239,OB,1341054283,No-Pair,0.999995,0.000005,5.212373e-07,No-Pair,0.999419,0.000581,0.998839
1059,-62400.00,-62271.85,0,0,128.15,22.0,12,1,1,internalbuy,NAloan,Non-TradeTrade,0,0,0,0,SS,2,2,4,5eeb0c4915545825a479af5f,5eeb0c4915545825a479aeb4,166_379903081_Advent Geneva,96_379903081_State Street,OB,1341054239,OB,1341054296,No-Pair,0.998241,0.000927,8.321138e-04,No-Pair,0.961412,0.038588,0.922823
1061,128.15,-62271.85,0,0,-62400.00,22.0,12,1,1,internalbuy,loanloan,Non-TradeTrade,0,0,0,0,SS,2,2,4,5eeb0c4915545825a479af79,5eeb0c4915545825a479aeb4,166_379903081_Advent Geneva,97_379903081_State Street,OB,1341054239,OB,1341054312,No-Pair,0.999828,0.000160,1.214178e-05,No-Pair,0.982333,0.017667,0.964666


In [944]:
def findPairs(lst, K):  
    res = [] 
    while lst: 
        num = lst.pop() 
        diff = K - num 
        if diff in lst: 
            res.append((diff, num)) 
          
    res.reverse() 
    return res 
      
# Driver code 
lst = list(X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='166_379903081_Advent Geneva']['SideA.ViewData.B-P Net Amount'].values)


In [945]:
lst

[6000000.0, 3646.66, -62400.0, 128.15]

In [946]:
K = X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='166_379903081_Advent Geneva']['SideB.ViewData.Accounting Net Amount'].max()


In [947]:
K

-62271.85

In [948]:
print(findPairs(lst, K))

[(-62400.0, 128.15)]


-41514.57

In [900]:
lst = list(X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='1_379903081_Advent Geneva']['SideA.ViewData.B-P Net Amount'].values)

In [867]:
X_test[X_test['SideA.ViewData.Side1_UniqueIds']=='119_379903081_State Street']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
1170,1774.91,291.21,0,0,-1483.70,1.0,1,1,1,internalcorp,loanloan,Non-TradeNon-Trade,0,0,0,0,SS,2,2,1,5eeb0c4915545825a479af65,5eeb0c4915545825a479af21,174_379903081_Advent Geneva,119_379903081_State Street,OB,1341054262,OB,1341054299,No-Pair,0.926585,0.073198,0.000218,No-Pair,0.911449,0.088551,0.822898
1171,1774.91,-155679.62,0,0,-157454.53,22.0,12,1,1,internalbuy,loanloan,Non-TradeTrade,0,0,0,0,SS,2,2,4,5eeb0c4915545825a479af65,5eeb0c4915545825a479aeb0,170_379903081_Advent Geneva,119_379903081_State Street,OB,1341054236,OB,1341054299,No-Pair,0.999288,0.000690,0.000022,No-Pair,0.982333,0.017667,0.964666
1172,1774.91,1635.88,0,0,-139.03,19.0,19,1,1,internalinterest,loaninterest,Non-TradeNon-Trade,0,0,0,0,SS,2,2,4,5eeb0c4915545825a479af65,5eeb0c4915545825a479aeea,302_379879573_Advent Geneva,119_379903081_State Street,OB,1309328795,OB,1341054299,No-Pair,0.983206,0.015733,0.001061,No-Pair,0.954239,0.045761,0.908479
1173,1774.91,-1345000.00,0,0,-1346774.91,8.0,1,1,1,internalbuy,loanoption loan,Non-TradeTrade,0,0,0,0,SS,2,2,1,5eeb0c4915545825a479af65,5eeb0c4915545825a479aea7,175_379903081_Advent Geneva,119_379903081_State Street,OB,1341054230,OB,1341054299,No-Pair,0.999762,0.000230,0.000008,No-Pair,0.998333,0.001667,0.996667


In [870]:
df

,Column1,Column2
0,1,14
1,1,13
2,1,16
3,2,18
4,2,19
5,3,14
6,3,11
7,3,24
8,3,15
9,4,22


In [873]:
meo[meo['ViewData.Side1_UniqueIds']=='501_379879573_State Street']




,ViewData.Account Type,ViewData.Accounting Net Amount,ViewData.Activity Code,ViewData.Age,ViewData.Age WK,ViewData.Asset Type Category,ViewData.Base Currency,ViewData.Base Net Amount,ViewData.Bloomberg_Yellow_Key,ViewData.BreakID,ViewData.Business Date,ViewData.CUSIP,ViewData.Cancel Amount,ViewData.Cancel Flag,ViewData.Currency,ViewData.Cust Net Amount,ViewData.Custodian,ViewData.Custodian Account,ViewData.Department,ViewData.Derived Source,ViewData.Description,ViewData.ExpiryDate,ViewData.ExternalComment1,ViewData.ExternalComment2,ViewData.ExternalComment3,ViewData.FX Rate,ViewData.Fund,ViewData.ISIN,ViewData.Interest Amount,ViewData.InternalComment1,ViewData.InternalComment2,ViewData.InternalComment3,ViewData.Investment Type,ViewData.Is Combined Data,ViewData.Keys,ViewData.Mapped Custodian Account,ViewData.Net Amount Difference,ViewData.Net Amount Difference Absolute,ViewData.Non Trade Description,ViewData.OTE Custodian Account,ViewData.Price,ViewData.Prime Broker,ViewData.Quantity,ViewData.SEDOL,ViewData.SPM ID,ViewData.Settle Date,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,ViewData.Source Combination,ViewData.Status,ViewData.Strike Price,ViewData.System Comments,ViewData.Task Business Date,ViewData.Ticker,ViewData.Trade Date,ViewData.Trade Expenses,ViewData.Transaction Category,ViewData.Transaction ID,ViewData.Transaction Type,ViewData.Underlying Cusip,ViewData.Underlying ISIN,ViewData.Underlying Investment ID,ViewData.Underlying Sedol,ViewData.Underlying Ticker,ViewData._ID


In [604]:
X_test_umb2[X_test_umb2['SideB.ViewData.Side0_UniqueIds']=='18_379899486_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideB.ViewData.Accounting Net Amount,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Transaction Type,Combined_Desc,Combined_TType,abs_amount_flag,tt_map_flag,All_key_nan,new_key_match,new_pb1,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2,Tolerance_level
7,5626.61,13600.0,1,1,7973.39,0.0,0,1,1,interestinterest,NAoption loan,Non-TradeNon-Trade,0,1,0,0,"OKEF86441202,OKEF86445502",0,0,0,18_379899486_Advent Geneva,5_379899486_HSBC,OB,1336252661,OB,1336252653,No-Pair,0.773956,0.218521,0.007522,UMB_One_to_One,0.280527,0.719473,0.438945


In [ ]:

prediction_table['UMB_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMB_One_to_One' in x else 0)
prediction_table['UMT_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMT_One_to_One' in x else 0)
prediction_table['UMR_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMR_One_to_One' in x else 0)


In [335]:
from itertools import combinations
# set the number of closest combinations to show, the targeted number and the list
show = dummy.shape[0]
target = dummy['SideB.ViewData.Accounting Net Amount'].max()
#lis = [497.96, 10, 5084, 156.43, 381.3, 3298.85, 625.68]
lis = list(dummy['SideA.ViewData.B-P Net Amount'].values)

diffs = []
for n in range(1, len(lis)+1):
    numbers = combinations(lis, n)
    # list the combinations and their absolute difference to target
    for combi in numbers:
        diffs.append([combi, abs(target - sum(combi))])

diffs.sort(key=lambda x: x[1])

for item in diffs[:show]:
    print(item[0], round(item[1],10))

KeyboardInterrupt: 

In [345]:
from past.builtins import xrange
from itertools import combinations

show = dummy.shape[0]
target = dummy['SideB.ViewData.Accounting Net Amount'].max()
#lis = [497.96, 10, 5084, 156.43, 381.3, 3298.85, 625.68]

lis = list(dummy['SideA.ViewData.B-P Net Amount'].values)

#lis = [497.96, 10, 5084, 156.43, 381.3, 3298.85, 625.68]
predicted_status = list(X_test['Predicted_action'].values)
for i , (j,k) in enumerate(zip(lis,predicted_status)):
    for comb in combinations(lis, i):
        if sum(comb) == target:
            print (comb)

(-3950000.0,)
(-3950000.0,)


In [258]:
umr_table = X_test[(X_test['Predicted_action']=='UMR_One_to_One')]
umr_table = umr_table.reset_index().drop('index',1)


X_test_left = X_test[~X_test['SideA.ViewData.Side1_UniqueIds'].isin(umr_table['SideA.ViewData.Side1_UniqueIds'].unique())]

X_test_left = X_test_left[~X_test_left['SideB.ViewData.Side0_UniqueIds'].isin(umr_table['SideB.ViewData.Side0_UniqueIds'].unique())]

In [259]:
X_test_left

,SideA.ViewData.B-P Net Amount,SideA.new_desc_cat,SideB.ViewData.Accounting Net Amount,SideB.new_desc_cat,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Investment_Type,Combined_Asset_Type_Category,abs_amount_flag,tt_map_flag,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2
0,19005.12,corp,9878.47,corp,0,0,-9126.65,75,72.0,0,0,QPQP,returnofcapdividend,reiteqty,cashcorp,0,0,4,4,2,0,0,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e49a,108_153149314_Advent Geneva,6_153139764_Bank of America Merrill Lynch,SMB-OB,139013197,SMB-OB,135961673,No-Pair,0.994054,0.005623,0.000323,No-Pair,0.996438,0.003562
1,19005.12,corp,-472173.83,NA,0,0,-491178.95,-59,-83.0,0,0,QPQP,returnofcapdividend,reitreit,cashcorp,0,0,4,4,3,0,0,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e484,24_153147536_Advent Geneva,6_153139764_Bank of America Merrill Lynch,OB,137428616,SMB-OB,135961673,No-Pair,0.995622,0.004336,0.000042,No-Pair,0.996218,0.003782
2,19005.12,corp,350371.87,NA,0,0,331366.75,-59,-83.0,0,0,QPQP,returnofcapdividend,reitreit,cashcorp,0,0,4,4,3,0,0,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e492,25_153147536_Advent Geneva,6_153139764_Bank of America Merrill Lynch,SMB-OB,137428755,SMB-OB,135961673,No-Pair,0.995794,0.003954,0.000252,No-Pair,0.996218,0.003782
3,19005.12,corp,143107.82,NA,0,0,124102.70,-59,-83.0,0,0,QPQP,returnofcapreturnofcap,reitreit,cashcorp,0,0,4,4,3,0,0,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e491,158_153147536_Advent Geneva,6_153139764_Bank of America Merrill Lynch,SMB-OB,137428757,SMB-OB,135961673,No-Pair,0.995216,0.004493,0.000291,No-Pair,0.996218,0.003782
4,19005.12,corp,-108668.98,corp,0,0,-127674.10,-59,-61.0,0,0,QPQP,returnofcapdividend,reitreit,cashcash,0,0,4,4,2,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47d,299_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,OB,135961703,SMB-OB,135961673,No-Pair,0.961448,0.038005,0.000546,No-Pair,0.949970,0.050030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75550,4266917.82,onlinetransferweeklycollectionsref#bb08d35pbd,1709148.39,dollar,0,0,-2557769.43,178,178.0,1,1,QPQP,cashtransfer,cash,cash,0,1,4,4,1,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec114077675b,9_153133289_Advent Geneva,2_153157651_Wells Fargo,OB,125420844,OB,146861678,No-Pair,0.999628,0.000232,0.000140,No-Pair,0.998736,0.001264
75551,4266917.82,onlinetransferweeklycollectionsref#bb08d35pbd,-282.02,dollar,0,0,-4267199.84,178,178.0,1,1,QPQP,cashtransfer,cash,cash,0,1,4,4,1,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec114077675c,8_153133289_Advent Geneva,2_153157651_Wells Fargo,OB,125420843,OB,146861678,No-Pair,0.999677,0.000220,0.000102,No-Pair,0.998736,0.001264
75552,4266917.82,onlinetransferweeklycollectionsref#bb08d35pbd,4091616.08,dollar,0,0,-175301.74,178,178.0,1,1,QPQP,cashtransfer,cash,cash,0,1,4,4,1,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec1140776765,1_153133289_Advent Geneva,2_153157651_Wells Fargo,OB,125420841,OB,146861678,No-Pair,0.999190,0.000497,0.000312,No-Pair,0.998736,0.001264
75553,4266917.82,onlinetransferweeklycollectionsref#bb08d35pbd,1379191.95,dollar,0,0,-2887725.87,87,87.0,1,1,QPQP,cashtransfer,cash,cash,0,1,4,4,1,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec1140776753,10_153146143_Advent Geneva,2_153157651_Wells Fargo,OB,136564415,OB,146861678,No-Pair,0.999635,0.000243,0.000121,No-Pair,0.998736,0.001264


In [260]:
#prediction_table = X_test_left.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action_2'].unique().reset_index()

In [261]:
prediction_table = X_test_left.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].unique().reset_index()

In [262]:
prediction_table['len'] = prediction_table['Predicted_action'].str.len()
prediction_table['No_Pair_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'No-Pair' in x else 0)
prediction_table['UMB_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMB_One_to_One' in x else 0)
prediction_table['UMT_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMT_One_to_One' in x else 0)
prediction_table['UMR_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMR_One_to_One' in x else 0)


In [263]:
open_table = prediction_table[(prediction_table['No_Pair_flag']==1) & (prediction_table['len']==1)]
open_table = open_table.reset_index().drop('index',1)

In [264]:
X_test_left2 = X_test_left[~X_test_left['SideB.ViewData.Side0_UniqueIds'].isin(open_table['SideB.ViewData.Side0_UniqueIds'].unique())]

In [265]:
#X_test_left2

In [266]:
prediction_table2 = X_test_left2.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action'].unique().reset_index()

In [267]:
prediction_table2['len'] = prediction_table2['Predicted_action'].str.len()
prediction_table2['No_Pair_flag'] = prediction_table2['Predicted_action'].apply(lambda x: 1 if 'No-Pair' in x else 0)
prediction_table2['UMB_flag'] = prediction_table2['Predicted_action'].apply(lambda x: 1 if 'UMB_One_to_One' in x else 0)
prediction_table2['UMT_flag'] = prediction_table2['Predicted_action'].apply(lambda x: 1 if 'UMT_One_to_One' in x else 0)
prediction_table2['UMR_flag'] = prediction_table2['Predicted_action'].apply(lambda x: 1 if 'UMR_One_to_One' in x else 0)

In [268]:
open_table2 = prediction_table2[(prediction_table2['No_Pair_flag']==1) & (prediction_table2['len']==1)]
open_table2 = open_table2.reset_index().drop('index',1)

In [269]:
X_test_left3 = X_test_left2[~X_test_left2['SideA.ViewData.Side1_UniqueIds'].isin(open_table2['SideA.ViewData.Side1_UniqueIds'].unique())]

In [270]:
X_test_umb = X_test_left3[X_test_left3['Predicted_action']!='No-Pair']
X_test_umb = X_test_umb.reset_index()
X_test_umb = X_test_umb.drop('index',1)

In [271]:
umb_table = X_test_umb.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].count().reset_index()

In [272]:
umb_table

,SideB.ViewData.Side0_UniqueIds,Predicted_action
0,100_153157129_Advent Geneva,1
1,103_153157324_Advent Geneva,1
2,104_153157324_Advent Geneva,1
3,1056_153157324_Advent Geneva,1
4,1057_153157324_Advent Geneva,1
...,...,...
408,96_153157324_Advent Geneva,2
409,97_153151185_Advent Geneva,2
410,98_153151185_Advent Geneva,2
411,99_153157129_Advent Geneva,1


In [273]:
X_test_umb

,SideA.ViewData.B-P Net Amount,SideA.new_desc_cat,SideB.ViewData.Accounting Net Amount,SideB.new_desc_cat,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Investment_Type,Combined_Asset_Type_Category,abs_amount_flag,tt_map_flag,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2
0,19005.12,corp,54389.06,corp,1,1,35383.94,0,0.0,0,0,QPQP,returnofcapreturnofcap,reitreit,cashcash,0,0,4,4,4,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e49e,380_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,SMB-OB,135961673,SMB-OB,135961673,UMB_One_to_One,0.057345,0.894391,0.048264,UMB_One_to_One,0.271144,0.728856
1,19005.12,corp,62377.36,corp,1,1,43372.24,0,0.0,0,0,QPQP,returnofcapdividend,reitreit,cashcash,0,0,4,4,4,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e496,280_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,SMB-OB,135961682,SMB-OB,135961673,UMB_One_to_One,0.056864,0.895275,0.047860,UMB_One_to_One,0.271144,0.728856
2,19005.12,corp,-108668.98,corp,1,1,-127674.10,0,0.0,0,0,QPQP,returnofcapdividend,reitreit,cashcash,0,0,4,4,4,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e481,279_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,OB,135961692,SMB-OB,135961673,UMB_One_to_One,0.173997,0.816579,0.009425,UMB_One_to_One,0.271144,0.728856
3,19005.12,corp,-108668.98,corp,1,1,-127674.10,0,0.0,0,0,QPQP,returnofcapdividend,reitreit,cashcash,0,0,4,2,2,0,1,5ef9863d7781ec114071e49c,5ef9863d7781ec114071e47d,299_153144970_Advent Geneva,5_153139764_Bank of America Merrill Lynch,OB,135961703,SMB-OB,135961677,UMB_One_to_One,0.145839,0.848224,0.005937,UMB_One_to_One,0.384465,0.615535
4,19005.12,corp,54389.06,corp,1,1,35383.94,0,0.0,0,0,QPQP,returnofcapreturnofcap,reitreit,cashcash,0,0,4,2,2,0,1,5ef9863d7781ec114071e49c,5ef9863d7781ec114071e49c,382_153144970_Advent Geneva,5_153139764_Bank of America Merrill Lynch,SMB-OB,135961677,SMB-OB,135961677,UMB_One_to_One,0.071050,0.894501,0.034449,UMB_One_to_One,0.384465,0.615535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,-373107.75,dollar,1119290.37,dollar,0,0,1492398.12,1,1.0,1,1,QPQP,collateraltransfer,moneycash,colcashcash,0,1,4,4,3,0,0,5ef987327781ec114072c1fb,5ef987327781ec114072c1f1,22_153157570_Advent Geneva,21_153157570_Morgan Stanley,OB,146823065,OB,146823093,UMB_One_to_One,0.478011,0.520680,0.001310,UMB_One_to_One,0.492781,0.507219
733,-373107.75,dollar,894078.34,dollar,0,0,1267186.09,1,1.0,1,1,QPQP,collateraltransfer,moneycash,colcashcash,0,1,4,4,3,0,0,5ef987327781ec114072c1fb,5ef987327781ec114072c1d7,20_153157570_Advent Geneva,21_153157570_Morgan Stanley,OB,146823059,OB,146823093,UMB_One_to_One,0.488531,0.510159,0.001311,UMB_One_to_One,0.492781,0.507219
734,-397334.35,dollar,1119290.37,dollar,0,0,1516624.72,1,1.0,1,1,QPQP,collateraltransfer,moneycash,colcashcash,0,1,4,4,3,0,0,5ef987327781ec114072c220,5ef987327781ec114072c1f1,22_153157570_Advent Geneva,23_153157570_Morgan Stanley,OB,146823065,OB,146823110,UMB_One_to_One,0.478011,0.520680,0.001310,UMB_One_to_One,0.492781,0.507219
735,-397334.35,dollar,894078.34,dollar,0,0,1291412.69,1,1.0,1,1,QPQP,collateraltransfer,moneycash,colcashcash,0,1,4,4,3,0,0,5ef987327781ec114072c220,5ef987327781ec114072c1d7,20_153157570_Advent Geneva,23_153157570_Morgan Stanley,OB,146823059,OB,146823110,UMB_One_to_One,0.488531,0.510159,0.001311,UMB_One_to_One,0.492781,0.507219


In [274]:
check_ids = ['16_153157591_Advent Geneva','30_153157422_Advent Geneva','78_153156543_Advent Geneva',
             '182_153156543_Advent Geneva','170_153156543_Advent Geneva','567_153157547_Advent Geneva']

In [275]:
for i, key in enumerate(check_ids):
    if i==4:
        print(key)
        dummy = X_test_umb[X_test_umb['SideB.ViewData.Side0_UniqueIds']=='134_153157549_Advent Geneva']

170_153156543_Advent Geneva


In [276]:
#X_test[X_test['SideB.ViewData.Side0_UniqueIds']=='182_153156543_Advent Geneva']

dummy

,SideA.ViewData.B-P Net Amount,SideA.new_desc_cat,SideB.ViewData.Accounting Net Amount,SideB.new_desc_cat,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Investment_Type,Combined_Asset_Type_Category,abs_amount_flag,tt_map_flag,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2


In [763]:
X_test_mtm

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
3,-80090.53,0.0,reit,corp,350371.87,0.0,reit,corp,1,1,430462.40,0,0.0,0,0,QPQP,returnofcapdividend,corpcorp,reitreit,0,1,5ef9863d7781ec114071e491,5ef9863d7781ec114071e492,SMB-OB,137428755,SMB-OB,137428757,25_153147536_Advent Geneva,9_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.329948,0.665287,0.001883,0.002883
4,-80090.53,0.0,reit,corp,143107.82,0.0,reit,corp,1,1,223198.35,0,0.0,0,0,QPQP,returnofcapreturnofcap,corpcorp,reitreit,0,1,5ef9863d7781ec114071e491,5ef9863d7781ec114071e491,SMB-OB,137428757,SMB-OB,137428757,158_153147536_Advent Geneva,9_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.363437,0.631401,0.002044,0.003119
5,10044.12,0.0,equity,corp,28627.31,0.0,eqty,corp,0,0,18583.19,-187,-188.0,0,0,QPQP,dividendreturnofcap,corpcorp,equityeqty,0,1,5ef9863d7781ec114071e49a,5ef9863d7781ec114071e497,SMB-OB,139013195,SMB-OB,139013197,175_153149314_Advent Geneva,39_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.468020,0.527676,0.001851,0.002453
6,10044.12,0.0,equity,corp,5061.38,0.0,eqty,corp,0,0,-4982.74,-187,-188.0,0,0,QPQP,dividenddividend,corpcorp,equityeqty,0,1,5ef9863d7781ec114071e49a,5ef9863d7781ec114071e48d,SMB-OB,139013189,SMB-OB,139013197,47_153149314_Advent Geneva,39_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.496214,0.496268,0.002109,0.005409
7,10044.12,0.0,equity,corp,-29426.66,1.0,eqty,corp,0,0,-39470.78,-187,-188.0,0,0,QPQP,dividenddividend,corpcorp,equityeqty,0,1,5ef9863d7781ec114071e49a,5ef9863d7781ec114071e482,OB,139012398,SMB-OB,139013197,46_153149314_Advent Geneva,39_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.331748,0.662394,0.002235,0.003623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,-14336.28,0.0,,nan,81254788.20,0.0,cash,cash,1,1,81269124.48,0,0.0,1,1,QPQP,cashtransfer,cash,cash,1,0,5ef9c3167781ec114077681d,5ef9c3167781ec1140776760,OB,125420837,OB,121293442,5_153133289_Advent Geneva,6_153128696_Wells Fargo,UMB_One_to_One,0.403886,0.592252,0.002513,0.001348
97,-36480.00,0.0,,nan,31065147.04,0.0,cash,cash,1,1,31101627.04,0,0.0,1,1,QPQP,cashtransfer,cash,cash,1,0,5ef9c3167781ec11407768c6,5ef9c3167781ec114077675d,OB,125420838,OB,121293443,7_153133289_Advent Geneva,7_153128696_Wells Fargo,UMB_One_to_One,0.484683,0.511937,0.002348,0.001032
98,-36480.00,0.0,,nan,81254788.20,0.0,cash,cash,1,1,81291268.20,0,0.0,1,1,QPQP,cashtransfer,cash,cash,1,0,5ef9c3167781ec11407768c6,5ef9c3167781ec1140776760,OB,125420837,OB,121293443,5_153133289_Advent Geneva,7_153128696_Wells Fargo,UMB_One_to_One,0.403886,0.592252,0.002513,0.001348
99,-230506.00,0.0,,nan,31065147.04,0.0,cash,cash,1,1,31295653.04,0,0.0,1,1,QPQP,cashtransfer,cash,cash,1,0,5ef9c3167781ec1140776987,5ef9c3167781ec114077675d,OB,125420838,OB,121293444,7_153133289_Advent Geneva,8_153128696_Wells Fargo,UMB_One_to_One,0.468680,0.527655,0.002567,0.001099


In [277]:
def get_relation(df, col1, col2):        
    first_max = df[[col1, col2]].groupby(col1).count().max()[0]
    second_max = df[[col1, col2]].groupby(col2).count().max()[0]
    if first_max==1:
        if second_max==1:
            return 'one-to-one'
        else:
            return 'one-to-many'
    else:
        if second_max==1:
            return 'many-to-one'
        else:
            return 'many-to-many'

#from itertools import product
#for col_i, col_j in product(df.columns, df.columns):
#    if col_i == col_j:
#        continue
#    print(col_i, col_j, get_relation(df, col_i, col_j))



In [278]:
print(get_relation(X_test_umb, 'SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'))

many-to-many


In [766]:
X_test_umb[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds']].groupby('SideB.ViewData.Side0_UniqueIds').count()

,SideA.ViewData.Side1_UniqueIds
SideB.ViewData.Side0_UniqueIds,
107_153149314_Advent Geneva,2
108_153149314_Advent Geneva,2
127_153157410_Advent Geneva,4
143_153147721_Advent Geneva,1
1441_153147925_Advent Geneva,1
1443_153147925_Advent Geneva,1
14_153151185_Advent Geneva,2
158_153147536_Advent Geneva,2
15_153151185_Advent Geneva,1


In [767]:
X_test_umb[X_test_umb['SideB.ViewData.Side0_UniqueIds']=='9_153154447_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT


In [768]:
X_test_umb[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds']].groupby('SideA.ViewData.Side1_UniqueIds').count()

,SideB.ViewData.Side0_UniqueIds
SideA.ViewData.Side1_UniqueIds,
100_153157424_Morgan Stanley,2
101_153157424_Morgan Stanley,2
102_153157549_Credit suisse,1
104_153157549_Credit suisse,1
111_153157424_Morgan Stanley,1
...,...
7_153128696_Wells Fargo,2
7_153139764_Bank of America Merrill Lynch,1
8_153128696_Wells Fargo,2


In [629]:
df = pd.DataFrame({
'Column1': [1, 1, 1, 2, 2, 3, 3, 3, 3,4],
'Column2': [14, 13, 16, 18, 19, 14, 11, 24, 15,22]})


In [635]:
df

,Column1,Column2
0,1,14
1,1,13
2,1,16
3,2,18
4,2,19
5,3,14
6,3,11
7,3,24
8,3,15
9,4,22


In [639]:
df.groupby(['Column2'])['Column1'].nunique()

Column2
11    1
13    1
14    2
15    1
16    1
18    1
19    1
22    1
24    1
Name: Column1, dtype: int64

In [279]:
def one_to_one(df, col1, col2):        
    grp = df[[col1, col2]].groupby(col1).count().reset_index()
    grp.columns = [col1, 'count']
    
    grp2 = df[[col1, col2]].groupby(col2).count().reset_index()
    grp2.columns = [col2, 'count']
    
    grp3 = grp[grp['count']==1]
    grp4 = grp2[grp2['count']==1]
    
 
    return grp3[col1].unique() , grp4[col2].unique()

In [280]:
one_to_many(df, 'Column1','Column2')

NameError: name 'one_to_many' is not defined

In [771]:
df.groupby(['Column1'])['Column2'].unique()

Column1
1        [14, 13, 16]
2            [18, 19]
3    [14, 11, 24, 15]
4                [22]
Name: Column2, dtype: object

In [772]:
df.groupby(['Column2']).count().reset_index()

,Column2,Column1
0,11,1
1,13,1
2,14,2
3,15,1
4,16,1
5,18,1
6,19,1
7,22,1
8,24,1


In [773]:
df

,Column1,Column2
0,1,14
1,1,13
2,1,16
3,2,18
4,2,19
5,3,14
6,3,11
7,3,24
8,3,15
9,4,22


In [281]:
def one_to_many(df, colB, colA):        
    grp = df[[colB, colA]].groupby(colA)[colB].nunique().reset_index()
    
    grp.columns = [colA, 'count']
    
    grp2 = grp[grp['count']==1]
 
    return grp2[colA].unique()

In [775]:
one_to_many(X_test_umb, 'SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds')

array(['102_153157549_Credit suisse', '104_153157549_Credit suisse',
       '111_153157424_Morgan Stanley', '113_153157570_Morgan Stanley',
       '114_153157570_Morgan Stanley', '115_153156963_Morgan Stanley',
       '137_153157410_Credit suisse', '142_153157410_Credit suisse',
       '1_153139764_Bank of America Merrill Lynch',
       '22_153156765_JP Morgan', '245_153156963_Morgan Stanley',
       '24_153157591_Goldman Sachs', '26_153156765_JP Morgan',
       '26_153157591_Goldman Sachs', '2_153153898_CITI',
       '303_153157549_Credit suisse', '31_153157547_CITI',
       '342_153157410_Credit suisse', '375_153157410_Credit suisse',
       '379_153157410_Credit suisse', '3_153148927_Wells Fargo',
       '3_153155253_CITI', '44_153157129_CITI', '4_153153898_CITI',
       '4_153156963_Morgan Stanley', '4_153157591_Goldman Sachs',
       '50_153156765_JP Morgan', '52_153154996_CITI',
       '55_153157424_Morgan Stanley', '5_153157591_Goldman Sachs',
       '6_153139764_Bank of America

In [776]:
X_test_otm = X_test_umb[X_test_umb['SideA.ViewData.Side1_UniqueIds'].isin(one_to_many(X_test_umb, 'SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds'))]

In [777]:
X_test_umb[X_test_umb['SideA.ViewData.Side1_UniqueIds'].isin(one_to_many(X_test_umb, 'SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds'))]

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
0,1.900512e+04,0.0,reit,cash,-1.086690e+05,1.0,reit,cash,0,0,-1.276741e+05,-59,-61.0,0,0,QPQP,returnofcapdividend,cashcash,reitreit,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47d,OB,135961703,SMB-OB,135961673,299_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.494960,0.499535,0.003222,0.002283
1,1.900512e+04,0.0,reit,cash,-1.086690e+05,1.0,reit,cash,0,0,-1.276741e+05,-62,-62.0,0,0,QPQP,returnofcapdividend,cashcash,reitreit,0,1,5ef9863d7781ec114071e494,5ef9863d7781ec114071e483,OB,135683081,SMB-OB,135961678,318_153144970_Advent Geneva,7_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.488083,0.506513,0.003194,0.002210
2,-1.842696e+04,0.0,reit,cash,6.237736e+04,0.0,reit,cash,1,1,8.080432e+04,0,0.0,0,0,QPQP,dividenddividend,cashcash,reitreit,0,1,5ef9863d7781ec114071e48e,5ef9863d7781ec114071e48e,SMB-OB,135961698,SMB-OB,135961698,78_153144970_Advent Geneva,1_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.492068,0.499997,0.003135,0.004800
35,5.054400e+02,0.0,adj,5,-7.317080e+04,0.0,abs,abs,0,0,-7.367624e+04,83,80.0,0,0,PLPL,adjtrade,abs,adjabs,0,1,5ef985a17781ec1ac885556f,5ef985a17781ec1ac885557f,OB,137596015,OB,146387808,143_153147721_Advent Geneva,44_153157129_CITI,UMB_One_to_One,0.439382,0.551060,0.003782,0.005776
38,-3.736773e+08,0.0,tba,abs,-3.740032e+08,0.0,tba,abs,1,1,-3.259030e+05,0,0.0,0,0,QPQP,buybuy,absabs,tbatba,0,1,5ef985a17781ec1ac8855675,5ef985a17781ec1ac8855675,SMB-OB,143109794,SMB-OB,143109794,90_153153898_Advent Geneva,2_153153898_CITI,UMB_One_to_One,0.370725,0.621625,0.003106,0.004544
39,-1.520361e+08,0.0,tba,abs,-1.521445e+08,0.0,tba,abs,1,1,-1.083330e+05,0,0.0,0,0,QPQP,buybuy,absabs,tbatba,0,1,5ef985a17781ec1ac885566f,5ef985a17781ec1ac885566f,SMB-OB,143109810,SMB-OB,143109810,96_153153898_Advent Geneva,4_153153898_CITI,UMB_One_to_One,0.387074,0.603105,0.003939,0.005882
40,5.197266e+07,0.0,tba,abs,5.201780e+07,0.0,tba,abs,1,1,4.513890e+04,0,0.0,0,0,QPQP,sellsell,absabs,tbatba,0,1,5ef985a17781ec1ac8855677,5ef985a17781ec1ac8855677,SMB-OB,144337743,SMB-OB,144337743,33_153154996_Advent Geneva,52_153154996_CITI,UMB_One_to_One,0.146663,0.849012,0.002144,0.002181
48,-5.101562e+07,0.0,tba,abs,-5.104896e+07,0.0,tba,abs,1,1,-3.333330e+04,0,0.0,0,0,QPQP,buybuy,absabs,tbatba,0,1,5ef985a17781ec1ac8855672,5ef985a17781ec1ac8855672,SMB-OB,144640330,SMB-OB,144640330,25_153155253_Advent Geneva,3_153155253_CITI,UMB_One_to_One,0.466095,0.522001,0.004724,0.007180
57,9.667014e+04,0.0,cpn,5,5.272917e+04,0.0,abs,abs,0,0,-4.394097e+04,25,1.0,0,0,QPQP,interestinterest,abs,cpnabs,0,1,5ef985a17781ec1ac8855627,5ef985a17781ec1ac8855610,OB,146592969,OB,146797541,531_153157324_Advent Geneva,31_153157547_CITI,UMB_One_to_One,0.397858,0.600339,0.000418,0.001385
67,1.266177e+05,0.0,cash,cash,4.851953e+04,0.0,abs,adr,0,1,-7.809814e+04,24,0.0,0,0,PLPL,interestinterest,cashadr,cashabs,0,1,5ef985a27781ec1ac88556a1,5ef985a27781ec1ac885568e,SPM,146799358,SPM,146799386,127_153157410_Advent Geneva,142_153157410_Credit suisse,UMB_One_to_One,0.419239,0.578014,0.000850,0.001896


In [652]:
X_test_otm.shape

(37, 34)

In [282]:
def one_to_one(df, col1, col2):        
    grp = df[[col1, col2]].groupby(col1).count().reset_index()
    grp.columns = [col1, 'count']
    
    grp2 = df[[col1, col2]].groupby(col2).count().reset_index()
    grp2.columns = [col2, 'count']
    
    grp3 = grp[grp['count']==1]
    grp4 = grp2[grp2['count']==1]
    
 
    return grp3[col1].unique() , grp4[col2].unique()

In [283]:
def one_to_many(df, col1, col2):        
    grp = df[[col1, col2]].groupby(col2).count().reset_index()
    grp.columns = [col2, 'count']
    
    grp2 = grp[grp['count']==1]
 
    return grp2[col2].unique()

In [284]:
def many_to_one(df, col1, col2):
    grp = df[[col1, col2]].groupby(col1).count().reset_index()
    grp.columns = [col1, 'count']
    
    grp2 = grp[grp['count']==1]
 
    return grp2[col1].unique()

In [494]:
#X_test_umb['key'] = X_test_umb['SideB.ViewData.Side0_UniqueIds'] + X_test_umb['SideA.ViewData.Side1_UniqueIds']

In [623]:
#X_test_umb['key'].value_counts()

b = X_test_umb[['SideB.ViewData.Side0_UniqueIds', 'SideA.ViewData.Side1_UniqueIds']].groupby('SideB.ViewData.Side0_UniqueIds')['SideA.ViewData.Side1_UniqueIds'].count().reset_index()

In [624]:
b

,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds
0,107_153149314_Advent Geneva,2
1,108_153149314_Advent Geneva,2
2,127_153157410_Advent Geneva,4
3,143_153147721_Advent Geneva,1
4,1441_153147925_Advent Geneva,1
...,...,...
56,8_153133289_Advent Geneva,1
57,90_153153898_Advent Geneva,1
58,96_153153898_Advent Geneva,5
59,97_153151185_Advent Geneva,2


In [625]:
b[b['SideB.ViewData.Side0_UniqueIds']=='158_153147536_Advent Geneva']

,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds
7,158_153147536_Advent Geneva,2


In [619]:
X_test_umb[X_test_umb['SideA.ViewData.Side1_UniqueIds']=='104_153157549_Credit suisse']

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
75,16531097.96,0.0,debt,cash,16296179.50,0.0,abs,adr,0,0,-234918.46,1,1.0,0,0,PLPL,p&isell,cashadr,debtabs,0,1,5ef985a27781ec1ac88556df,5ef985a27781ec1ac88558dd,OB,146800005,OB,146799640,299_153157549_Advent Geneva,104_153157549_Credit suisse,UMB_One_to_One,0.234447,0.756786,0.003777,0.004989
76,16531097.96,0.0,debt,cash,48519.53,0.0,abs,adr,0,0,-16482578.43,25,1.0,0,0,PLPL,p&iinterest,cashadr,debtabs,0,1,5ef985a27781ec1ac88556df,5ef985a27781ec1ac885568e,SPM,146799358,OB,146799640,127_153157410_Advent Geneva,104_153157549_Credit suisse,UMB_One_to_One,0.481036,0.508289,0.003693,0.006982


In [600]:
X_test_mtm['SideB.ViewData.Side0_UniqueIds'].value_counts()

22_153157570_Advent Geneva     8
20_153157570_Advent Geneva     8
7_153133289_Advent Geneva      6
5_153133289_Advent Geneva      6
569_153157547_Advent Geneva    4
567_153157547_Advent Geneva    4
127_153157410_Advent Geneva    4
568_153157547_Advent Geneva    4
175_153149314_Advent Geneva    3
192_153157410_Advent Geneva    3
47_153149314_Advent Geneva     3
46_153149314_Advent Geneva     3
184_153149314_Advent Geneva    2
41_153156073_Advent Geneva     2
1_153157570_Advent Geneva      2
108_153149314_Advent Geneva    1
45_153157587_Advent Geneva     1
47_153157587_Advent Geneva     1
25_153147536_Advent Geneva     1
107_153149314_Advent Geneva    1
34_153154996_Advent Geneva     1
158_153147536_Advent Geneva    1
Name: SideB.ViewData.Side0_UniqueIds, dtype: int64

In [601]:
X_test_umb[X_test_umb['SideB.ViewData.Side0_UniqueIds']=='158_153147536_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
5,82578.52,0.0,reit,corp,143107.82,0.0,reit,corp,1,1,60529.30,0,0.0,0,0,QPQP,dividendreturnofcap,corpcorp,reitreit,0,1,5ef9863d7781ec114071e492,5ef9863d7781ec114071e491,SMB-OB,137428757,SMB-OB,137428755,158_153147536_Advent Geneva,11_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.292797,0.694260,0.005671,0.007272
7,-80090.53,0.0,reit,corp,143107.82,0.0,reit,corp,1,1,223198.35,0,0.0,0,0,QPQP,returnofcapreturnofcap,corpcorp,reitreit,0,1,5ef9863d7781ec114071e491,5ef9863d7781ec114071e491,SMB-OB,137428757,SMB-OB,137428757,158_153147536_Advent Geneva,9_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.363437,0.631401,0.002044,0.003119


In [602]:
X_test_umb[X_test_umb['SideA.ViewData.Side1_UniqueIds']=='11_153139764_Bank of America Merrill Lynch']

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
3,82578.52,0.0,reit,corp,-472173.83,1.0,reit,corp,1,1,-554752.35,0,0.0,0,0,QPQP,dividenddividend,corpcorp,reitreit,0,1,5ef9863d7781ec114071e492,5ef9863d7781ec114071e484,OB,137428616,SMB-OB,137428755,24_153147536_Advent Geneva,11_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.372856,0.605214,0.007654,0.014276
4,82578.52,0.0,reit,corp,350371.87,0.0,reit,corp,1,1,267793.35,0,0.0,0,0,QPQP,dividenddividend,corpcorp,reitreit,0,1,5ef9863d7781ec114071e492,5ef9863d7781ec114071e492,SMB-OB,137428755,SMB-OB,137428755,25_153147536_Advent Geneva,11_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.220432,0.762543,0.006000,0.011026
5,82578.52,0.0,reit,corp,143107.82,0.0,reit,corp,1,1,60529.30,0,0.0,0,0,QPQP,dividendreturnofcap,corpcorp,reitreit,0,1,5ef9863d7781ec114071e492,5ef9863d7781ec114071e491,SMB-OB,137428757,SMB-OB,137428755,158_153147536_Advent Geneva,11_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.292797,0.694260,0.005671,0.007272


In [607]:
#X_test_umb.to_csv('umb_example.csv')

In [604]:
X_test_mto[X_test_mto['SideA.ViewData.Side1_UniqueIds']=='11_153139764_Bank of America Merrill Lynch']

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
0,82578.52,0.0,reit,corp,-472173.83,1.0,reit,corp,1,1,-554752.35,0,0.0,0,0,QPQP,dividenddividend,corpcorp,reitreit,0,1,5ef9863d7781ec114071e492,5ef9863d7781ec114071e484,OB,137428616,SMB-OB,137428755,24_153147536_Advent Geneva,11_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.372856,0.605214,0.007654,0.014276


In [496]:
#dd = X_test_umb2[['SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds']].groupby('SideB.ViewData.Side0_UniqueIds').count().reset_index()
#dd[dd['SideA.ViewData.Side1_UniqueIds']==1]

In [497]:
oto_ids_0,oto_ids_1  = one_to_one(X_test_umb,'SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds')
otm_ids = one_to_many(X_test_umb,'SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds')
mto_ids = many_to_one(X_test_umb,'SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds')

In [498]:
X_test_oto = X_test_umb[(X_test_umb['SideB.ViewData.Side0_UniqueIds'].isin(oto_ids_0)) & (X_test_umb['SideA.ViewData.Side1_UniqueIds'].isin(oto_ids_1))]

In [499]:
X_test_oto.shape

(12, 34)

In [500]:
X_test_umb2 = X_test_umb[~(X_test_umb['SideB.ViewData.Side0_UniqueIds'].isin(X_test_oto['SideB.ViewData.Side0_UniqueIds'].unique()))]
X_test_umb2 = X_test_umb2.reset_index()
X_test_umb2 = X_test_umb2.drop('index',1)

In [501]:
X_test_umb2.shape

(141, 34)

In [502]:
#X_test_otm['SideA.ViewData.Side1_UniqueIds'].nunique()

In [503]:
#X_test_umb[X_test_umb['SideB.ViewData.Side0_UniqueIds'].isin(mto_ids)]

X_test_otm = X_test_umb2[X_test_umb2['SideA.ViewData.Side1_UniqueIds'].isin(otm_ids)]

X_test_umb3 = X_test_umb2[~((X_test_umb2['SideA.ViewData.Side1_UniqueIds'].isin(X_test_otm['SideA.ViewData.Side1_UniqueIds'].unique())) | (X_test_umb2['SideB.ViewData.Side0_UniqueIds'].isin(X_test_otm['SideB.ViewData.Side0_UniqueIds'].unique())))]
X_test_umb3 = X_test_umb3.reset_index()
X_test_umb3 = X_test_umb3.drop('index',1)


In [504]:
X_test_umb3.shape

(104, 34)

In [505]:
X_test_mto = X_test_umb3[X_test_umb3['SideB.ViewData.Side0_UniqueIds'].isin(mto_ids)]

In [506]:
X_test['ViewData.Combined Transaction Type'].nunique()

119

In [507]:
print("The shape of one to one is ", X_test_oto.shape)
print("The shape of one to many is ", X_test_otm.shape)
print("The shape of many to one is ", X_test_mto.shape)

The shape of one to one is  (12, 34)
The shape of one to many is  (25, 34)
The shape of many to one is  (17, 34)


In [508]:
X_test_mtm = X_test_umb3[~((X_test_umb3['SideB.ViewData.Side0_UniqueIds'].isin(X_test_mto['SideB.ViewData.Side0_UniqueIds'])) | (X_test_umb3['SideA.ViewData.Side1_UniqueIds'].isin(X_test_mto['SideA.ViewData.Side1_UniqueIds'])))]

In [509]:
print("The shape of many to many is ", X_test_mtm.shape)

The shape of many to many is  (69, 34)


In [510]:
print(get_relation(X_test_mtm, 'SideB.ViewData.Side0_UniqueIds','SideA.ViewData.Side1_UniqueIds'))

many-to-many


In [511]:
X_test_mtm['SideB.ViewData.Side0_UniqueIds'].value_counts()

22_153157570_Advent Geneva     8
20_153157570_Advent Geneva     8
7_153133289_Advent Geneva      6
5_153133289_Advent Geneva      6
569_153157547_Advent Geneva    4
567_153157547_Advent Geneva    4
127_153157410_Advent Geneva    4
568_153157547_Advent Geneva    4
175_153149314_Advent Geneva    3
192_153157410_Advent Geneva    3
47_153149314_Advent Geneva     3
46_153149314_Advent Geneva     3
184_153149314_Advent Geneva    2
41_153156073_Advent Geneva     2
1_153157570_Advent Geneva      2
108_153149314_Advent Geneva    1
45_153157587_Advent Geneva     1
47_153157587_Advent Geneva     1
25_153147536_Advent Geneva     1
107_153149314_Advent Geneva    1
34_153154996_Advent Geneva     1
158_153147536_Advent Geneva    1
Name: SideB.ViewData.Side0_UniqueIds, dtype: int64

In [512]:
X_test_umb[X_test_umb['SideB.ViewData.Side0_UniqueIds']=='255_153157549_Advent Geneva']

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT


In [533]:
X_test[(X_test['SideB.ViewData.Side0_UniqueIds']=='170_153156543_Advent Geneva')]

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
62995,-0.01,0.0,equity,pb,-19894980.0,0.0,eqty,cash,1,0,-19894979.99,0,5.0,0,0,RSCRSC,buybuy,pbcash,equityeqty,0,1,5ef987327781ec114072c2be,5ef987327781ec114072c2cc,SDB,146212937,OB,146213646,170_153156543_Advent Geneva,245_153156963_Morgan Stanley,UMB_One_to_One,0.071336,0.925818,0.001650,0.001196
62996,-19894980.00,0.0,money,pb,-19894980.0,0.0,eqty,cash,0,1,0.00,7,0.0,1,1,RSCRSC,wirebuy,pbcash,moneyeqty,0,0,5ef987327781ec114072c1da,5ef987327781ec114072c2cc,SDB,146212937,OB,146823047,170_153156543_Advent Geneva,164_153157570_Morgan Stanley,UMR_One_to_One,0.002139,0.002125,0.995127,0.000609


In [170]:
pd.pivot_table(X_test, values='ViewData.Combined Fund', index=['ViewData.Combined Transaction Type'],  columns=['Predicted_action'], aggfunc='count').reset_index().to_csv('trans_type_testing_distribution.csv')

In [547]:
X_test[(X_test['SideA.ViewData.Side1_UniqueIds']=='1141_153157324_CITI')& (X_test['probability_UMB']>0.05)]

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Asset_Type_Category,Combined_Investment_Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
32707,15601.22,0.0,cpn,5,13.90,0.0,cash,abs,1,1,-15587.32,0,0.0,0,0,QPQP,interestaccountingrelated,abs,cpncash,0,0,5ef985a17781ec1ac88552fc,5ef985a17781ec1ac8855647,OB,146797569,SPM,146796431,567_153157547_Advent Geneva,1141_153157324_CITI,No-Pair,0.903051,0.093363,0.001452,0.002134
32709,15601.22,0.0,cpn,5,-164.81,0.0,cash,abs,1,1,-15766.03,0,0.0,0,0,QPQP,interestaccountingrelated,abs,cpncash,0,0,5ef985a17781ec1ac88552fc,5ef985a17781ec1ac885564d,OB,146797575,SPM,146796431,568_153157547_Advent Geneva,1141_153157324_CITI,No-Pair,0.903051,0.093363,0.001452,0.002134
32711,15601.22,0.0,cpn,5,-112.78,0.0,cash,abs,1,1,-15714.00,0,0.0,0,0,QPQP,interestaccountingrelated,abs,cpncash,0,0,5ef985a17781ec1ac88552fc,5ef985a17781ec1ac8855652,OB,146797579,SPM,146796431,569_153157547_Advent Geneva,1141_153157324_CITI,No-Pair,0.903051,0.093363,0.001452,0.002134
32726,15601.22,0.0,cpn,5,-2643.08,0.0,cash,abs,1,1,-18244.30,0,0.0,0,0,QPQP,interestaccountingrelated,abs,cpncash,0,0,5ef985a17781ec1ac88552fc,5ef985a17781ec1ac8855544,SPM,146797532,SPM,146796431,570_153157547_Advent Geneva,1141_153157324_CITI,No-Pair,0.909871,0.086800,0.001358,0.001971
32747,15601.22,0.0,cpn,5,-1700.22,0.0,cash,abs,1,1,-17301.44,0,0.0,0,0,QPQP,interestaccountingrelated,abs,cpncash,0,0,5ef985a17781ec1ac88552fc,5ef985a17781ec1ac8855541,SPM,146797529,SPM,146796431,571_153157547_Advent Geneva,1141_153157324_CITI,No-Pair,0.909871,0.086800,0.001358,0.001971


In [296]:
X_test[(X_test['SideB.ViewData.Side0_UniqueIds']=='170_153156543_Advent Geneva') & (X_test['Predicted_action']=='UMB_One_to_One')]

,SideA.ViewData.B-P Net Amount,SideA.new_desc_cat,SideB.ViewData.Accounting Net Amount,SideB.new_desc_cat,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,Combined_Investment_Type,Combined_Asset_Type_Category,abs_amount_flag,tt_map_flag,SideB.Date,SideA.ViewData.Settle Date,SideB.ViewData.Settle Date,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,Predicted_action_2,probability_No_pair_2,probability_UMB_2
62991,-0.01,NA,-19894980.0,NA,1,0,-19894979.99,0,5.0,0,0,RSCRSC,buybuy,equityeqty,pbcash,0,1,4,2,4,0,1,5ef987327781ec114072c2be,5ef987327781ec114072c2cc,170_153156543_Advent Geneva,245_153156963_Morgan Stanley,SDB,146212937,OB,146213646,UMB_One_to_One,0.158539,0.832082,0.009379,UMB_One_to_One,0.031545,0.968455


In [ ]:
100_153157424_Morgan Stanley,101_153157424_Morgan Stanley,45_153157424_Morgan Stanley,46_153157424_Morgan Stanley

62_153157356_JP Morgan,66_153157356_JP Morgan,64_153157356_JP Morgan

125_153157587_Advent Geneva,126_153157587_Advent Geneva,127_153157587_Advent Geneva

check_ids_0 = ['602_153153147_Credit suisse','635_153157324_CITI','1294_153157324_CITI',
               '738_153157324_CITI','953_153157324_CITI','778_153157324_CITI','402_153157324_CITI',
               '1018_153157324_CITI','579_153157324_CITI','878_153157324_CITI','656_153157324_CITI',
               '620_153157324_CITI','593_153157324_CITI','629_153157324_CITI','13_153157547_CITI',
               '880_153157324_CITI','1340_153157324_CITI','828_153157324_CITI','1025_153157324_CITI',
               '860_153157324_CITI','943_153157324_CITI','994_153157324_CITI','658_153157324_CITI',
               '1021_153157324_CITI','737_153157324_CITI','362_153157324_CITI','954_153157324_CITI',
               '981_153157324_CITI','1141_153157324_CITI']



## Prediction Table

In [134]:
X_test.loc[(X_test['Predicted_action']=='UMR_One_to_One') & ((X_test['Amount_diff_2']!=0) | (X_test['Amount_diff_2']!=0)),'Predicted_action'] = 'Unrecognized' 

In [135]:
pd.set_option('max_columns', 100)

In [175]:
X_test[X_test['Predicted_action']=='UMB_One_to_One']

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Price,SideA.ViewData.Quantity,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Price,SideB.ViewData.Quantity,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
0,19005.12,0.0,0.0,0.0,reit,cash,54389.06,0.0000,0.00,0.0,reit,cash,0,0,35383.94,-92,-95.0,0,0,QPQP,returnofcapreturnofcap,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47e,OB,135683259,SMB-OB,135961673,383_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.079249,0.918283,0.001618,0.000850
1,19005.12,0.0,0.0,0.0,reit,cash,-108668.98,0.0918,0.00,1.0,reit,cash,0,0,-127674.10,-59,-61.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47d,OB,135961703,SMB-OB,135961673,299_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.377831,0.618096,0.003728,0.000345
2,19005.12,0.0,0.0,0.0,reit,cash,-108668.98,0.0918,0.00,1.0,reit,cash,0,0,-127674.10,-30,-33.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e487,OB,135961702,SMB-OB,135961673,77_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.361014,0.635066,0.003587,0.000333
3,19005.12,0.0,0.0,0.0,reit,cash,62377.36,0.0918,0.00,0.0,reit,cash,0,0,43372.24,-59,-61.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e499,SMB-OB,135961699,SMB-OB,135961673,300_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.228325,0.768779,0.001997,0.000899
5,19005.12,0.0,0.0,0.0,reit,cash,62377.36,0.0918,0.00,0.0,reit,cash,0,0,43372.24,-92,-95.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e480,OB,135683249,SMB-OB,135961673,319_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.091870,0.905653,0.001601,0.000876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77568,4711562.67,0.0,0.0,0.0,nan,nan,25872893.30,1.0000,25872893.30,0.0,cash,cash,0,0,21161330.63,80,80.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec114077694e,5ef9c3167781ec1140776754,OB,136564417,OB,145890838,6_153146143_Advent Geneva,3_153156665_Wells Fargo,UMB_One_to_One,0.401575,0.594508,0.002944,0.000973
77575,4711562.67,0.0,0.0,0.0,nan,nan,81254788.20,1.0000,81254788.20,0.0,cash,cash,0,0,76543225.53,171,171.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec114077694e,5ef9c3167781ec1140776760,OB,125420837,OB,145890838,5_153133289_Advent Geneva,3_153156665_Wells Fargo,UMB_One_to_One,0.479200,0.515311,0.004119,0.001371
77964,4266917.82,0.0,0.0,0.0,nan,nan,25872893.30,1.0000,25872893.30,0.0,cash,cash,0,0,21605975.48,87,87.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec1140776754,OB,136564417,OB,146861678,6_153146143_Advent Geneva,2_153157651_Wells Fargo,UMB_One_to_One,0.401575,0.594508,0.002944,0.000973
77971,4266917.82,0.0,0.0,0.0,nan,nan,81254788.20,1.0000,81254788.20,0.0,cash,cash,0,0,76987870.38,178,178.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec1140776760,OB,125420837,OB,146861678,5_153133289_Advent Geneva,2_153157651_Wells Fargo,UMB_One_to_One,0.479200,0.515311,0.004119,0.001371


In [137]:
test_file[['SideA.ViewData.Price','SideA.ViewData.Quantity', 'SideA.ViewData.Transaction Type',
 'SideA.ViewData.Trade Date','SideA.ViewData.Settle Date', 'SideA.ISIN_NA','SideA.ViewData.CUSIP','SideA.ViewData.ISIN','SideB.ViewData.BreakID_B_side','SideA.ViewData.BreakID_A_side']]

,SideA.ViewData.Price,SideA.ViewData.Quantity,SideA.ViewData.Transaction Type,SideA.ViewData.Trade Date,SideA.ViewData.Settle Date,SideA.ISIN_NA,SideA.ViewData.CUSIP,SideA.ViewData.ISIN,SideB.ViewData.BreakID_B_side,SideA.ViewData.BreakID_A_side
0,NaN,0.0,returnofcap,07-30-2019,08-09-2019,0,00123Q104,US00123Q1040,135683259,135961673
1,NaN,0.0,returnofcap,07-30-2019,08-09-2019,0,00123Q104,US00123Q1040,135961703,135961673
2,NaN,0.0,returnofcap,07-30-2019,08-09-2019,0,00123Q104,US00123Q1040,135961702,135961673
3,NaN,0.0,returnofcap,07-30-2019,08-09-2019,0,00123Q104,US00123Q1040,135961699,135961673
4,NaN,0.0,returnofcap,07-30-2019,08-09-2019,0,00123Q104,US00123Q1040,137428616,135961673
...,...,...,...,...,...,...,...,...,...,...
84336,NaN,NaN,cash,06-26-2020,06-26-2020,1,nan,nan,143651932,146861678
84337,NaN,NaN,cash,06-26-2020,06-26-2020,1,nan,nan,143651933,146861678
84338,NaN,NaN,cash,06-26-2020,06-26-2020,1,nan,nan,143651937,146861678
84339,NaN,NaN,cash,06-26-2020,06-26-2020,1,nan,nan,136564411,146861678


In [138]:
###### Probability filter for UMT and UMB ################

#X_test.loc[(X_test['Predicted_action']=='UMT_One_to_One') & (X_test['probability_UMT']<0.90) & (X_test['probability_No_pair']>0.05),'Predicted_action'] = 'No-Pair' 

#X_test.loc[(X_test['Predicted_action']=='UMB_One_to_One') & (X_test['probability_UMB']<0.75) & (X_test['probability_No_pair']>0.2),'Predicted_action'] = 'No-Pair' 

#X_test.loc[(X_test['Predicted_action']=='UMR_One_to_One') & (X_test['probability_UMR']<0.90) & (X_test['probability_No_pair']>0.05),'Predicted_actionX_test.loc[(X_test['Predicted_action']=='No-Pair') & (X_test['probability_No_pair']<0.9) & (X_test['probability_UMB']>0.05),'Predicted_action'] = 'UMB_One_to_One' 


#X_test.loc[(X_test['Predicted_action']=='No-Pair') & (X_test['probability_No_pair']<0.95) & (X_test['probability_UMB']>0.05),'Predicted_action'] = 'UMB_One_to_One' 

#X_test.loc[(X_test['Predicted_action']=='UMR_One_to_One') & (X_test['Settle_date_diff']>4),'Predicted_action'] = 'No-Pair' 
#X_test.loc[(X_test['Predicted_action']=='UMR_One_to_One') & (X_test['Settle_date_diff']<-4),'Predicted_action'] = 'No-Pair' 

In [139]:
#X_test.loc[(X_test['SideB.ViewData.Status']=='SDB') & (X_test['SideA.ViewData.Status']=='OB') & (X_test['Predicted_action']=='No-Pair'),'Predicted_action'] = 'SDB/Open Break'

In [176]:
prediction_table =  X_test.groupby('SideB.ViewData.BreakID_B_side')['Predicted_action'].unique().reset_index()

In [177]:
prediction_table

,SideB.ViewData.BreakID_B_side,Predicted_action
0,82817946,"[No-Pair, UMB_One_to_One]"
1,125420837,"[No-Pair, UMB_One_to_One]"
2,125420838,[No-Pair]
3,125420840,"[No-Pair, UMB_One_to_One]"
4,125420841,"[No-Pair, UMB_One_to_One]"
...,...,...
702,146830534,"[UMB_One_to_One, UMR_One_to_One, No-Pair]"
703,146865830,"[UMB_One_to_One, UMR_One_to_One, No-Pair]"
704,146865831,"[No-Pair, UMR_One_to_One]"
705,146865832,"[No-Pair, UMR_One_to_One, UMB_One_to_One]"


In [178]:
#X_test[X_test['SideB.ViewData.BreakID_B_side']==82817946]

In [179]:
#prob1 = X_test.groupby('SideB.ViewData.BreakID_B_side')['probability_No_pair'].mean().reset_index()

In [180]:
prediction_table['len'] = prediction_table['Predicted_action'].str.len()

In [182]:
prediction_table['len'] = prediction_table['Predicted_action'].str.len()
prediction_table['No_Pair_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'No-Pair' in x else 0)
prediction_table['UMB_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMB_One_to_One' in x else 0)
prediction_table['UMT_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMT_One_to_One' in x else 0)
prediction_table['UMR_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMR_One_to_One' in x else 0)


In [183]:
prediction_table

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag
0,82817946,"[No-Pair, UMB_One_to_One]",2,1,1,0,0
1,125420837,"[No-Pair, UMB_One_to_One]",2,1,1,0,0
2,125420838,[No-Pair],1,1,0,0,0
3,125420840,"[No-Pair, UMB_One_to_One]",2,1,1,0,0
4,125420841,"[No-Pair, UMB_One_to_One]",2,1,1,0,0
...,...,...,...,...,...,...,...
702,146830534,"[UMB_One_to_One, UMR_One_to_One, No-Pair]",3,1,1,0,1
703,146865830,"[UMB_One_to_One, UMR_One_to_One, No-Pair]",3,1,1,0,1
704,146865831,"[No-Pair, UMR_One_to_One]",2,1,0,0,1
705,146865832,"[No-Pair, UMR_One_to_One, UMB_One_to_One]",3,1,1,0,1


In [184]:
X_test[X_test['Predicted_action']=='UMB_One_to_One']

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Price,SideA.ViewData.Quantity,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Price,SideB.ViewData.Quantity,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
0,19005.12,0.0,0.0,0.0,reit,cash,54389.06,0.0000,0.00,0.0,reit,cash,0,0,35383.94,-92,-95.0,0,0,QPQP,returnofcapreturnofcap,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47e,OB,135683259,SMB-OB,135961673,383_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.079249,0.918283,0.001618,0.000850
1,19005.12,0.0,0.0,0.0,reit,cash,-108668.98,0.0918,0.00,1.0,reit,cash,0,0,-127674.10,-59,-61.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47d,OB,135961703,SMB-OB,135961673,299_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.377831,0.618096,0.003728,0.000345
2,19005.12,0.0,0.0,0.0,reit,cash,-108668.98,0.0918,0.00,1.0,reit,cash,0,0,-127674.10,-30,-33.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e487,OB,135961702,SMB-OB,135961673,77_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.361014,0.635066,0.003587,0.000333
3,19005.12,0.0,0.0,0.0,reit,cash,62377.36,0.0918,0.00,0.0,reit,cash,0,0,43372.24,-59,-61.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e499,SMB-OB,135961699,SMB-OB,135961673,300_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.228325,0.768779,0.001997,0.000899
5,19005.12,0.0,0.0,0.0,reit,cash,62377.36,0.0918,0.00,0.0,reit,cash,0,0,43372.24,-92,-95.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e480,OB,135683249,SMB-OB,135961673,319_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.091870,0.905653,0.001601,0.000876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77568,4711562.67,0.0,0.0,0.0,nan,nan,25872893.30,1.0000,25872893.30,0.0,cash,cash,0,0,21161330.63,80,80.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec114077694e,5ef9c3167781ec1140776754,OB,136564417,OB,145890838,6_153146143_Advent Geneva,3_153156665_Wells Fargo,UMB_One_to_One,0.401575,0.594508,0.002944,0.000973
77575,4711562.67,0.0,0.0,0.0,nan,nan,81254788.20,1.0000,81254788.20,0.0,cash,cash,0,0,76543225.53,171,171.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec114077694e,5ef9c3167781ec1140776760,OB,125420837,OB,145890838,5_153133289_Advent Geneva,3_153156665_Wells Fargo,UMB_One_to_One,0.479200,0.515311,0.004119,0.001371
77964,4266917.82,0.0,0.0,0.0,nan,nan,25872893.30,1.0000,25872893.30,0.0,cash,cash,0,0,21605975.48,87,87.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec1140776754,OB,136564417,OB,146861678,6_153146143_Advent Geneva,2_153157651_Wells Fargo,UMB_One_to_One,0.401575,0.594508,0.002944,0.000973
77971,4266917.82,0.0,0.0,0.0,nan,nan,81254788.20,1.0000,81254788.20,0.0,cash,cash,0,0,76987870.38,178,178.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec1140776760,OB,125420837,OB,146861678,5_153133289_Advent Geneva,2_153157651_Wells Fargo,UMB_One_to_One,0.479200,0.515311,0.004119,0.001371


In [190]:
X_test

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Price,SideA.ViewData.Quantity,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Price,SideB.ViewData.Quantity,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT
0,19005.12,0.0,0.0,0.0,reit,cash,54389.06,0.0000,0.00,0.0,reit,cash,0,0,35383.94,-92,-95.0,0,0,QPQP,returnofcapreturnofcap,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47e,OB,135683259,SMB-OB,135961673,383_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.079249,0.918283,0.001618,0.000850
1,19005.12,0.0,0.0,0.0,reit,cash,-108668.98,0.0918,0.00,1.0,reit,cash,0,0,-127674.10,-59,-61.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47d,OB,135961703,SMB-OB,135961673,299_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.377831,0.618096,0.003728,0.000345
2,19005.12,0.0,0.0,0.0,reit,cash,-108668.98,0.0918,0.00,1.0,reit,cash,0,0,-127674.10,-30,-33.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e487,OB,135961702,SMB-OB,135961673,77_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.361014,0.635066,0.003587,0.000333
3,19005.12,0.0,0.0,0.0,reit,cash,62377.36,0.0918,0.00,0.0,reit,cash,0,0,43372.24,-59,-61.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e499,SMB-OB,135961699,SMB-OB,135961673,300_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,UMB_One_to_One,0.228325,0.768779,0.001997,0.000899
4,19005.12,0.0,0.0,0.0,reit,cash,-472173.83,0.1855,0.00,1.0,reit,corp,0,0,-491178.95,-59,-83.0,0,0,QPQP,returnofcapdividend,0,0,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e484,OB,137428616,SMB-OB,135961673,24_153147536_Advent Geneva,6_153139764_Bank of America Merrill Lynch,No-Pair,0.986232,0.013456,0.000271,0.000042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77976,4266917.82,0.0,0.0,0.0,nan,nan,21743243.42,1.0000,21743243.42,0.0,cash,cash,0,0,17476325.60,31,31.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec1140776757,OB,143651933,OB,146861678,11_153154447_Advent Geneva,2_153157651_Wells Fargo,UMB_One_to_One,0.490680,0.505435,0.002794,0.001091
77977,4266917.82,0.0,0.0,0.0,nan,nan,-6961.63,1.0000,6961.63,0.0,cash,cash,0,0,-4273879.45,56,56.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec1140776761,OB,143651932,OB,146861678,9_153154447_Advent Geneva,2_153157651_Wells Fargo,No-Pair,0.932503,0.065746,0.001421,0.000330
77978,4266917.82,0.0,0.0,0.0,nan,nan,212613.13,1.0000,212613.13,0.0,cash,cash,0,0,-4054304.69,56,56.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec114077675e,OB,143651926,OB,146861678,8_153154447_Advent Geneva,2_153157651_Wells Fargo,No-Pair,0.904982,0.092927,0.001621,0.000470
77979,4266917.82,0.0,0.0,0.0,nan,nan,-282.02,1.0000,282.02,0.0,cash,cash,0,0,-4267199.84,178,178.0,1,1,QPQP,cashtransfer,1,0,5ef9c3167781ec1140776997,5ef9c3167781ec114077675c,OB,125420843,OB,146861678,8_153133289_Advent Geneva,2_153157651_Wells Fargo,No-Pair,0.904954,0.092874,0.001749,0.000424


In [185]:
umr_array = X_test[X_test['Predicted_action']=='UMR_One_to_One'].groupby(['SideB.ViewData.BreakID_B_side'])['SideA.ViewData.BreakID_A_side'].unique().reset_index()
umt_array = X_test[X_test['Predicted_action']=='UMT_One_to_One'].groupby(['SideB.ViewData.BreakID_B_side'])['SideA.ViewData.BreakID_A_side'].unique().reset_index()
umb_array = X_test[X_test['Predicted_action']=='UMB_One_to_One'].groupby(['SideB.ViewData.BreakID_B_side'])['SideA.ViewData.BreakID_A_side'].unique().reset_index()

In [191]:
umr_array_ids = X_test[X_test['Predicted_action']=='UMR_One_to_One'].groupby(['SideB.ViewData.Side0_UniqueIds'])['SideA.ViewData.Side1_UniqueIds'].unique().reset_index()
umt_array_ids = X_test[X_test['Predicted_action']=='UMT_One_to_One'].groupby(['SideB.ViewData.Side0_UniqueIds'])['SideA.ViewData.Side1_UniqueIds'].unique().reset_index()
umb_array_ids = X_test[X_test['Predicted_action']=='UMB_One_to_One'].groupby(['SideB.ViewData.Side0_UniqueIds'])['SideA.ViewData.Side1_UniqueIds'].unique().reset_index()

In [186]:
umr_array.columns = ['SideB.ViewData.BreakID_B_side', 'Predicted_UMR_array']
umt_array.columns = ['SideB.ViewData.BreakID_B_side', 'Predicted_UMT_array']
umb_array.columns = ['SideB.ViewData.BreakID_B_side', 'Predicted_UMB_array']

In [192]:
umr_array_ids.columns = ['SideB.ViewData.Side0_UniqueIds', 'Predicted_UMR_ids']
umt_array_ids.columns = ['SideB.ViewData.Side0_UniqueIds', 'Predicted_UMT_ids']
umb_array_ids.columns = ['SideB.ViewData.Side0_UniqueIds', 'Predicted_UMB_ids']

In [187]:
prediction_table = pd.merge(prediction_table,umr_array, on='SideB.ViewData.BreakID_B_side', how='left' )
prediction_table = pd.merge(prediction_table,umt_array, on='SideB.ViewData.BreakID_B_side', how='left' )
prediction_table = pd.merge(prediction_table,umb_array, on='SideB.ViewData.BreakID_B_side', how='left' )

In [197]:
prediction_table_ids = X_test.groupby('SideB.ViewData.Side0_UniqueIds')['Predicted_action'].unique().reset_index()

In [198]:
prediction_table_ids = pd.merge(prediction_table_ids,umr_array_ids, on='SideB.ViewData.Side0_UniqueIds', how='left')
prediction_table_ids = pd.merge(prediction_table_ids,umt_array_ids, on='SideB.ViewData.Side0_UniqueIds', how='left')
prediction_table_ids = pd.merge(prediction_table_ids,umb_array_ids, on='SideB.ViewData.Side0_UniqueIds', how='left')

In [221]:
check_ids = ['16_153157591_Advent Geneva','30_153157422_Advent Geneva','78_153156543_Advent Geneva',
             '182_153156543_Advent Geneva','170_153156543_Advent Geneva','567_153157547_Advent Geneva']


check_ids_0 = ['602_153153147_Credit suisse','635_153157324_CITI','1294_153157324_CITI',
               '738_153157324_CITI','953_153157324_CITI','778_153157324_CITI','402_153157324_CITI',
               '1018_153157324_CITI','579_153157324_CITI','878_153157324_CITI','656_153157324_CITI',
               '620_153157324_CITI','593_153157324_CITI','629_153157324_CITI','13_153157547_CITI',
               '880_153157324_CITI','1340_153157324_CITI','828_153157324_CITI','1025_153157324_CITI',
               '860_153157324_CITI','943_153157324_CITI','994_153157324_CITI','658_153157324_CITI',
               '1021_153157324_CITI','737_153157324_CITI','362_153157324_CITI','954_153157324_CITI',
               '981_153157324_CITI','1141_153157324_CITI']


In [224]:
#prediction_table_ids[prediction_table_ids['SideA.ViewData.Side1_UniqueIds'].isin(check_ids_0)]

In [251]:
for i, ids in enumerate(check_ids_0):
    if i==9:
        dummy = X_test[X_test['SideA.ViewData.Side1_UniqueIds']==ids]

In [252]:
dummy[dummy['Predicted_action']!='No-Pair'][['SideA.ViewData.Side1_UniqueIds','SideB.ViewData.Side0_UniqueIds']]

,SideA.ViewData.Side1_UniqueIds,SideB.ViewData.Side0_UniqueIds
35827,878_153157324_CITI,568_153157547_Advent Geneva
35829,878_153157324_CITI,569_153157547_Advent Geneva
35835,878_153157324_CITI,567_153157547_Advent Geneva


In [275]:
#prediction_table
X_test[(X_test['SideA.ViewData.Side1_UniqueIds']=='878_153157324_CITI') & (X_test['probability_UMB']>0.02)]


,SideA.ViewData.B-P Net Amount,SideA.ViewData.Price,SideA.ViewData.Quantity,SideA.ViewData.Cancel Flag,SideA.ViewData.Investment Type,SideA.ViewData.Asset Type Category,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Price,SideB.ViewData.Quantity,SideB.ViewData.Cancel Flag,SideB.ViewData.Investment Type,SideB.ViewData.Asset Type Category,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT,Predicted_action_2,probability_No_pair_2,probability_Pair_2
35711,986.67,0.0,0.0,0.0,abs,abs,537.17,0.0,0.0,0.0,abs,abs,1,1,-449.50,0,0.0,0,0,QPQP,interestinterest,0,1,5ef985a17781ec1ac885531d,5ef985a17781ec1ac885531d,SMB-OB,146796493,SMB-OB,146796493,338_153157547_Advent Geneva,878_153157324_CITI,No-Pair,0.963884,0.035924,0.000111,0.000080,No-Pair,0.871948,0.128052
35712,986.67,0.0,0.0,0.0,abs,abs,449.50,0.0,0.0,0.0,abs,abs,1,1,-537.17,0,0.0,0,0,QPQP,interestinterest,0,1,5ef985a17781ec1ac885531d,5ef985a17781ec1ac88555ad,OB,146796513,SMB-OB,146796493,337_153157547_Advent Geneva,878_153157324_CITI,No-Pair,0.962973,0.036836,0.000111,0.000080,No-Pair,0.876787,0.123213
35827,986.67,0.0,0.0,0.0,abs,abs,-164.81,0.0,0.0,0.0,cash,abs,1,1,-1151.48,0,0.0,0,0,QPQP,interestaccountingrelated,0,0,5ef985a17781ec1ac885531d,5ef985a17781ec1ac885564d,OB,146797575,SMB-OB,146796493,568_153157547_Advent Geneva,878_153157324_CITI,UMB_One_to_One,0.431304,0.554970,0.011421,0.002306,Pair,0.311332,0.688668
35829,986.67,0.0,0.0,0.0,abs,abs,-112.78,0.0,0.0,0.0,cash,abs,1,1,-1099.45,0,0.0,0,0,QPQP,interestaccountingrelated,0,0,5ef985a17781ec1ac885531d,5ef985a17781ec1ac8855652,OB,146797579,SMB-OB,146796493,569_153157547_Advent Geneva,878_153157324_CITI,UMB_One_to_One,0.449637,0.529307,0.018359,0.002697,Pair,0.311055,0.688945
35835,986.67,0.0,0.0,0.0,abs,abs,13.90,0.0,0.0,0.0,cash,abs,1,1,-972.77,0,0.0,0,0,QPQP,interestaccountingrelated,0,0,5ef985a17781ec1ac885531d,5ef985a17781ec1ac8855647,OB,146797569,SMB-OB,146796493,567_153157547_Advent Geneva,878_153157324_CITI,UMB_One_to_One,0.443420,0.536210,0.017780,0.002589,Pair,0.322595,0.677405
35866,986.67,0.0,0.0,0.0,abs,abs,-1700.22,0.0,0.0,0.0,cash,abs,1,1,-2686.89,0,0.0,0,0,QPQP,interestaccountingrelated,0,0,5ef985a17781ec1ac885531d,5ef985a17781ec1ac8855541,SPM,146797529,SMB-OB,146796493,571_153157547_Advent Geneva,878_153157324_CITI,No-Pair,0.506765,0.481751,0.009864,0.001619,Pair,0.416081,0.583919
35867,986.67,0.0,0.0,0.0,abs,abs,-2643.08,0.0,0.0,0.0,cash,abs,1,1,-3629.75,0,0.0,0,0,QPQP,interestaccountingrelated,0,0,5ef985a17781ec1ac885531d,5ef985a17781ec1ac8855544,SPM,146797532,SMB-OB,146796493,570_153157547_Advent Geneva,878_153157324_CITI,No-Pair,0.528985,0.460920,0.008599,0.001496,No-Pair,0.500526,0.499474


In [189]:
prediction_table

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array
0,82817946,"[No-Pair, UMB_One_to_One]",2,1,1,0,0,NaN,NaN,[146799635]
1,125420837,"[No-Pair, UMB_One_to_One]",2,1,1,0,0,NaN,NaN,"[121293439, 121293440, 121293441, 121293442, 1..."
2,125420838,[No-Pair],1,1,0,0,0,NaN,NaN,NaN
3,125420840,"[No-Pair, UMB_One_to_One]",2,1,1,0,0,NaN,NaN,[121293445]
4,125420841,"[No-Pair, UMB_One_to_One]",2,1,1,0,0,NaN,NaN,[121293443]
...,...,...,...,...,...,...,...,...,...,...
702,146830534,"[UMB_One_to_One, UMR_One_to_One, No-Pair]",3,1,1,0,1,[146382937],NaN,"[137756357, 137756367]"
703,146865830,"[UMB_One_to_One, UMR_One_to_One, No-Pair]",3,1,1,0,1,[146682038],NaN,"[146495242, 146495243, 146682040, 146865838, 1..."
704,146865831,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146682043],NaN,NaN
705,146865832,"[No-Pair, UMR_One_to_One, UMB_One_to_One]",3,1,1,0,1,[146682035],NaN,"[146682037, 146682042, 146682044, 146682049]"


In [153]:
prediction_table['Final_prediction'] = prediction_table.apply(lambda x: 'UMR_One_to_One' if x['UMR_flag']==1 else('UMT_One_to_One' if x['len']==1 and x['UMT_flag']==1 else('UMB_One_to_UMB' if x['len']==1 and x['UMB_flag']==1 else('No-Pair' if x['len']==1 else 'Undecided'))), axis=1)

In [154]:
prediction_table['Final_prediction'].value_counts()

No-Pair           552
Undecided          83
UMR_One_to_One     65
UMT_One_to_One      2
UMB_One_to_UMB      2
Name: Final_prediction, dtype: int64

In [155]:
prediction_table['Final_prediction'].value_counts()

No-Pair           552
Undecided          83
UMR_One_to_One     65
UMT_One_to_One      2
UMB_One_to_UMB      2
Name: Final_prediction, dtype: int64

In [156]:
prediction_table['UMT_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMT_One_to_One' in x else 0)
prediction_table['UMB_flag'] = prediction_table['Predicted_action'].apply(lambda x: 1 if 'UMB_One_to_One' in x else 0)

In [157]:
prediction_table.loc[(prediction_table['UMB_flag']==1) & (prediction_table['len']==2),'Final_prediction']='UMB_One_to_One'
prediction_table.loc[(prediction_table['UMT_flag']==1) & (prediction_table['len']==2),'Final_prediction']='UMT_One_to_One'


In [158]:
prediction_table.loc[(prediction_table['Final_prediction']=='Undecided') & (prediction_table['len']==2),'Final_prediction']='No-Pair/Unrecognized'

In [159]:
prediction_table.loc[(prediction_table['Final_prediction']=='Undecided') & (prediction_table['UMT_flag']==1),'Final_prediction']='UMT_One_to_One'

In [160]:
prediction_table.loc[(prediction_table['Final_prediction']=='Undecided') & (prediction_table['UMB_flag']==1),'Final_prediction']='UMB_One_to_One'

In [161]:
prediction_table['Final_prediction'].value_counts()

No-Pair                 552
UMB_One_to_One           71
UMR_One_to_One           65
UMT_One_to_One           13
UMB_One_to_UMB            2
No-Pair/Unrecognized      1
Name: Final_prediction, dtype: int64

In [162]:
prediction_table[prediction_table['SideB.ViewData.BreakID_B_side'] == 1334352050]

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction


In [163]:
#X_test[(X_test['Predicted_action']=='UMR_One_to_One') & (X_test['SideB.ViewData.BreakID_B_side']==1346769635)]

In [164]:
prediction_table.loc[prediction_table['Final_prediction']=='UMR_One_to_One', 'Final_predicted_break'] = prediction_table.loc[prediction_table['Final_prediction']=='UMR_One_to_One', 'Predicted_UMR_array']

In [165]:
prediction_table.loc[prediction_table['Final_prediction']=='UMT_One_to_One', 'Final_predicted_break'] = prediction_table.loc[prediction_table['Final_prediction']=='UMT_One_to_One', 'Predicted_UMT_array']
prediction_table.loc[prediction_table['Final_prediction']=='UMB_One_to_One', 'Final_predicted_break'] = prediction_table.loc[prediction_table['Final_prediction']=='UMB_One_to_One', 'Predicted_UMB_array']
#prediction_table.loc[prediction_table['Final_prediction']=='No-Pair', 'Final_predicted_break'] = prediction_table.loc[prediction_table['Final_prediction']=='No-Pair', '']

In [166]:
prediction_table['predicted_break_len'] = prediction_table['Final_predicted_break'].str.len()

In [167]:
prediction_table[prediction_table['SideB.ViewData.BreakID_B_side'] == 1334352050]

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len


In [168]:
#prediction_table[(prediction_table['predicted_break_len']>1) & (prediction_table['Final_prediction']=='UMT_One_to_One')]

In [169]:
#prediction_table[['SideB.ViewData.BreakID_B_side', 'Final_prediction', 'Final_predicted_break']]

In [170]:
X_test['prob_key'] = X_test['SideB.ViewData.BreakID_B_side'].astype(str) + X_test['Predicted_action']
prediction_table['prob_key'] = prediction_table['SideB.ViewData.BreakID_B_side'].astype(str) + prediction_table['Final_prediction']

In [171]:
user_prob = X_test.groupby('prob_key')[['probability_UMR','probability_UMT','probability_UMB']].max().reset_index()
open_prob = X_test.groupby('prob_key')['probability_No_pair'].mean().reset_index()

In [172]:
#prediction_table = prediction_table.drop(,1)

prediction_table = pd.merge(prediction_table,user_prob, on='prob_key', how='left')
prediction_table = pd.merge(prediction_table,open_prob, on='prob_key', how='left')

In [173]:
prediction_table = prediction_table.drop('prob_key',1)

In [174]:
prediction_table[prediction_table['SideB.ViewData.BreakID_B_side'] == 1334352050]

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair


In [175]:
prediction_table

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair
0,82817946,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.052263,0.002674,0.006601,0.993558
1,125420837,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000025,0.000020,0.002485,0.999850
2,125420838,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000047,0.000034,0.001343,0.999901
3,125420840,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000057,0.000053,0.001778,0.999840
4,125420841,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000222,0.000229,0.000495,0.999923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,146830534,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146382937],NaN,NaN,UMR_One_to_One,[146382937],1.0,0.997731,0.001426,0.000786,0.000057
700,146865830,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146682038],NaN,NaN,UMR_One_to_One,[146682038],1.0,0.999923,0.000071,0.000003,0.000003
701,146865831,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146682043],NaN,NaN,UMR_One_to_One,[146682043],1.0,0.999919,0.000075,0.000003,0.000003
702,146865832,"[No-Pair, UMR_One_to_One, UMB_One_to_One]",3,1,1,0,1,[146682035],NaN,"[146682042, 146682049]",UMR_One_to_One,[146682035],1.0,0.999919,0.000075,0.000003,0.000003


In [176]:
prediction_table = pd.merge(prediction_table, X_test[['SideB.ViewData.BreakID_B_side','SideA.ViewData._ID','SideB.ViewData._ID']].drop_duplicates(['SideB.ViewData.BreakID_B_side','SideB.ViewData._ID']), on ='SideB.ViewData.BreakID_B_side', how='left')

In [177]:
prediction_table[prediction_table['SideB.ViewData.BreakID_B_side'] ==1334352050]

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID


In [178]:
prediction_table.dtypes

SideB.ViewData.BreakID_B_side      int64
Predicted_action                  object
len                                int64
No_Pair_flag                       int64
UMB_flag                           int64
UMT_flag                           int64
UMR_flag                           int64
Predicted_UMR_array               object
Predicted_UMT_array               object
Predicted_UMB_array               object
Final_prediction                  object
Final_predicted_break             object
predicted_break_len              float64
probability_UMR                  float64
probability_UMT                  float64
probability_UMB                  float64
probability_No_pair              float64
SideA.ViewData._ID                object
SideB.ViewData._ID                object
dtype: object

## Merging PB side Break ID's

In [5065]:
#pb_break_ids = prediction_table[~prediction_table['Final_predicted_break'].isnull()][['Final_prediction','Final_predicted_break']]

In [4358]:
#pb_break_ids = pb_break_ids.reset_index()
#pb_break_ids = pb_break_ids.drop('index',1)

In [1706]:
#pb_break_ids['Final_predicted_break'] = pb_break_ids['Final_predicted_break'].apply(lambda x: str(x).replace("[",''))
#pb_break_ids['Final_predicted_break'] = pb_break_ids['Final_predicted_break'].apply(lambda x: str(x).replace("]",''))

In [2444]:
#pb_break_ids['Final_predicted_break'].unique()

In [1708]:
#id_list = []
#id_list2 = []

#for i in pb_break_ids['Final_predicted_break'].unique():
#    id_list.append(i.split(' '))
#for j in np.concatenate(id_list,axis=0):
#    if j!='':
#        id_list2.append(j.replace("\n",''))

In [1709]:
#new_ob_ids =[]
#
#for i in X_test['SideA.ViewData.BreakID_A_side'].astype(str).unique():
#    if i not in np.array(id_list2,dtype="O"):
#        new_ob_ids.append(i)

In [1710]:
#prediction_table2 = pd.DataFrame(np.array(new_ob_ids))

In [1711]:
#prediction_table2.columns = ['SideB.ViewData.BreakID_B_side']

In [1712]:
#prediction_table2['Final_prediction'] = 'No-Pair'

In [1713]:
#prediction_table2['Side'] = 'P-B Side'

In [1714]:
#prediction_table['Side'] = 'Accounting Side'

In [2074]:
#prediction_table3 = prediction_table

In [2442]:
#prediction_table3 = pd.concat([prediction_table, prediction_table2], axis=0)

In [1716]:
#prediction_table3 = prediction_table3.reset_index()
#prediction_table3 = prediction_table3.drop('index',1)

In [1717]:
#prediction_table3 = prediction_table3[prediction_table.columns]

In [2443]:
#prediction_table3[['SideB.ViewData.BreakID_B_side', 'Final_prediction', 'Final_predicted_break','Side']]

In [1719]:
#ids_for_comment = prediction_table3[['SideB.ViewData.BreakID_B_side', 'Final_prediction', 'Final_predicted_break','Side']]

In [2445]:
#ids_for_comment

In [1721]:
#ids_for_comment.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/Input for comment prediction/prediction_table_testing_HST_RecData_125_1159652110_06-19-2020.csv')

In [6161]:
prediction_table[prediction_table['SideB.ViewData.BreakID_B_side'] == 144895568]

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID
592,144895568,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,"[144895648, 144895650]",NaN,NaN,UMR_One_to_One,"[144895648, 144895650]",2.0,0.955154,0.024246,0.007381,0.013218,5ee7361e7781ec1c306bd0f2,5ee7361e7781ec1c306bd300


In [6169]:
data = pd.read_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Soros/JuneData/MEO/MeoCollections_SOROS.MEO_HST_RecData_153_2020-06-11.csv')

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,30,54,55,56,57,86,113,115,116,123,128,173,183,193,211,269,273,275,281) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6175]:
data['ViewData.Status'].value_counts()

OB     1604
SMR     421
SPM     121
SDB      69
SMB      54
SMT      33
CNF      15
UMB      11
CMF       9
UDB       8
UOB       4
Name: ViewData.Status, dtype: int64

In [6174]:
data[data['ViewData.Side1_UniqueIds']=='356_153155545_Credit suisse']['ViewData.Status']

961    CMF
Name: ViewData.Status, dtype: object

In [6173]:
X_test[X_test['SideA.ViewData.Side1_UniqueIds']=='356_153155545_Credit suisse']

X_test[X_test['SideB.ViewData.BreakID_B_side']==144895379]

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Price,SideA.ViewData.Quantity,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Price,SideB.ViewData.Quantity,Trade_Date_match,Settle_Date_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT,prob_key
12297,-55104.31,0.0,0.0,-9013072.5,105.75,8523000.0,0,0,-8957968.19,-813,-814,1,1,QPQP,taxrepo,0,0,5ee7361e7781ec1c306bd0f2,5ee7361e7781ec1c306bd223,SPM,144895379,OB,29780825,145_153155545_Advent Geneva,2_36266_Credit suisse,No-Pair,0.980653,0.006041,0.007255,0.006051,144895379No-Pair
12351,48151.50,0.0,0.0,-9013072.5,105.75,8523000.0,0,0,-9061224.00,-57,-58,0,0,QPQP,intrepo,0,0,5ee7361e7781ec1c306bd0ec,5ee7361e7781ec1c306bd223,SPM,144895379,OB,136676054,145_153155545_Advent Geneva,388_153146279_Credit suisse,No-Pair,0.981467,0.005877,0.006882,0.005774,144895379No-Pair
12405,79312.50,0.0,0.0,-9013072.5,105.75,8523000.0,0,0,-9092385.00,-57,-58,0,0,QPQP,intrepo,0,0,5ee7361e7781ec1c306bd0ee,5ee7361e7781ec1c306bd223,SPM,144895379,OB,136676077,145_153155545_Advent Geneva,390_153146279_Credit suisse,No-Pair,0.981467,0.005877,0.006882,0.005774,144895379No-Pair
12459,52875.00,0.0,0.0,-9013072.5,105.75,8523000.0,0,0,-9065947.50,-57,-58,0,0,QPQP,intrepo,0,0,5ee7361e7781ec1c306bd0ef,5ee7361e7781ec1c306bd223,SPM,144895379,OB,136676111,145_153155545_Advent Geneva,389_153146279_Credit suisse,No-Pair,0.981467,0.005877,0.006882,0.005774,144895379No-Pair
12513,-88125.00,0.0,0.0,-9013072.5,105.75,8523000.0,0,0,-8924947.50,-57,-58,0,0,QPQP,intrepo,0,0,5ee7361e7781ec1c306bd0f7,5ee7361e7781ec1c306bd223,SPM,144895379,OB,136676122,145_153155545_Advent Geneva,386_153146279_Credit suisse,No-Pair,0.981194,0.005910,0.006898,0.005998,144895379No-Pair
12567,-132187.50,0.0,0.0,-9013072.5,105.75,8523000.0,0,0,-8880885.00,-57,-58,0,0,QPQP,intrepo,0,0,5ee7361e7781ec1c306bd0eb,5ee7361e7781ec1c306bd223,SPM,144895379,OB,136676145,145_153155545_Advent Geneva,385_153146279_Credit suisse,No-Pair,0.981194,0.005910,0.006898,0.005998,144895379No-Pair
12621,-80252.50,0.0,0.0,-9013072.5,105.75,8523000.0,0,0,-8932820.00,-57,-58,0,0,QPQP,intrepo,0,0,5ee7361e7781ec1c306bd0e7,5ee7361e7781ec1c306bd223,SPM,144895379,OB,136676160,145_153155545_Advent Geneva,387_153146279_Credit suisse,No-Pair,0.981194,0.005910,0.006898,0.005998,144895379No-Pair
12675,8386988.55,0.0,-65000000.0,-9013072.5,105.75,8523000.0,0,1,-17400061.05,-1,0,1,1,QPQP,sellrepo,0,0,5ee7361e7781ec1c306bd3a3,5ee7361e7781ec1c306bd223,SPM,144895379,OB,144395443,145_153155545_Advent Geneva,429_153155062_Credit suisse,No-Pair,0.985566,0.004657,0.005727,0.004050,144895379No-Pair
12729,-9805872.27,0.0,10233000.0,-9013072.5,105.75,8523000.0,1,1,792799.77,0,0,0,0,QPQP,covershortrepo,0,0,5ee7360d7781ec1c306bce91,5ee7361e7781ec1c306bd223,SPM,144895379,SDB,144660956,145_153155545_Advent Geneva,394_153155317_Credit suisse,No-Pair,0.965352,0.009151,0.013958,0.011539,144895379No-Pair
12783,-5803336.66,0.0,6047000.0,-9013072.5,105.75,8523000.0,1,1,-3209735.84,0,0,0,0,QPQP,covershortrepo,0,0,5ee7360d7781ec1c306bce96,5ee7361e7781ec1c306bd223,SPM,144895379,SDB,144660957,145_153155545_Advent Geneva,412_153155317_Credit suisse,No-Pair,0.970850,0.010877,0.010052,0.008221,144895379No-Pair


## Merging with User Action Data

In [179]:
prediction_table3 = prediction_table

In [180]:
aua = pd.read_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Soros/JuneData/AUA/AUACollections_SOROS.AUA_HST_RecData_153_2020-06-25.csv')

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,7,30,54,55,56,57,67,68,79,86,92,95,97,103,105,106,112,113,114,115,116,117,123,128,173,183,211,212,266,268,270,272,274,280) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [181]:
#aua = /Soros/JuneData/MEO/MeoCollections_SOROS.AUA_HST_RecData_153_2020-06-25.csv"

In [182]:
#test_file.to_csv("//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/OakTree/JuneData/X_Test/x_test_2020-06-29.csv")

In [183]:
aua.shape

(16916, 287)

In [184]:
aua = aua[~((aua['LastPerformedAction']==0) & (aua['ViewData.Status']=='SDB'))]
aua = aua.reset_index()
aua = aua.drop('index',1)

In [185]:
aua['ViewData.Status'].value_counts()

UDB    9348
OB     6402
SMB     411
UMB     296
UCB     237
UMR      84
SDB      44
UMT      21
UOB      10
CMF       9
SPM       8
UCC       4
CNF       2
Name: ViewData.Status, dtype: int64

In [186]:
aua = aua[aua['ViewData.Status'].isin(['UMR','UMB','UMT','OB','SDB','UCB'])]
aua = aua.reset_index()
aua = aua.drop('index',1)

In [187]:
aua['ViewData.Status'].value_counts()

OB     6402
UMB     296
UCB     237
UMR      84
SDB      44
UMT      21
Name: ViewData.Status, dtype: int64

In [188]:
prediction_table3

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID
0,82817946,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.052263,0.002674,0.006601,0.993558,5ef985a27781ec1ac88556ec,5ef985a27781ec1ac8855708
1,125420837,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000025,0.000020,0.002485,0.999850,5ef9c3167781ec114077695c,5ef9c3167781ec1140776760
2,125420838,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000047,0.000034,0.001343,0.999901,5ef9c3167781ec114077695c,5ef9c3167781ec114077675d
3,125420840,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000057,0.000053,0.001778,0.999840,5ef9c3167781ec114077695c,5ef9c3167781ec1140776767
4,125420841,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000222,0.000229,0.000495,0.999923,5ef9c3167781ec114077695c,5ef9c3167781ec1140776765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,146830534,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146382937],NaN,NaN,UMR_One_to_One,[146382937],1.0,0.997731,0.001426,0.000786,0.000057,5ef996b47781ec114074df27,5ef996b47781ec114074dff2
700,146865830,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146682038],NaN,NaN,UMR_One_to_One,[146682038],1.0,0.999923,0.000071,0.000003,0.000003,5ef9c91b7781ec11407780d5,5ef9c91b7781ec11407780ad
701,146865831,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146682043],NaN,NaN,UMR_One_to_One,[146682043],1.0,0.999919,0.000075,0.000003,0.000003,5ef9c91b7781ec11407780c9,5ef9c91b7781ec11407780ae
702,146865832,"[No-Pair, UMR_One_to_One, UMB_One_to_One]",3,1,1,0,1,[146682035],NaN,"[146682042, 146682049]",UMR_One_to_One,[146682035],1.0,0.999919,0.000075,0.000003,0.000003,5ef9c91b7781ec11407780bb,5ef9c91b7781ec11407780af


In [189]:
aua_id_match = aua[['MetaData.0._ParentID','ViewData.Status','ViewData.Age','ViewData.BreakID','ViewData._ID','ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds']]

aua_id_match.columns = ['SideB.ViewData._ID','Actual_Status','ViewData.Age','ViewData.BreakID','AUA_ViewData._ID','ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds']

aua_id_match = aua_id_match.drop_duplicates()
aua_id_match = aua_id_match.reset_index()
aua_id_match = aua_id_match.drop('index',1)

########################################################################################################
aua_open_status = aua[['ViewData.BreakID','ViewData.Status']]

aua_open_status.columns = ['SideB.ViewData.BreakID_B_side','Actual_Status_Open']
aua_open_status = aua_open_status.drop_duplicates()
aua_open_status = aua_open_status.reset_index()
aua_open_status = aua_open_status.drop('index',1)


In [190]:
aua_id_match

,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds
0,5ef9c91b7781ec11407780af,UMR,0,146866094,5ef9d162887a8a13500596e8,10_153157658_Advent Geneva,3_153157457_Lending Club
1,5ef9c91b7781ec11407780b0,UMR,0,146866093,5ef9d161887a8a13500596d0,11_153157658_Advent Geneva,6_153157457_Lending Club
2,5ef9c91b7781ec11407780ae,UMR,0,146866092,5ef9d161887a8a13500596b8,9_153157658_Advent Geneva,7_153157457_Lending Club
3,5ef9c91b7781ec11407780ad,UMR,0,146866091,5ef9d160887a8a13500596a0,8_153157658_Advent Geneva,10_153157457_Lending Club
4,5ef9c91b7781ec11407780da,UCB,0,146865842,5ef9d059887a8a13500584cb,2_153157658_Advent Geneva,NaN
...,...,...,...,...,...,...,...
1846,5ef5935f887a8922844b9687,OB,1,146622887,5ef9856b7781ec1ac884efe7,927_153157324_Advent Geneva,NaN
1847,5ef5934e887a8922844b8f8a,OB,1,146622714,5ef9856b7781ec1ac884efd5,1509_153157324_Advent Geneva,NaN
1848,5ef5934e887a8922844b8f89,OB,1,146622713,5ef9856b7781ec1ac884efd2,1508_153157324_Advent Geneva,NaN
1849,5ef5934f887a8922844b8fd5,OB,4,146622720,5ef9856b7781ec1ac884efc8,29_153156735_Advent Geneva,NaN


In [191]:
aua_open_status['SideB.ViewData.BreakID_B_side'] = aua_open_status['SideB.ViewData.BreakID_B_side'].astype(int).astype(str)
prediction_table3['SideB.ViewData.BreakID_B_side'] = prediction_table3['SideB.ViewData.BreakID_B_side'].astype(int).astype(str)

In [192]:
aua_id_match.shape

(1851, 7)

In [193]:
prediction_table3['SideB.ViewData._ID'] = prediction_table3['SideB.ViewData._ID'].fillna('Not_generated')
prediction_table3['SideA.ViewData._ID'] = prediction_table3['SideA.ViewData._ID'].fillna('Not_generated')

In [194]:
prediction_table3[prediction_table3['SideB.ViewData.BreakID_B_side'] == '1334352050']

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID


In [195]:
#aua_id_match['len_side0'] = aua_id_match.apply(lambda x: len(x['Actual_Status'].split(',')), axis=1)
#aua_id_match['len_side1'] = aua_id_match.apply(lambda x: len(x['Actual_Status'].split(',')), axis=1)

In [196]:
#aua_one_side = aua_id_match.groupby(['ViewData.Side1_UniqueIds'])['Actual_Status'].unique().reset_index()
#aua_zero_side = aua_id_match.groupby(['ViewData.Side0_UniqueIds'])['Actual_Status'].unique().reset_index()

In [197]:
aua_id_match['combined_flag'] = aua_id_match.apply(lambda x: 1 if 'Combined' in x['AUA_ViewData._ID'] else 0,axis=1)

In [198]:
#aua_id_match[''.sort_values(['ViewData.Side0_UniqueIds'])

In [199]:
aua_id_match1  = aua_id_match[aua_id_match['combined_flag']!=1]
aua_id_match1 = aua_id_match1.reset_index()
aua_id_match1 = aua_id_match1.drop('index',1)

In [200]:
side1_repeat = aua_id_match1['ViewData.Side1_UniqueIds'].value_counts().reset_index()
side0_repeat = aua_id_match1['ViewData.Side0_UniqueIds'].value_counts().reset_index()


In [201]:
side1_repeat

,index,ViewData.Side1_UniqueIds
0,146_153157410_Credit suisse,3
1,143_153157410_Credit suisse,3
2,142_153157410_Credit suisse,3
3,379_153157410_Credit suisse,3
4,139_153157410_Credit suisse,3
...,...,...
1129,1_153156113_Goldman Sachs,1
1130,3_153113371_Wells Fargo,1
1131,"164_153157570_Morgan Stanley,245_153156963_Mor...",1
1132,24_153156113_Goldman Sachs,1


In [202]:
side1_repeat[side1_repeat['ViewData.Side1_UniqueIds']>1]

,index,ViewData.Side1_UniqueIds
0,146_153157410_Credit suisse,3
1,143_153157410_Credit suisse,3
2,142_153157410_Credit suisse,3
3,379_153157410_Credit suisse,3
4,139_153157410_Credit suisse,3
...,...,...
69,5_153157547_CITI,2
70,12_153157124_Goldman Sachs,2
71,1_153157124_Goldman Sachs,2
72,34_153157547_CITI,2


In [203]:
aua_id_match1['1_repeat_flag'] = aua_id_match1.apply(lambda x: 1 if x['ViewData.Side1_UniqueIds'] in side1_repeat[side1_repeat['ViewData.Side1_UniqueIds']>1]['index'].values else 0, axis=1)
aua_id_match1['0_repeat_flag'] = aua_id_match1.apply(lambda x: 1 if x['ViewData.Side0_UniqueIds'] in side0_repeat[side0_repeat['ViewData.Side0_UniqueIds']>1]['index'].values else 0, axis=1)

In [204]:
aua_id_match2 = aua_id_match1[~((aua_id_match1['1_repeat_flag']==1) & (aua_id_match1['Actual_Status']=='OB'))]
aua_id_match2 = aua_id_match2.reset_index()
aua_id_match2 = aua_id_match2.drop('index',1)

In [205]:
aua_id_match3 = aua_id_match2[~((aua_id_match2['0_repeat_flag']==1) & (aua_id_match2['Actual_Status']=='OB'))]
aua_id_match3 = aua_id_match3.reset_index()
aua_id_match3 = aua_id_match3.drop('index',1)

In [206]:

#aua_zero_side['len_side0'].value_counts()
#aua_open_status['SideB.ViewData.BreakID_B_side'].nunique()

In [207]:
#aua_sub99[aua_sub99['ViewData.Side0_UniqueIds'] == '789_125897734_Advent Geneva']

In [208]:
aua_id_match3[aua_id_match3['ViewData.Side0_UniqueIds'] == '789_125897734_Advent Geneva']

,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag


In [209]:
aua_id_match3[aua_id_match3['SideB.ViewData._ID'] == '5ee721a315545830acc67099']

,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag


In [210]:
aua_id_match3

,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag
0,5ef9c91b7781ec11407780af,UMR,0,146866094,5ef9d162887a8a13500596e8,10_153157658_Advent Geneva,3_153157457_Lending Club,0,0,0
1,5ef9c91b7781ec11407780b0,UMR,0,146866093,5ef9d161887a8a13500596d0,11_153157658_Advent Geneva,6_153157457_Lending Club,0,0,0
2,5ef9c91b7781ec11407780ae,UMR,0,146866092,5ef9d161887a8a13500596b8,9_153157658_Advent Geneva,7_153157457_Lending Club,0,0,0
3,5ef9c91b7781ec11407780ad,UMR,0,146866091,5ef9d160887a8a13500596a0,8_153157658_Advent Geneva,10_153157457_Lending Club,0,0,0
4,5ef9c91b7781ec11407780da,UCB,0,146865842,5ef9d059887a8a13500584cb,2_153157658_Advent Geneva,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1658,5ef5935f887a8922844b9687,OB,1,146622887,5ef9856b7781ec1ac884efe7,927_153157324_Advent Geneva,NaN,0,0,0
1659,5ef5934e887a8922844b8f8a,OB,1,146622714,5ef9856b7781ec1ac884efd5,1509_153157324_Advent Geneva,NaN,0,0,0
1660,5ef5934e887a8922844b8f89,OB,1,146622713,5ef9856b7781ec1ac884efd2,1508_153157324_Advent Geneva,NaN,0,0,0
1661,5ef5934f887a8922844b8fd5,OB,4,146622720,5ef9856b7781ec1ac884efc8,29_153156735_Advent Geneva,NaN,0,0,0


In [211]:
pb_side = X_test.groupby('SideA.ViewData.BreakID_A_side')['Predicted_action'].unique().reset_index()


In [212]:
pb_side['len'] = pb_side['Predicted_action'].apply(lambda x: len(x))

In [213]:
pb_side['No_Pair_flag'] = pb_side.apply(lambda x: 1 if 'No-Pair' in x['Predicted_action'] else 0, axis=1)

In [214]:
pb_side_open_ids = pb_side[(pb_side['len']==1) & (pb_side['No_Pair_flag']==1)]['SideA.ViewData.BreakID_A_side']

In [215]:
prediction_table3[prediction_table3['SideB.ViewData.BreakID_B_side']=='1335439926']

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID


In [216]:
aua_id_match3[aua_id_match3['ViewData.Side0_UniqueIds'].isnull()]

,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag
12,NaN,OB,3,146865846,5ef9c91b7781ec11407780de,NaN,9_153157658_Lending Club,0,0,0
13,NaN,OB,3,146865845,5ef9c91b7781ec11407780dd,NaN,3_153157658_Lending Club,0,0,0
14,NaN,OB,3,146865848,5ef9c91b7781ec11407780e0,NaN,6_153157658_Lending Club,0,0,0
15,NaN,OB,3,146865844,5ef9c91b7781ec11407780dc,NaN,8_153157658_Lending Club,0,0,0
17,NaN,OB,5,146495242,5ef9c91b7781ec11407780d5,NaN,9_153157256_Lending Club,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1597,NaN,UCB,0,146797560,5ef9bc33887a8a13500530ac,NaN,26_153157547_CITI,0,1,0
1599,NaN,UCB,0,146797571,5ef9bc33887a8a13500530aa,NaN,11_153157547_CITI,0,1,0
1601,NaN,UCB,0,146797545,5ef9bc33887a8a13500530a8,NaN,2_153157547_CITI,0,1,0
1638,NaN,OB,4,146592926,5ef985a17781ec1ac8855554,NaN,1380_153157324_CITI,0,0,0


In [217]:
prediction_table_new = pd.merge(prediction_table3, aua_id_match3, on='SideB.ViewData._ID', how='left')

In [218]:
prediction_table_new[prediction_table_new['SideB.ViewData.BreakID_B_side'] == '1334352050']

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag


In [219]:
prediction_table_new[prediction_table_new['Actual_Status']=='OB']

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag
422,146796856,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000545,0.000443,0.003455,0.999901,5ef985a17781ec1ac88551e5,5ef985a17781ec1ac88553f4,OB,4.0,146796856.0,5ef9878b887a8a1350048ff5,94_153157324_Advent Geneva,NaN,0.0,0.0,0.0
424,146796858,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000545,0.000443,0.003455,0.999901,5ef985a17781ec1ac88551e5,5ef985a17781ec1ac88553f5,OB,4.0,146796858.0,5ef9878b887a8a1350048ff3,949_153157324_Advent Geneva,NaN,0.0,0.0,0.0
507,146797484,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000548,0.000444,0.003908,0.999897,5ef985a17781ec1ac88551e5,5ef985a17781ec1ac885552d,OB,4.0,146797484.0,5ef9878b887a8a1350048ff4,105_153157324_Advent Geneva,NaN,0.0,0.0,0.0
508,146797485,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000548,0.000444,0.003908,0.999897,5ef985a17781ec1ac88551e5,5ef985a17781ec1ac885552c,OB,4.0,146797485.0,5ef9878b887a8a1350048ff2,106_153157324_Advent Geneva,NaN,0.0,0.0,0.0
560,146799341,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.003864,0.017116,0.222159,0.977952,5ef985a27781ec1ac88556ec,5ef985a27781ec1ac885568c,OB,4.0,146799341.0,5ef98e85887a8a135004b2f9,192_153157410_Advent Geneva,NaN,0.0,0.0,0.0
562,146799358,"[No-Pair, UMB_One_to_One]",2,1,1,0,0,NaN,NaN,"[146799364, 146799397]",UMB_One_to_One,"[146799364, 146799397]",2.0,0.005587,0.007740,0.584125,0.402548,5ef985a27781ec1ac8855715,5ef985a27781ec1ac885568e,OB,4.0,146799358.0,5ef98e85887a8a135004b304,127_153157410_Advent Geneva,NaN,0.0,0.0,0.0


In [220]:
aua_id_match4 = aua_id_match3.rename(columns = {'ViewData.BreakID': 'SideB.ViewData.BreakID_B_side'})
aua_id_match4 = aua_id_match4.rename(columns = {'Actual_Status': 'Actual_Status_Open'})

In [221]:
aua_id_match4['SideB.ViewData.BreakID_B_side'] = aua_id_match4['SideB.ViewData.BreakID_B_side'].astype(str)

In [222]:
#prediction_table_new = pd.merge(prediction_table_new ,aua_open_status, on='SideB.ViewData.BreakID_B_side', how='left')
prediction_table_new = pd.merge(prediction_table_new ,aua_id_match4[['SideB.ViewData.BreakID_B_side','Actual_Status_Open']], on='SideB.ViewData.BreakID_B_side', how='left')

In [223]:
#prediction_table_new

In [224]:
prediction_table_new[prediction_table_new['Actual_Status_Open']=='OB']['Final_prediction'].value_counts()

No-Pair           276
UMB_One_to_One     23
UMT_One_to_One      1
Name: Final_prediction, dtype: int64

In [225]:
prediction_table_new.loc[prediction_table_new['Final_prediction']=='No-Pair/Unrecognized','Final_prediction'] = 'No-Pair'

In [226]:
prediction_table_new.loc[prediction_table_new['Actual_Status'].isnull()]

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag,Actual_Status_Open
0,82817946,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.052263,0.002674,0.006601,0.993558,5ef985a27781ec1ac88556ec,5ef985a27781ec1ac8855708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,125420837,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000025,0.000020,0.002485,0.999850,5ef9c3167781ec114077695c,5ef9c3167781ec1140776760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
2,125420838,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000047,0.000034,0.001343,0.999901,5ef9c3167781ec114077695c,5ef9c3167781ec114077675d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
3,125420840,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000057,0.000053,0.001778,0.999840,5ef9c3167781ec114077695c,5ef9c3167781ec1140776767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
4,125420841,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000222,0.000229,0.000495,0.999923,5ef9c3167781ec114077695c,5ef9c3167781ec1140776765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,146830445,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000332,0.000539,0.046918,0.952211,5ef996b47781ec114074debd,5ef996b47781ec114074deca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680,146830490,"[No-Pair, UMB_One_to_One]",2,1,1,0,0,NaN,NaN,[146830490],UMB_One_to_One,[146830490],1.0,0.004140,0.022130,0.770423,0.203308,5ef996b47781ec114074dfec,5ef996b47781ec114074dfe7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
684,146830496,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000043,0.000048,0.000974,0.999547,5ef996b47781ec114074df87,5ef996b47781ec114074df8e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
685,146830498,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.003868,0.012777,0.399792,0.916288,5ef996b47781ec114074df87,5ef996b47781ec114074df99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB


In [227]:
prediction_table_new.loc[~prediction_table_new['Actual_Status_Open'].isnull(),'Actual_Status'] = prediction_table_new.loc[~prediction_table_new['Actual_Status_Open'].isnull(),'Actual_Status_Open']

In [228]:
prediction_table_new.loc[~prediction_table_new['Actual_Status_Open'].isnull(),:]

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag,Actual_Status_Open
1,125420837,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000025,0.000020,0.002485,0.999850,5ef9c3167781ec114077695c,5ef9c3167781ec1140776760,OB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
2,125420838,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000047,0.000034,0.001343,0.999901,5ef9c3167781ec114077695c,5ef9c3167781ec114077675d,OB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
3,125420840,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000057,0.000053,0.001778,0.999840,5ef9c3167781ec114077695c,5ef9c3167781ec1140776767,OB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
4,125420841,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000222,0.000229,0.000495,0.999923,5ef9c3167781ec114077695c,5ef9c3167781ec1140776765,OB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
5,125420843,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.001363,0.028513,0.000373,0.999856,5ef9c3167781ec114077695c,5ef9c3167781ec114077675c,OB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,146830516,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000831,0.000326,0.001243,0.999526,5ef996b47781ec114074df00,5ef996b47781ec114074e013,UCB,0.0,146830516.0,5ef9c40f887a8a1350053c12,15_153157591_Advent Geneva,NaN,0.0,0.0,1.0,UCB
693,146830517,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.003267,0.000178,0.001464,0.998085,5ef996b47781ec114074df7a,5ef996b47781ec114074e01c,UCB,0.0,146830517.0,5ef9c40f887a8a1350053c11,4_153157591_Advent Geneva,NaN,0.0,0.0,1.0,UCB
694,146830518,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000498,0.000294,0.001499,0.999029,5ef996b47781ec114074df7a,5ef996b47781ec114074dfea,UCB,0.0,146830518.0,5ef9c40f887a8a1350053bfe,5_153157591_Advent Geneva,NaN,0.0,0.0,1.0,UCB
696,146830526,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,No-Pair,NaN,NaN,0.000890,0.000464,0.002082,0.999350,5ef996b47781ec114074df00,5ef996b47781ec114074e00a,UCB,0.0,146830526.0,5ef9c40f887a8a1350053c13,13_153157591_Advent Geneva,NaN,0.0,0.0,1.0,UCB


In [229]:
prediction_table_new[prediction_table_new['SideB.ViewData.BreakID_B_side'] =='1334352248']

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag,Actual_Status_Open


In [230]:
prediction_table_new.loc[prediction_table_new['Actual_Status']=='OB','Actual_Status'] = 'Open Break'

In [231]:
prediction_table_new.loc[prediction_table_new['Final_prediction']=='No-Pair','Final_prediction'] = 'Open Break'
prediction_table_new.loc[prediction_table_new['Final_prediction']=='UMR_One_to_One','Final_prediction'] = 'UMR'
prediction_table_new.loc[prediction_table_new['Final_prediction']=='UMT_One_to_One','Final_prediction'] = 'UMT'
prediction_table_new.loc[prediction_table_new['Final_prediction']=='UMB_One_to_One','Final_prediction'] = 'UMB'

In [232]:
prediction_table_new['Actual_Status'].isnull().sum()

264

In [233]:
prediction_table_new = prediction_table_new[~prediction_table_new['Actual_Status'].isnull()]
prediction_table_new = prediction_table_new.reset_index()
prediction_table_new = prediction_table_new.drop('index',1)

In [234]:
prediction_table_new

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag,Actual_Status_Open
0,125420837,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,Open Break,NaN,NaN,0.000025,0.000020,0.002485,0.999850,5ef9c3167781ec114077695c,5ef9c3167781ec1140776760,Open Break,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
1,125420838,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,Open Break,NaN,NaN,0.000047,0.000034,0.001343,0.999901,5ef9c3167781ec114077695c,5ef9c3167781ec114077675d,Open Break,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
2,125420840,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,Open Break,NaN,NaN,0.000057,0.000053,0.001778,0.999840,5ef9c3167781ec114077695c,5ef9c3167781ec1140776767,Open Break,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
3,125420841,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,Open Break,NaN,NaN,0.000222,0.000229,0.000495,0.999923,5ef9c3167781ec114077695c,5ef9c3167781ec1140776765,Open Break,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
4,125420843,[No-Pair],1,1,0,0,0,NaN,NaN,NaN,Open Break,NaN,NaN,0.001363,0.028513,0.000373,0.999856,5ef9c3167781ec114077695c,5ef9c3167781ec114077675c,Open Break,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,146830534,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146382937],NaN,NaN,UMR,[146382937],1.0,0.997731,0.001426,0.000786,0.000057,5ef996b47781ec114074df27,5ef996b47781ec114074dff2,UMR,0.0,146865175.0,5ef9c40d887a8a1350053b57,111_153157591_Advent Geneva,74_153157124_Goldman Sachs,0.0,1.0,1.0,NaN
437,146865830,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146682038],NaN,NaN,UMR,[146682038],1.0,0.999923,0.000071,0.000003,0.000003,5ef9c91b7781ec11407780d5,5ef9c91b7781ec11407780ad,UMR,0.0,146866091.0,5ef9d160887a8a13500596a0,8_153157658_Advent Geneva,10_153157457_Lending Club,0.0,0.0,0.0,NaN
438,146865831,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146682043],NaN,NaN,UMR,[146682043],1.0,0.999919,0.000075,0.000003,0.000003,5ef9c91b7781ec11407780c9,5ef9c91b7781ec11407780ae,UMR,0.0,146866092.0,5ef9d161887a8a13500596b8,9_153157658_Advent Geneva,7_153157457_Lending Club,0.0,0.0,0.0,NaN
439,146865832,"[No-Pair, UMR_One_to_One, UMB_One_to_One]",3,1,1,0,1,[146682035],NaN,"[146682042, 146682049]",UMR,[146682035],1.0,0.999919,0.000075,0.000003,0.000003,5ef9c91b7781ec11407780bb,5ef9c91b7781ec11407780af,UMR,0.0,146866094.0,5ef9d162887a8a13500596e8,10_153157658_Advent Geneva,3_153157457_Lending Club,0.0,0.0,0.0,NaN


## Final Actual vs Predicted Table - Process Initiation

In [236]:
meo = pd.read_csv("//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Soros/JuneData/MEO/MeoCollections_SOROS.MEO_HST_RecData_153_2020-06-25.csv",usecols=new_cols)

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (184,274,276) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [237]:
meo = meo[['ViewData.BreakID','ViewData.Side1_UniqueIds','ViewData.Side0_UniqueIds','ViewData.Age','ViewData.Status']].drop_duplicates()

In [238]:
meo['key'] = meo['ViewData.Side0_UniqueIds'].astype(str) + meo['ViewData.Side1_UniqueIds'].astype(str)

In [239]:
aua_id_match5 = aua_id_match3.rename(columns ={'Actual_Status': 'ViewData.Status'})

In [240]:
aua_sub = aua_id_match5[['ViewData.Side1_UniqueIds','ViewData.Side0_UniqueIds','ViewData.Age','ViewData.Status']].drop_duplicates()

In [241]:
aua_sub['key'] = aua_sub['ViewData.Side0_UniqueIds'].astype(str) + aua_sub['ViewData.Side1_UniqueIds'].astype(str)

In [242]:
prediction_table_new['ViewData.BreakID'] = prediction_table_new['SideB.ViewData.BreakID_B_side']
prediction_table_new['ViewData.BreakID'] = prediction_table_new['ViewData.BreakID'].astype(str)

In [243]:
meo['ViewData.BreakID'] = meo['ViewData.BreakID'].astype(str)

In [244]:
prediction_table_new1 = pd.merge(prediction_table_new, meo[['ViewData.BreakID','key']], on='ViewData.BreakID', how='left')

In [245]:
prediction_table_new1[prediction_table_new1['Final_prediction']=='UMR']

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag,Actual_Status_Open,key
63,146212904,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146823126],NaN,NaN,UMR,[146823126],1.0,0.829535,0.010222,0.158804,1.439684e-03,5ef987327781ec114072c2c5,5ef987327781ec114072c2c9,UMT,0.0,146212904,5ef9b5ce887a8a1350052e2b,78_153156543_Advent Geneva,"4_153156963_Morgan Stanley,78_153157570_Morgan...",0.0,0.0,0.0,NaN,78_153156543_Advent Genevanan
64,146212918,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146823101],NaN,NaN,UMR,[146823101],1.0,0.921821,0.007568,0.069855,7.555075e-04,5ef987327781ec114072c2c1,5ef987327781ec114072c2ce,UMT,0.0,146212918,5ef9b5ce887a8a1350052ded,182_153156543_Advent Geneva,"75_153157570_Morgan Stanley,115_153156963_Morg...",0.0,0.0,0.0,NaN,182_153156543_Advent Genevanan
65,146212937,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146823047],NaN,NaN,UMR,[146823047],1.0,0.969347,0.005667,0.024608,3.788009e-04,5ef987327781ec114072c2be,5ef987327781ec114072c2cc,UMT,0.0,146212937,5ef9b5cd887a8a1350052db2,170_153156543_Advent Geneva,"164_153157570_Morgan Stanley,245_153156963_Mor...",0.0,0.0,0.0,NaN,170_153156543_Advent Genevanan
71,146435255,"[UMR_One_to_One, No-Pair]",2,1,0,0,1,[146823035],NaN,NaN,UMR,[146823035],1.0,0.999946,0.000046,0.000001,6.609025e-06,5ef987327781ec114072c1cc,5ef987327781ec114072c182,UMR,0.0,146435255,5ef9b5cc887a8a1350052caf,3_153157169_Advent Geneva,2_153157570_Morgan Stanley,0.0,0.0,0.0,NaN,3_153157169_Advent Genevanan
72,146435273,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146823036],NaN,NaN,UMR,[146823036],1.0,0.999943,0.000048,0.000001,8.035061e-06,5ef987327781ec114072c1cc,5ef987327781ec114072c16d,UMR,0.0,146435273,5ef9b5cb887a8a1350052c7f,2_153157169_Advent Geneva,1_153157570_Morgan Stanley,0.0,0.0,0.0,NaN,2_153157169_Advent Genevanan
73,146435414,[UMR_One_to_One],1,0,0,0,1,[146823045],NaN,NaN,UMR,[146823045],1.0,0.999970,0.000025,0.000001,2.819199e-06,5ef987327781ec114072c1d5,5ef987327781ec114072c17e,UMR,0.0,146435414,5ef9b5cb887a8a1350052c97,107_153157169_Advent Geneva,131_153157570_Morgan Stanley,0.0,0.0,0.0,NaN,107_153157169_Advent Genevanan
264,146797529,"[No-Pair, UMR_One_to_One]",2,1,0,0,1,[146592927],NaN,NaN,UMR,[146592927],1.0,0.999882,0.000098,0.000016,3.118584e-06,5ef985a17781ec1ac88551e5,5ef985a17781ec1ac8855541,UMR,0.0,146797529,5ef9876b887a8a1350048f3c,571_153157547_Advent Geneva,15_153157324_CITI,0.0,0.0,0.0,NaN,571_153157547_Advent Genevanan
265,146797530,[UMR_One_to_One],1,0,0,0,1,[146592946],NaN,NaN,UMR,[146592946],1.0,0.999883,0.000098,0.000016,3.019690e-06,5ef985a17781ec1ac885553f,5ef985a17781ec1ac8855543,UMR,0.0,146797530,5ef9876c887a8a1350048f84,3_153157547_Advent Geneva,12_153157324_CITI,0.0,0.0,0.0,NaN,3_153157547_Advent Genevanan
266,146797531,[UMR_One_to_One],1,0,0,0,1,[146592977],NaN,NaN,UMR,[146592977],1.0,0.999885,0.000096,0.000017,2.952175e-06,5ef985a17781ec1ac885553d,5ef985a17781ec1ac8855542,UMR,0.0,146797531,5ef9876c887a8a1350048f54,576_153157547_Advent Geneva,19_153157324_CITI,0.0,0.0,0.0,NaN,576_153157547_Advent Genevanan
267,146797532,"[No-Pair, UMB_One_to_One, UMR_One_to_One]",3,1,1,0,1,[146592965],NaN,[146592927],UMR,[146592965],1.0,0.999882,0.000098,0.000016,3.118584e-06,5ef985a17781ec1ac88551e5,5ef985a17781ec1ac8855544,UMR,0.0,146797532,5ef9876c887a8a1350048f9c,570_153157547_Advent Geneva,13_153157324_CITI,0.0,0.0,0.0,NaN,570_153157547_Advent Genevanan


In [246]:
prediction_table_new1[prediction_table_new1['ViewData.Side0_UniqueIds']=='789_125897734_Advent Geneva']

,SideB.ViewData.BreakID_B_side,Predicted_action,len,No_Pair_flag,UMB_flag,UMT_flag,UMR_flag,Predicted_UMR_array,Predicted_UMT_array,Predicted_UMB_array,Final_prediction,Final_predicted_break,predicted_break_len,probability_UMR,probability_UMT,probability_UMB,probability_No_pair,SideA.ViewData._ID,SideB.ViewData._ID,Actual_Status,ViewData.Age,ViewData.BreakID,AUA_ViewData._ID,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,combined_flag,1_repeat_flag,0_repeat_flag,Actual_Status_Open,key


In [247]:
aua_sub1 = pd.merge(aua_sub, prediction_table_new1[['key','Final_prediction','probability_UMR','probability_No_pair','probability_UMT','probability_UMB','Final_predicted_break']], on='key', how='left')

In [248]:
aua_sub1[aua_sub1['ViewData.Side0_UniqueIds']=='789_125897734_Advent Geneva']

,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,ViewData.Status,key,Final_prediction,probability_UMR,probability_No_pair,probability_UMT,probability_UMB,Final_predicted_break


In [249]:
aua_sub1

,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,ViewData.Status,key,Final_prediction,probability_UMR,probability_No_pair,probability_UMT,probability_UMB,Final_predicted_break
0,3_153157457_Lending Club,10_153157658_Advent Geneva,0,UMR,10_153157658_Advent Geneva3_153157457_Lending ...,NaN,NaN,NaN,NaN,NaN,NaN
1,6_153157457_Lending Club,11_153157658_Advent Geneva,0,UMR,11_153157658_Advent Geneva6_153157457_Lending ...,NaN,NaN,NaN,NaN,NaN,NaN
2,7_153157457_Lending Club,9_153157658_Advent Geneva,0,UMR,9_153157658_Advent Geneva7_153157457_Lending Club,NaN,NaN,NaN,NaN,NaN,NaN
3,10_153157457_Lending Club,8_153157658_Advent Geneva,0,UMR,8_153157658_Advent Geneva10_153157457_Lending ...,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2_153157658_Advent Geneva,0,UCB,2_153157658_Advent Genevanan,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1660,NaN,927_153157324_Advent Geneva,1,OB,927_153157324_Advent Genevanan,NaN,NaN,NaN,NaN,NaN,NaN
1661,NaN,1509_153157324_Advent Geneva,1,OB,1509_153157324_Advent Genevanan,NaN,NaN,NaN,NaN,NaN,NaN
1662,NaN,1508_153157324_Advent Geneva,1,OB,1508_153157324_Advent Genevanan,NaN,NaN,NaN,NaN,NaN,NaN
1663,NaN,29_153156735_Advent Geneva,4,OB,29_153156735_Advent Genevanan,NaN,NaN,NaN,NaN,NaN,NaN


In [250]:
no_open = prediction_table_new1[prediction_table_new1['Final_prediction']!='Open Break'].reset_index()
no_open = no_open.drop('index',1)

no_open['key'] = no_open['ViewData.Side0_UniqueIds'].astype(str) + no_open['ViewData.Side1_UniqueIds'].astype(str)

In [251]:
#aua_sub1[aua_sub1['Final_prediction']=='UMR_One_to_One']
X_test['key'] = X_test['SideB.ViewData.Side0_UniqueIds'].astype(str) + X_test['SideA.ViewData.Side1_UniqueIds'].astype(str)

In [252]:
aua_sub[aua_sub['ViewData.Side0_UniqueIds'] == '789_125897734_Advent Geneva']

,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,ViewData.Status,key


In [253]:
aua_sub = pd.merge(aua_sub1, no_open[['key','Final_prediction']], on='key', how='left')

In [254]:
aua_sub11 = aua_sub1[aua_sub1['Final_prediction']=='Open Break']
aua_sub11 = aua_sub11.reset_index()
aua_sub11 = aua_sub11.drop('index',1)

In [255]:
aua_sub11['probability_UMR'].fillna(0.00355,inplace=True)
aua_sub11['probability_UMB'].fillna(0.003124,inplace=True)
aua_sub11['probability_UMT'].fillna(0.00255,inplace=True)
aua_sub11['probability_No_pair'].fillna(0.99034,inplace=True)

In [256]:
aua_sub22 = aua_sub1[aua_sub1['Final_prediction']!='Open Break'][['ViewData.Side1_UniqueIds', 'ViewData.Side0_UniqueIds', 'ViewData.Age','ViewData.Status', 'key']]

aua_sub22 = aua_sub22.reset_index()
aua_sub22 = aua_sub22.drop('index',1)
aua_sub22 = pd.merge(aua_sub22, no_open[['key','Final_prediction','probability_UMR','probability_No_pair','probability_UMT','probability_UMB','Final_predicted_break']], on='key', how='left')
aua_sub22 = aua_sub22.reset_index()
aua_sub22 = aua_sub22.drop('index',1)


In [257]:
aua_sub33 = pd.concat([aua_sub11,aua_sub22], axis=0)
aua_sub33 = aua_sub33.reset_index()
aua_sub33 = aua_sub33.drop('index',1)

In [258]:
aua_sub33['ViewData.Side0_UniqueIds'] = aua_sub33['ViewData.Side0_UniqueIds'].astype(str)
aua_sub33['ViewData.Side1_UniqueIds'] = aua_sub33['ViewData.Side1_UniqueIds'].astype(str)

In [259]:
aua_sub33['len_side0'] = aua_sub33.apply(lambda x: len(x['ViewData.Side0_UniqueIds'].split(',')), axis=1)
aua_sub33['len_side1'] = aua_sub33.apply(lambda x: len(x['ViewData.Side1_UniqueIds'].split(',')), axis=1)

In [260]:
aua_sub33.loc[(aua_sub33['len_side0']>1) & (aua_sub33['len_side1']==1) & (aua_sub33['ViewData.Status']=='OB') ,'Type'] = 'One_side_aggregation'
aua_sub33.loc[(aua_sub33['len_side0']>1) & (aua_sub33['len_side1']==1) & (aua_sub33['ViewData.Status']!='OB') ,'Type'] = 'One_to_Many'

aua_sub33.loc[(aua_sub33['len_side0']==1) & (aua_sub33['len_side1']>1) & (aua_sub33['ViewData.Status']=='OB') ,'Type'] = 'One_side_aggregation'
aua_sub33.loc[(aua_sub33['len_side0']==1) & (aua_sub33['len_side1']>1) & (aua_sub33['ViewData.Status']!='OB') ,'Type'] = 'One_to_Many'
aua_sub33.loc[(aua_sub33['len_side0']>1) & (aua_sub33['len_side1']>1) & (aua_sub33['ViewData.Status']!='OB') ,'Type'] = 'Many_to_Many'

aua_sub33.loc[(aua_sub33['len_side0']==1) & (aua_sub33['len_side1']==1) ,'Type'] = 'One_to_One/Open'


In [261]:
aua_sub44 = aua_sub33[(aua_sub33['ViewData.Status']=='UMB') & (aua_sub33['ViewData.Age']>1)]
aua_sub44 = aua_sub44.reset_index()
aua_sub44 = aua_sub44.drop('index',1)

In [262]:
aua_sub44['Final_prediction'].fillna('UMB-Carry-Forward',inplace= True)
aua_sub44['probability_UMR'].fillna(0.0001,inplace= True)
aua_sub44['probability_UMB'].fillna(0.9998,inplace= True)
aua_sub44['probability_UMT'].fillna(0.0000,inplace= True)
aua_sub44['probability_No_pair'].fillna(0.0000,inplace= True)

In [263]:
aua_sub55 = aua_sub33[~((aua_sub33['ViewData.Status']=='UMB') & (aua_sub33['ViewData.Age']>1))]
aua_sub55 = aua_sub55.reset_index()
aua_sub55 = aua_sub55.drop('index',1)

In [264]:
aua_sub66 = pd.concat([aua_sub55,aua_sub44], axis=0)
aua_sub66 = aua_sub66.reset_index()
aua_sub66 = aua_sub66.drop('index',1)

In [265]:
aua_sub66.loc[(aua_sub66['ViewData.Status']=='UMB') & (aua_sub66['ViewData.Age']>1),'ViewData.Status'] = 'UMB-Carry-Forward'
aua_sub66.loc[(aua_sub66['ViewData.Status']=='OB'),'ViewData.Status'] = 'Open Break'

In [266]:
aua_sub66

,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,ViewData.Status,key,Final_prediction,probability_UMR,probability_No_pair,probability_UMT,probability_UMB,Final_predicted_break,len_side0,len_side1,Type
0,nan,6_153133289_Advent Geneva,154,Open Break,6_153133289_Advent Genevanan,Open Break,0.000057,0.999840,0.000053,0.001778,NaN,1,1,One_to_One/Open
1,nan,1_153133289_Advent Geneva,154,Open Break,1_153133289_Advent Genevanan,Open Break,0.000222,0.999923,0.000229,0.000495,NaN,1,1,One_to_One/Open
2,nan,7_153154447_Advent Geneva,24,Open Break,7_153154447_Advent Genevanan,Open Break,0.000043,0.999912,0.000032,0.001460,NaN,1,1,One_to_One/Open
3,nan,4_153133289_Advent Geneva,154,Open Break,4_153133289_Advent Genevanan,Open Break,0.000057,0.999710,0.000052,0.005792,NaN,1,1,One_to_One/Open
4,nan,6_153154447_Advent Geneva,24,Open Break,6_153154447_Advent Genevanan,Open Break,0.000294,0.999856,0.000169,0.000299,NaN,1,1,One_to_One/Open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,69_153154765_CITI,23_153154765_Advent Geneva,20,UMB-Carry-Forward,23_153154765_Advent Geneva69_153154765_CITI,UMB-Carry-Forward,0.000100,0.000000,0.000000,0.999800,NaN,1,1,One_to_One/Open
1661,1_153155908_CITI,3_153155908_Advent Geneva,13,UMB-Carry-Forward,3_153155908_Advent Geneva1_153155908_CITI,UMB-Carry-Forward,0.000100,0.000000,0.000000,0.999800,NaN,1,1,One_to_One/Open
1662,61_153153898_CITI,95_153153898_Advent Geneva,26,UMB-Carry-Forward,95_153153898_Advent Geneva61_153153898_CITI,UMB-Carry-Forward,0.000100,0.000000,0.000000,0.999800,NaN,1,1,One_to_One/Open
1663,77_153154765_CITI,21_153154765_Advent Geneva,20,UMB-Carry-Forward,21_153154765_Advent Geneva77_153154765_CITI,UMB-Carry-Forward,0.000100,0.000000,0.000000,0.999800,NaN,1,1,One_to_One/Open


## Read No-Pair Id File

In [5926]:
no_pair_id_data = pd.read_csv("//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/X_Test/no_pair_ids_125_2020-06-8.csv")

In [268]:
no_pair_ids

array(['101_153152538_Goldman Sachs', '102_153157570_Morgan Stanley',
       '103_153131370_Advent Geneva', '104_153131370_Advent Geneva',
       '105_153131370_Advent Geneva', '107_153157587_Advent Geneva',
       '108_153157587_Advent Geneva', '109_153149318_Morgan Stanley',
       '109_153157587_Advent Geneva', '10_153132306_Advent Geneva',
       '10_153132306_Morgan Stanley', '10_153147178_Advent Geneva',
       '10_153147178_Morgan Stanley', '10_153149112_Morgan Stanley',
       '10_153156649_Advent Geneva', '10_153157547_CITI',
       '110_153149805_Goldman Sachs', '110_153157587_Advent Geneva',
       '113_153149112_Advent Geneva', '114_153149112_Advent Geneva',
       '115_153149112_Advent Geneva', '116_153149112_Advent Geneva',
       '116_153149122_Goldman Sachs', '116_153152806_Goldman Sachs',
       '117_153149112_Advent Geneva', '118_153149112_Advent Geneva',
       '119_153149112_Advent Geneva', '11_153132306_Advent Geneva',
       '11_153147178_Advent Geneva', '11_15315

In [5927]:
no_pair_ids = no_pair_id_data['filter_key'].unique()

In [269]:
aua_sub66.loc[aua_sub66['ViewData.Side1_UniqueIds'].isin(no_pair_ids),'Final_prediction'] = aua_sub66.loc[aua_sub66['ViewData.Side1_UniqueIds'].isin(no_pair_ids),'ViewData.Status']
aua_sub66.loc[aua_sub66['ViewData.Side0_UniqueIds'].isin(no_pair_ids),'Final_prediction'] = aua_sub66.loc[aua_sub66['ViewData.Side0_UniqueIds'].isin(no_pair_ids),'ViewData.Status']

In [270]:
aua_sub66['Type'].value_counts()

One_to_One/Open    1625
One_to_Many          37
Many_to_Many          3
Name: Type, dtype: int64

In [271]:
#aua_sub66

In [272]:
pb_side_grp = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['Predicted_action'].unique().reset_index()

In [273]:
pb_side_grp

,SideA.ViewData.Side1_UniqueIds,Predicted_action
0,1000_153157324_CITI,[No-Pair]
1,1006_153157324_CITI,[No-Pair]
2,100_153151582_Goldman Sachs,[No-Pair]
3,100_153157424_Morgan Stanley,[UMB_One_to_One]
4,100_153157570_Morgan Stanley,"[UMB_One_to_One, No-Pair]"
...,...,...
1194,9_153154904_Wells Fargo,[No-Pair]
1195,9_153155907_Goldman Sachs,[No-Pair]
1196,9_153157256_Lending Club,[No-Pair]
1197,9_153157457_Lending Club,[No-Pair]


In [274]:
pb_side_grp_status = X_test.groupby(['SideA.ViewData.Side1_UniqueIds'])['SideA.ViewData.Status'].unique().reset_index()
pb_side_grp_status['SideA.ViewData.Status'] = pb_side_grp_status['SideA.ViewData.Status'].apply(lambda x: str(x).replace("[",""))
pb_side_grp_status['SideA.ViewData.Status'] = pb_side_grp_status['SideA.ViewData.Status'].apply(lambda x: str(x).replace("]",""))
pb_side_grp['len'] = pb_side_grp.apply(lambda x: len(x['Predicted_action']), axis=1)
pb_side_grp['No_pair_flag'] = pb_side_grp.apply(lambda x: 1 if x['len'] == 1 and "No-Pair" in x['Predicted_action'] else 0, axis=1)


In [275]:
pb_side_grp = pd.merge(pb_side_grp,pb_side_grp_status, on='SideA.ViewData.Side1_UniqueIds', how='left')

In [276]:
#pb_side_grp['SideA.ViewData.Status'].value_counts()

In [277]:
#pb_side_grp = pd.merge(pb_side_grp,pb_side_grp_status, on='SideA.ViewData.Side1_UniqueIds', how='left')
pb_side_grp['Final_status'] = pb_side_grp.apply(lambda x: "Open Break" if x['SideA.ViewData.Status']=="'OB'" else("SDB" if x['SideA.ViewData.Status']=="'SDB'" else "NA"),axis=1)
pb_side_grp = pb_side_grp.rename(columns = {'SideA.ViewData.Side1_UniqueIds':'ViewData.Side1_UniqueIds'})



pb_side_grp1 = pb_side_grp[pb_side_grp['No_pair_flag']==1]
pb_side_grp1 = pb_side_grp1.reset_index()
pb_side_grp1 = pb_side_grp1.drop('index',1)

In [278]:
aua_sub77 = pd.merge(aua_sub66 ,pb_side_grp1[['ViewData.Side1_UniqueIds','Final_status']], on ='ViewData.Side1_UniqueIds',how='left')

In [279]:
aua_sub77.loc[(~aua_sub77['Final_status'].isnull()) & (aua_sub77['ViewData.Side0_UniqueIds']=='nan'),'Final_prediction'] = aua_sub77.loc[(~aua_sub77['Final_status'].isnull()) & (aua_sub77['ViewData.Side0_UniqueIds']=='nan'),'Final_status']

In [280]:
pb_side_grp_B = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['Predicted_action'].unique().reset_index()

In [281]:
pb_side_grp_B_status = X_test.groupby(['SideB.ViewData.Side0_UniqueIds'])['SideB.ViewData.Status'].unique().reset_index()
pb_side_grp_B_status['SideB.ViewData.Status'] = pb_side_grp_B_status['SideB.ViewData.Status'].apply(lambda x: str(x).replace("[",""))
pb_side_grp_B_status['SideB.ViewData.Status'] = pb_side_grp_B_status['SideB.ViewData.Status'].apply(lambda x: str(x).replace("]",""))
pb_side_grp_B['len'] = pb_side_grp_B.apply(lambda x: len(x['Predicted_action']), axis=1)
pb_side_grp_B['No_pair_flag'] = pb_side_grp_B.apply(lambda x: 1 if x['len'] == 1 and "No-Pair" in x['Predicted_action'] else 0, axis=1)


In [282]:
pb_side_grp_B = pd.merge(pb_side_grp_B,pb_side_grp_B_status, on='SideB.ViewData.Side0_UniqueIds', how='left')
pb_side_grp_B['Final_status_B'] = pb_side_grp_B.apply(lambda x: "Open Break" if x['SideB.ViewData.Status']=="'OB'" else("SDB" if x['SideB.ViewData.Status']=="'SDB'" else "NA"),axis=1)
pb_side_grp_B = pb_side_grp_B.rename(columns = {'SideB.ViewData.Side0_UniqueIds':'ViewData.Side0_UniqueIds'})



pb_side_grp2 = pb_side_grp_B[pb_side_grp_B['No_pair_flag']==1]
pb_side_grp2 = pb_side_grp2.reset_index()
pb_side_grp2 = pb_side_grp2.drop('index',1)

In [283]:
aua_sub88 = pd.merge(aua_sub77 ,pb_side_grp2[['ViewData.Side0_UniqueIds','Final_status_B']], on ='ViewData.Side0_UniqueIds',how='left')

In [284]:
aua_sub88.loc[(~aua_sub88['Final_status_B'].isnull()) & (aua_sub88['ViewData.Side1_UniqueIds']=='nan'),'Final_prediction'] = aua_sub88.loc[(~aua_sub88['Final_status_B'].isnull()) & (aua_sub88['ViewData.Side1_UniqueIds']=='nan'),'Final_status_B']

In [285]:
aua_sub99 = aua_sub88[(aua_sub88['ViewData.Status']!='SDB')]
aua_sub99 = aua_sub99.reset_index()
aua_sub99 = aua_sub99.drop('index',1)

In [286]:
aua_sub99['Final_prediction'] = aua_sub99['Final_prediction'].fillna('Open Break')
aua_sub99 = aua_sub99.reset_index()
aua_sub99 = aua_sub99.drop('index',1)

In [287]:
aua_sub99['ViewData.Status'] = aua_sub99['ViewData.Status'].astype(str)
aua_sub99['Final_prediction'] = aua_sub99['Final_prediction'].astype(str)


In [288]:
#X_test
aua_sub[aua_sub['ViewData.Side0_UniqueIds']=='789_125897734_Advent Geneva']

,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,ViewData.Status,key,Final_prediction_x,probability_UMR,probability_No_pair,probability_UMT,probability_UMB,Final_predicted_break,Final_prediction_y


In [289]:
#aua[aua['ViewData.Side0_UniqueIds'] == '789_125897734_Advent Geneva']

## Summary file 

In [290]:
break_id_merge = meo[meo['ViewData.Status'].isin(['OB','SDB','UOB','UDB','SPM'])][['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','ViewData.BreakID']].drop_duplicates()
break_id_merge = break_id_merge.reset_index()
break_id_merge = break_id_merge.drop('index',1)


In [291]:
break_id_merge

,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,ViewData.BreakID
0,NaN,6_153157658_Lending Club,146865848
1,NaN,2_153157658_Lending Club,146865847
2,NaN,9_153157658_Lending Club,146865846
3,NaN,3_153157658_Lending Club,146865845
4,NaN,8_153157658_Lending Club,146865844
...,...,...,...
1704,NaN,113_153157324_CITI,146590029
1705,1791_153157324_Advent Geneva,NaN,146590000
1706,NaN,349_153157324_CITI,146590203
1707,NaN,401_153157324_CITI,146590202


In [292]:
break_id_merge['key'] = break_id_merge['ViewData.Side0_UniqueIds'].astype(str) + break_id_merge['ViewData.Side1_UniqueIds'].astype(str)

In [293]:
final = pd.merge(aua_sub99,break_id_merge[['key','ViewData.BreakID']], on='key', how='left')

In [294]:
aua_sub[aua_sub['ViewData.Side1_UniqueIds']=='233_125897734_Morgan Stanley']

,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,ViewData.Status,key,Final_prediction_x,probability_UMR,probability_No_pair,probability_UMT,probability_UMB,Final_predicted_break,Final_prediction_y


In [295]:
break_id_merge[break_id_merge['ViewData.BreakID']=='1334352248']

,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,ViewData.BreakID,key


In [296]:
break_id_merge.dtypes

ViewData.Side0_UniqueIds    object
ViewData.Side1_UniqueIds    object
ViewData.BreakID            object
key                         object
dtype: object

In [297]:
#final[final['ViewData.BreakID'].isnull()]

final = pd.merge(final,break_id_merge[['ViewData.Side0_UniqueIds','ViewData.BreakID']], on='ViewData.Side0_UniqueIds', how='left')

In [298]:
final.loc[final['ViewData.BreakID_x'].isnull(),'ViewData.BreakID_x'] = final.loc[final['ViewData.BreakID_x'].isnull(),'ViewData.BreakID_y']

In [299]:
final = final.rename(columns={'ViewData.BreakID_x':'ViewData.BreakID'})
final = final.drop('ViewData.BreakID_y',1)

In [300]:
final1 = final[(final['Type']=='One_to_One/Open') & (final['probability_No_pair'].isnull())]
final1 = final1.reset_index()
final1 = final1.drop('index',1)


final2 = final[~((final['Type']=='One_to_One/Open') & (final['probability_No_pair'].isnull()))]
final2 = final2.reset_index()
final2 = final2.drop('index',1)

In [301]:
final1['probability_UMR'].fillna(0.0024,inplace=True)
final1['probability_UMB'].fillna(0.004124,inplace=True)
final1['probability_UMT'].fillna(0.00155,inplace=True)
final1['probability_No_pair'].fillna(0.9922,inplace=True)

In [302]:
final3 = pd.concat([final1, final2], axis=0)

In [303]:
final3['ML_flag'] = final3.apply(lambda x: "ML" if x['Type']=='One_to_One/Open' else "Non-ML", axis=1)

In [304]:
prediction_cols = ['ViewData.BreakID', 'ViewData.Side1_UniqueIds', 'ViewData.Side0_UniqueIds','ViewData.Age' ,
       'probability_No_pair', 'probability_UMR','probability_UMB', 'probability_UMT',
       'Final_predicted_break', 'Type', 'ML_flag','ViewData.Status', 'Final_prediction']


final4 = final3[prediction_cols]

final4 = final4.rename(columns ={'ViewData.Status':'Actual_Status', 'Final_prediction': 'Predicted_Status'})

In [305]:
final4[final4['ML_flag']=='ML']

,ViewData.BreakID,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,probability_No_pair,probability_UMR,probability_UMB,probability_UMT,Final_predicted_break,Type,ML_flag,Actual_Status,Predicted_Status
0,146865842,nan,2_153157658_Advent Geneva,0,0.9922,0.0024,0.004124,0.00155,NaN,One_to_One/Open,ML,UCB,UCB
1,146865829,nan,1_153157658_Advent Geneva,0,0.9922,0.0024,0.004124,0.00155,NaN,One_to_One/Open,ML,UCB,UCB
2,146865836,nan,3_153157658_Advent Geneva,0,0.9922,0.0024,0.004124,0.00155,NaN,One_to_One/Open,ML,UCB,UCB
3,146682031,nan,8_153157457_Advent Geneva,0,0.9922,0.0024,0.004124,0.00155,NaN,One_to_One/Open,ML,UCB,UCB
4,146865834,nan,6_153157658_Advent Geneva,0,0.9922,0.0024,0.004124,0.00155,NaN,One_to_One/Open,ML,UCB,UCB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,NaN,69_153154765_CITI,23_153154765_Advent Geneva,20,0.0000,0.0001,0.999800,0.00000,NaN,One_to_One/Open,ML,UMB-Carry-Forward,UMB-Carry-Forward
460,NaN,1_153155908_CITI,3_153155908_Advent Geneva,13,0.0000,0.0001,0.999800,0.00000,NaN,One_to_One/Open,ML,UMB-Carry-Forward,UMB-Carry-Forward
461,NaN,61_153153898_CITI,95_153153898_Advent Geneva,26,0.0000,0.0001,0.999800,0.00000,NaN,One_to_One/Open,ML,UMB-Carry-Forward,UMB-Carry-Forward
462,NaN,77_153154765_CITI,21_153154765_Advent Geneva,20,0.0000,0.0001,0.999800,0.00000,NaN,One_to_One/Open,ML,UMB-Carry-Forward,UMB-Carry-Forward


In [306]:
#crosstab_table

In [307]:
NA_status_file = final4[(final4['Type']=='One_to_One/Open') & (final4['Predicted_Status']=='NA')]
NA_status_file = NA_status_file.reset_index()
NA_status_file = NA_status_file.drop('index',1)

In [308]:
final5 = final4[~((final4['Type']=='One_to_One/Open') & (final4['Predicted_Status']=='NA'))]
final5 = final5.reset_index()
final5 = final5.drop('index',1)

In [309]:
NA_status_file_A_side = NA_status_file[NA_status_file['ViewData.Side0_UniqueIds']=='nan']
NA_status_file_B_side = NA_status_file[NA_status_file['ViewData.Side1_UniqueIds']=='nan']

In [310]:
gg = X_test[X_test['SideA.ViewData.BreakID_A_side'].isin(NA_status_file_A_side['ViewData.BreakID'].unique())].groupby(['SideA.ViewData.BreakID_A_side'])['Predicted_action'].unique().reset_index()
gg.columns = ['ViewData.BreakID','Predicted_action']
gg['NA_prediction_A'] = 'Open Break'

kk = X_test[X_test['SideB.ViewData.BreakID_B_side'].isin(NA_status_file_B_side['ViewData.BreakID'].unique())].groupby(['SideB.ViewData.BreakID_B_side'])['Predicted_action'].unique().reset_index()
kk.columns = ['ViewData.BreakID','Predicted_action']
kk['NA_prediction_B'] = 'Open Break'

In [311]:
gg['ViewData.BreakID'] = gg['ViewData.BreakID'].astype(str)
kk['ViewData.BreakID'] = kk['ViewData.BreakID'].astype(str)

In [312]:
final6 = pd.merge(NA_status_file, gg[['ViewData.BreakID','NA_prediction_A']], on='ViewData.BreakID', how='left')
final6 = pd.merge(final6, kk[['ViewData.BreakID','NA_prediction_B']], on='ViewData.BreakID', how='left')

In [313]:
final6.loc[final6['NA_prediction_A'].isnull(),'Predicted_Status'] = 'Open Break'
final6.loc[final6['NA_prediction_B'].isnull(),'Predicted_Status'] = 'Open Break'

In [314]:
final6 = final6.drop(['NA_prediction_A','NA_prediction_B'],1)

In [315]:
final5[final5['ViewData.Side0_UniqueIds']=='789_125897734_Advent Geneva']

,ViewData.BreakID,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,probability_No_pair,probability_UMR,probability_UMB,probability_UMT,Final_predicted_break,Type,ML_flag,Actual_Status,Predicted_Status


In [316]:
final7 = pd.concat([final5, final6], axis=0)
final7 = final7.reset_index()
final7 = final7.drop('index',1)

In [317]:
X_test.head(3)

,SideA.ViewData.B-P Net Amount,SideA.ViewData.Price,SideA.ViewData.Quantity,SideB.ViewData.Accounting Net Amount,SideB.ViewData.Price,SideB.ViewData.Quantity,Trade_Date_match,Settle_Date_match,Fund_match,Amount_diff_2,Trade_date_diff,Settle_date_diff,SideA.ISIN_NA,SideB.ISIN_NA,ViewData.Combined Fund,ViewData.Combined Transaction Type,All_key_nan,new_key_match,SideA.ViewData._ID,SideB.ViewData._ID,amount_percent,SideB.ViewData.Status,SideB.ViewData.BreakID_B_side,SideA.ViewData.Status,SideA.ViewData.BreakID_A_side,SideB.ViewData.Side0_UniqueIds,SideA.ViewData.Side1_UniqueIds,Predicted_action,probability_No_pair,probability_UMB,probability_UMR,probability_UMT,prob_key,key
0,19005.12,0.0,0.0,54389.06,0.0000,0.0,0,0,1,35383.94,-92,-95.0,0,0,QPQP,returnofcapreturnofcap,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47e,34.942909,OB,135683259,SMB-OB,135961673,383_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,No-Pair,0.869639,0.122840,0.002844,0.004677,135683259No-Pair,383_153144970_Advent Geneva6_153139764_Bank of...
1,19005.12,0.0,0.0,-108668.98,0.0918,0.0,0,0,1,-127674.10,-59,-61.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e47d,-17.489002,OB,135961703,SMB-OB,135961673,299_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,No-Pair,0.927125,0.069851,0.000948,0.002076,135961703No-Pair,299_153144970_Advent Geneva6_153139764_Bank of...
2,19005.12,0.0,0.0,-108668.98,0.0918,0.0,0,0,1,-127674.10,-30,-33.0,0,0,QPQP,returnofcapdividend,0,1,5ef9863d7781ec114071e49e,5ef9863d7781ec114071e487,-17.489002,OB,135961702,SMB-OB,135961673,77_153144970_Advent Geneva,6_153139764_Bank of America Merrill Lynch,No-Pair,0.980732,0.017789,0.000382,0.001097,135961702No-Pair,77_153144970_Advent Geneva6_153139764_Bank of ...


In [318]:
pair_match = X_test[X_test['Predicted_action']!='No-Pair']
pair_match = pair_match.reset_index()
pair_match = pair_match.drop('index',1)


In [319]:
pair_match = pair_match[['Predicted_action',
       'probability_No_pair', 'probability_UMB', 'probability_UMR',
       'probability_UMT', 'key']]
pair_match.columns = ['New_Predicted_action',
       'New_probability_No_pair', 'New_probability_UMB', 'New_probability_UMR',
       'New_probability_UMT','key']

In [320]:
pair_match['New_Predicted_action'] = pair_match['New_Predicted_action'].apply(lambda x: 'UMR' if x=='UMR_One_to_One' else("UMT" if x=='UMT_One_to_One' else("UMB" if x== "UMB_One_to_One" else x)))

In [321]:
final7['key'] = final7['ViewData.Side0_UniqueIds'].astype(str) + final7['ViewData.Side1_UniqueIds'].astype(str)

In [322]:
final8 = pd.merge(final7,pair_match, on='key', how='left')

In [323]:
final8[~final8['New_Predicted_action'].isnull()]['Predicted_Status'].value_counts()

UMR           54
Open Break     2
UMT            1
Name: Predicted_Status, dtype: int64

In [324]:
final8[~final8['New_Predicted_action'].isnull()]['Actual_Status'].value_counts()

UMR    56
UMT     1
Name: Actual_Status, dtype: int64

In [325]:
final8[~final8['New_Predicted_action'].isnull()]['New_Predicted_action'].value_counts()

UMR    56
UMT     1
Name: New_Predicted_action, dtype: int64

In [326]:
final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'Predicted_Status'] = final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'New_Predicted_action']
final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'probability_No_pair'] = final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'New_probability_No_pair']
final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'probability_UMB'] = final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'New_probability_UMB']
final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'probability_UMR'] = final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'New_probability_UMR']
final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'probability_UMT'] = final8.loc[(~final8['New_Predicted_action'].isnull()) & (final8['New_Predicted_action']!= final8['Predicted_Status']),'New_probability_UMT']

In [327]:
final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMT'),'probability_UMT'] = final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMT'),'New_probability_UMT']
final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMR'),'probability_UMR'] = final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMR'),'New_probability_UMR']
final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMB'),'probability_UMB'] = final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMB'),'New_probability_UMB']
#final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='Open Break'),'probability_UMB'] = final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMB'),'New_probability_UMB']


final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMT'),'probability_No_Pair'] = 0.002
final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMR'),'probability_No_Pair'] = 0.002
final8.loc[(final8['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMB'),'probability_No_Pair'] = 0.002

In [328]:
umr_break_array_match = prediction_table[prediction_table['Final_prediction']=='UMR_One_to_One'][['SideB.ViewData.BreakID_B_side','Final_predicted_break']]
umt_break_array_match = prediction_table[prediction_table['Final_prediction']=='UMT_One_to_One'][['SideB.ViewData.BreakID_B_side','Final_predicted_break']]

umb_break_array_match = prediction_table[prediction_table['Final_prediction']=='UMB_One_to_One'][['SideB.ViewData.BreakID_B_side','Final_predicted_break']]

umr_break_array_match.columns = np.array(['ViewData.BreakID','New_Final_predicted_break_UMR'])
umt_break_array_match.columns = np.array(['ViewData.BreakID','New_Final_predicted_break_UMT'])
umb_break_array_match.columns = np.array(['ViewData.BreakID','New_Final_predicted_break_UMB'])


In [329]:
#umr_break_array_match['New_Final_predicted_break_UMR'] = umr_break_array_match['New_Final_predicted_break_UMR'].astype(str) 
#umb_break_array_match['New_Final_predicted_break_UMB'] = umb_break_array_match['New_Final_predicted_break_UMB'].astype(str) 
#umt_break_array_match['New_Final_predicted_break_UMT'] = umt_break_array_match['New_Final_predicted_break_UMT'].astype(str) 

In [330]:
final9 = pd.merge(final8, umr_break_array_match, on ='ViewData.BreakID', how='left')
final9 = pd.merge(final9, umt_break_array_match, on ='ViewData.BreakID', how='left')
final9 = pd.merge(final9, umb_break_array_match, on ='ViewData.BreakID', how='left')


In [331]:
final9.loc[(final9['Final_predicted_break'].isnull()) & (final9['Predicted_Status']=='UMT'),'Final_predicted_break'] = final9.loc[(final9['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMT'),'New_Final_predicted_break_UMT']
final9.loc[(final9['Final_predicted_break'].isnull()) & (final9['Predicted_Status']=='UMR'),'Final_predicted_break'] = final9.loc[(final9['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMR'),'New_Final_predicted_break_UMR']
final9.loc[(final9['Final_predicted_break'].isnull()) & (final9['Predicted_Status']=='UMB'),'Final_predicted_break'] = final9.loc[(final9['Final_predicted_break'].isnull()) & (final8['Predicted_Status']=='UMB'),'New_Final_predicted_break_UMB']


In [332]:
#final9[(final9['Actual_Status']=='UMB') & (final9['Predicted_Status']=='UMB') & (final9['ML_flag']=='ML')]['Final_predicted_break']

In [333]:
final9.columns

Index(['ViewData.BreakID', 'ViewData.Side1_UniqueIds',
       'ViewData.Side0_UniqueIds', 'ViewData.Age', 'probability_No_pair',
       'probability_UMR', 'probability_UMB', 'probability_UMT',
       'Final_predicted_break', 'Type', 'ML_flag', 'Actual_Status',
       'Predicted_Status', 'key', 'New_Predicted_action',
       'New_probability_No_pair', 'New_probability_UMB', 'New_probability_UMR',
       'New_probability_UMT', 'probability_No_Pair',
       'New_Final_predicted_break_UMR', 'New_Final_predicted_break_UMT',
       'New_Final_predicted_break_UMB'],
      dtype='object')

In [334]:
final9 = final9.drop(['key','New_Predicted_action',
       'New_probability_No_pair', 'New_probability_UMB', 'New_probability_UMR',
       'New_probability_UMT','New_Final_predicted_break_UMR',
       'New_Final_predicted_break_UMT', 'New_Final_predicted_break_UMB'], 1)




In [335]:
final9

,ViewData.BreakID,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,probability_No_pair,probability_UMR,probability_UMB,probability_UMT,Final_predicted_break,Type,ML_flag,Actual_Status,Predicted_Status,probability_No_Pair
0,146865842,nan,2_153157658_Advent Geneva,0,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,ML,UCB,UCB,NaN
1,146865829,nan,1_153157658_Advent Geneva,0,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,ML,UCB,UCB,NaN
2,146865836,nan,3_153157658_Advent Geneva,0,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,ML,UCB,UCB,NaN
3,146682031,nan,8_153157457_Advent Geneva,0,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,ML,UCB,UCB,NaN
4,146865834,nan,6_153157658_Advent Geneva,0,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,ML,UCB,UCB,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,146797484,nan,105_153157324_Advent Geneva,4,0.999897,0.000548,0.003908,0.000444,NaN,One_to_One/Open,ML,Open Break,Open Break,NaN
1634,146796858,nan,949_153157324_Advent Geneva,4,0.999901,0.000545,0.003455,0.000443,NaN,One_to_One/Open,ML,Open Break,Open Break,NaN
1635,146797485,nan,106_153157324_Advent Geneva,4,0.999897,0.000548,0.003908,0.000444,NaN,One_to_One/Open,ML,Open Break,Open Break,NaN
1636,146796444,nan,952_153157324_Advent Geneva,0,0.999876,0.000505,0.004926,0.000460,NaN,One_to_One/Open,ML,UCB,Open Break,NaN


In [336]:
#final8['Type'].value_counts()

In [337]:
#meo1 = pd.read_csv("//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO/MeoCollections.MEO_HST_RecData_125_2020-06-8.csv",usecols=new_cols)

In [338]:
#meo1[meo1['ViewData.Side1_UniqueIds']=='6_125858636_Goldman Sachs']

## Merging columns from the transaction table

In [344]:
import pandas as pd

cols_to_show = [
'Account Type',
'Accounting Net Amount',
#'Accounting Net Amount Difference',
#'Activity Code',
'Age',
'Alt ID 1',
'Asset Type Category',
#'Bloomberg_Yellow_Key',
'B-P Net Amount',
#'B-P Net Amount Difference',
#'B-P Net Amount Difference Absolute',
'BreakID',
'Business Date',
#'Call Put Indicator',
'Cancel Amount',
'Cancel Flag',
'Commission',
'Currency',
'CUSIP',
'Custodian',
#'Custodian Account',
'Department',
'Description',
'ExpiryDate',
'ExternalComment2',
'Fund',
#'FX Rate',
#'Interest Amount',
'InternalComment2',
'Investment ID',
'Investment Type',
'Is Combined Data',
'ISIN',
'Keys',
'Knowledge Date',
'Mapped Custodian Account',
'Net Amount Difference',
'Non Trade Description',
#'OTE Custodian Account',
#'OTE Ticker',
'PB Account Numeric',
'Portfolio ID',
'Portolio',
'Price',
'Prime Broker',
#'Principal Amount',
'Quantity',
#'Sec Fees',
'SEDOL',
'Settle Date',
'Status',
#'Strike Price',
'System Comments',
'Ticker',
'Trade Date',
'Trade Expenses',
'Transaction Category',
'Transaction ID',
'Transaction Type',
'Underlying Cusip',
'Underlying Investment ID',
'Underlying ISIN',
'Underlying Sedol',
'Underlying Ticker',
'UserTran1',
'UserTran2',
'Value Date',
] 
add = ['ViewData.Side0_UniqueIds', 'ViewData.Side1_UniqueIds']
viewdata_cols_to_show = ['ViewData.' + x for x in cols_to_show] + add


In [347]:
meo_final = pd.read_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Soros/JuneData/MEO/MeoCollections_SOROS.MEO_HST_RecData_153_2020-06-25.csv',usecols = viewdata_cols_to_show)
aua_final = pd.read_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Soros/JuneData/AUA/AUACollections_SOROS.AUA_HST_RecData_153_2020-06-25.csv',usecols = viewdata_cols_to_show)


C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,173,183,212,266,268,270,272,274,280) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [348]:
#final_predictions = pd.read_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions/Final_Predictions_Table_HST_RecData_125_2020-06-1.csv')


In [349]:
final_predictions = final9.copy()

In [350]:
#final_predictions.groupby(['Actual_Status'])['Predicted_Status'].value_counts()


In [351]:
#final_predictions[(final_predictions['Actual_Status'] == 'Open Break') & (final_predictions['Predicted_Status'] == 'UMR')][['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds']]



In [352]:
#final_predictions.groupby(['ViewData.Side0_UniqueIds'])['ViewData.Side1_UniqueIds'].value_counts()


In [353]:
#final12[final12['Actual_Status']=='UMT']

In [354]:
final_predictions_both_present = final_predictions[(final_predictions['ViewData.Side0_UniqueIds'] !='nan') & (final_predictions['ViewData.Side1_UniqueIds']!='nan')]
final_predictions_side0_only = final_predictions[(final_predictions['ViewData.Side0_UniqueIds']!='nan') & (final_predictions['ViewData.Side1_UniqueIds'] =='nan')]
final_predictions_side1_only = final_predictions[(final_predictions['ViewData.Side0_UniqueIds']=='nan') & (final_predictions['ViewData.Side1_UniqueIds'] != 'nan')]
final_predictions_both_null = final_predictions[(final_predictions['ViewData.Side0_UniqueIds']=='nan') & (final_predictions['ViewData.Side1_UniqueIds']=='nan')]


In [355]:
final_predictions_both_present.shape

(125, 14)

In [356]:
aua_final = aua_final.drop_duplicates()
aua_final = aua_final.reset_index()
aua_final = aua_final.drop('index',1)



In [357]:
final_predictions_both_present_aua_merge = pd.merge(final_predictions_both_present,aua_final, on=['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds'], how='left' )
final_predictions_side0_only_aua_merge = pd.merge(final_predictions_side0_only,aua_final, on='ViewData.Side0_UniqueIds', how='left' )
final_predictions_side1_only_aua_merge = pd.merge(final_predictions_side1_only,aua_final, on='ViewData.Side1_UniqueIds', how='left' )



In [358]:
#final_predictions_side1_only_aua_merge

In [359]:
final_predictions_side0_only_aua_merge = final_predictions_side0_only_aua_merge.drop(['ViewData.BreakID_y', 'ViewData.Side1_UniqueIds_y', 'ViewData.Age_y'], 1)

final_predictions_side0_only_aua_merge = final_predictions_side0_only_aua_merge.rename(columns={'ViewData.BreakID_x': 'ViewData.BreakID'})
final_predictions_side0_only_aua_merge = final_predictions_side0_only_aua_merge.rename(columns={'ViewData.Side1_UniqueIds_x': 'ViewData.Side1_UniqueIds'})
final_predictions_side0_only_aua_merge = final_predictions_side0_only_aua_merge.rename(columns={'ViewData.Age_x': 'ViewData.Age'})


final_predictions_side1_only_aua_merge = final_predictions_side1_only_aua_merge.drop(['ViewData.BreakID_y', 'ViewData.Side0_UniqueIds_y', 'ViewData.Age_y'], 1)

final_predictions_side1_only_aua_merge = final_predictions_side1_only_aua_merge.rename(columns={'ViewData.BreakID_x': 'ViewData.BreakID'})
final_predictions_side1_only_aua_merge = final_predictions_side1_only_aua_merge.rename(columns={'ViewData.Side0_UniqueIds_x': 'ViewData.Side0_UniqueIds'})
final_predictions_side1_only_aua_merge = final_predictions_side1_only_aua_merge.rename(columns={'ViewData.Age_x': 'ViewData.Age'})



final_predictions_both_present_aua_merge = final_predictions_both_present_aua_merge.drop(['ViewData.BreakID_y', 'ViewData.Age_y'], 1)

final_predictions_both_present_aua_merge = final_predictions_both_present_aua_merge.rename(columns={'ViewData.BreakID_x': 'ViewData.BreakID'})
final_predictions_both_present_aua_merge = final_predictions_both_present_aua_merge.rename(columns={'ViewData.Age_x': 'ViewData.Age'})
                                                                                                   
                                                                                                                                                                                             

In [360]:
#final_prediction_show_cols = final_predictions_both_present_aua_merge.append([final_predictions_side0_only_aua_merge,final_predictions_side1_only_aua_merge])


In [361]:
final11 = pd.concat([final_predictions_both_present_aua_merge, final_predictions_side0_only_aua_merge,final_predictions_side1_only_aua_merge], axis=0)

In [362]:
final11 = final11.reset_index()
final11 = final11.drop('index',1)

In [363]:
final12 = final11.drop_duplicates(['ViewData.BreakID', 'ViewData.Side1_UniqueIds','ViewData.Side0_UniqueIds', 'ViewData.Age'])

In [364]:
final12.loc[(final12['Actual_Status']=='UCB'), 'ML_flag'] ='Non-ML'
final12.loc[(final12['Actual_Status']=='UCB'), 'Type'] = 'Closed Breaks'

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [365]:
final12.loc[final12['Actual_Status']=='UCB','Predicted_Status'] = 'No-Prediction'

In [366]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
print(classification_report(final12[final12['Type']=='One_to_One/Open']['Actual_Status'], final12[final12['Type']=='One_to_One/Open']['Predicted_Status']))

                   precision    recall  f1-score   support

       Open Break       0.99      1.00      0.99      1276
              UMB       0.00      0.00      0.00         4
UMB-Carry-Forward       1.00      1.00      1.00        16
              UMR       1.00      0.90      0.95        62
              UMT       1.00      0.33      0.50         3

         accuracy                           0.99      1361
        macro avg       0.80      0.65      0.69      1361
     weighted avg       0.99      0.99      0.99      1361



In [5675]:
report = classification_report(final12[final12['Type']=='One_to_One/Open']['Actual_Status'], final12[final12['Type']=='One_to_One/Open']['Predicted_Status'], output_dict=True)
accuracy_table = pd.DataFrame(report).transpose()

In [5676]:
from sklearn.metrics import confusion_matrix
crosstab_table = pd.crosstab(final12[final12['Type']=='One_to_One/Open']['Actual_Status'], final12[final12['Type']=='One_to_One/Open']['Predicted_Status'])

In [5678]:
crosstab_table

Predicted_Status,Open Break,UMR,UMT
Actual_Status,,,
Open Break,810,1,1
UMB,6,0,3
UMR,0,185,0
UMT,0,0,66


## Save results

In [ ]:
final12.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions/test2_show_cols.csv')


In [ ]:
accuracy_table.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions/Accuracy_table_all_june.csv')

In [ ]:
crosstab_table.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions/Crosstab_table_all_june.csv')

## Enitre month prediction

In [15]:
import pandas as pd 
import glob
df_list = []

path = "//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125/*.csv"
for fname in glob.glob(path):
    
    if "Accuracy_" not in fname and "Crosstab" not in fname and "All_june" not in fname:
        print(fname)
        df_list.append(pd.read_csv(fname))

//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125\Final_Predictions_Table_HST_RecData_125_2020-06-1.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125\Final_Predictions_Table_HST_RecData_125_2020-06-10.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125\Final_Predictions_Table_HST_RecData_125_2020-06-11.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125\Final_Predictions_Table_HST_RecData_125_2020-06-14.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125\Final_Predictions_Table_HST_RecData_125_2020-06-15.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125\Final_Predictions_Table_HST_RecData_125_2020-06-16.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125\Final_Predictions_Table_HST_RecData_125_2020-06-17.csv
//vitbl

In [21]:
all_june_data = pd.concat(df_list)

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [22]:
#all_june_data = all_june_data[final12.columns]

In [23]:
all_june_data.columns

Index(['Actual_Status', 'Final_predicted_break', 'ML_flag', 'Predicted_Status',
       'Type', 'Unnamed: 0', 'ViewData.Account Type',
       'ViewData.Accounting Net Amount', 'ViewData.Activity Code',
       'ViewData.Age', 'ViewData.Alt ID 1', 'ViewData.Asset Type Category',
       'ViewData.B-P Net Amount', 'ViewData.Bloomberg_Yellow_Key',
       'ViewData.BreakID', 'ViewData.Business Date', 'ViewData.CUSIP',
       'ViewData.Call Put Indicator', 'ViewData.Cancel Amount',
       'ViewData.Cancel Flag', 'ViewData.Commission', 'ViewData.Currency',
       'ViewData.Custodian', 'ViewData.Custodian Account',
       'ViewData.Department', 'ViewData.Description', 'ViewData.ExpiryDate',
       'ViewData.ExternalComment2', 'ViewData.FX Rate', 'ViewData.Fund',
       'ViewData.ISIN', 'ViewData.Interest Amount',
       'ViewData.InternalComment2', 'ViewData.Investment ID',
       'ViewData.Investment Type', 'ViewData.Is Combined Data',
       'ViewData.Keys', 'ViewData.Knowledge Date',
       '

In [19]:
##################### For Soros #######################

all_june_data = all_june_data[['ViewData.BreakID', 'ViewData.Side1_UniqueIds',
       'ViewData.Side0_UniqueIds', 'ViewData.Age', 'probability_No_pair',
       'probability_UMR', 'probability_UMB', 'probability_UMT',
       'Final_predicted_break', 'Type', 'ML_flag', 'Actual_Status',
       'Predicted_Status', 'ViewData.Account Type',
       'ViewData.Accounting Net Amount',
       'ViewData.Alt ID 1', 'ViewData.Asset Type Category',
       'ViewData.B-P Net Amount',
       'ViewData.Business Date', 'ViewData.CUSIP',
    'ViewData.Cancel Amount',
       'ViewData.Cancel Flag', 'ViewData.Commission', 'ViewData.Currency',
       'ViewData.Custodian', 'ViewData.Custodian Account',
       'ViewData.Department', 'ViewData.Description', 'ViewData.ExpiryDate',
       'ViewData.ExternalComment2', 'ViewData.Fund',
       'ViewData.ISIN',
       'ViewData.InternalComment2', 'ViewData.Investment ID',
       'ViewData.Investment Type', 'ViewData.Is Combined Data',
       'ViewData.Keys', 'ViewData.Knowledge Date',
       'ViewData.Mapped Custodian Account', 'ViewData.Net Amount Difference',
       'ViewData.Non Trade Description',
        'ViewData.PB Account Numeric',
       'ViewData.Portfolio ID', 'ViewData.Portolio', 'ViewData.Price',
       'ViewData.Prime Broker',
       'ViewData.Quantity', 'ViewData.SEDOL',
       'ViewData.Settle Date', 'ViewData.Status',
       'ViewData.System Comments', 'ViewData.Ticker', 'ViewData.Trade Date',
       'ViewData.Trade Expenses', 'ViewData.Transaction Category',
       'ViewData.Transaction ID', 'ViewData.Transaction Type',
       'ViewData.Underlying Cusip', 'ViewData.Underlying ISIN',
       'ViewData.Underlying Investment ID', 'ViewData.Underlying Sedol',
       'ViewData.Underlying Ticker', 'ViewData.UserTran1',
       'ViewData.UserTran2', 'ViewData.Value Date']]

In [25]:
all_june_data = all_june_data[['ViewData.BreakID', 'ViewData.Side1_UniqueIds',
       'ViewData.Side0_UniqueIds', 'ViewData.Age', 'probability_No_pair',
       'probability_UMR', 'probability_UMB', 'probability_UMT',
       'Final_predicted_break', 'Type', 'ML_flag', 'Actual_Status',
       'Predicted_Status', 'ViewData.Account Type',
       'ViewData.Accounting Net Amount', 'ViewData.Activity Code',
       'ViewData.Alt ID 1', 'ViewData.Asset Type Category',
       'ViewData.B-P Net Amount', 'ViewData.Bloomberg_Yellow_Key',
       'ViewData.Business Date', 'ViewData.CUSIP',
       'ViewData.Call Put Indicator', 'ViewData.Cancel Amount',
       'ViewData.Cancel Flag', 'ViewData.Commission', 'ViewData.Currency',
       'ViewData.Custodian', 'ViewData.Custodian Account',
       'ViewData.Department', 'ViewData.Description', 'ViewData.ExpiryDate',
       'ViewData.ExternalComment2', 'ViewData.FX Rate', 'ViewData.Fund',
       'ViewData.ISIN', 'ViewData.Interest Amount',
       'ViewData.InternalComment2', 'ViewData.Investment ID',
       'ViewData.Investment Type', 'ViewData.Is Combined Data',
       'ViewData.Keys', 'ViewData.Knowledge Date',
       'ViewData.Mapped Custodian Account', 'ViewData.Net Amount Difference',
       'ViewData.Non Trade Description', 'ViewData.OTE Custodian Account',
       'ViewData.OTE Ticker', 'ViewData.PB Account Numeric',
       'ViewData.Portfolio ID', 'ViewData.Portolio', 'ViewData.Price',
       'ViewData.Prime Broker', 'ViewData.Principal Amount',
       'ViewData.Quantity', 'ViewData.SEDOL', 'ViewData.Sec Fees',
       'ViewData.Settle Date', 'ViewData.Status', 'ViewData.Strike Price',
       'ViewData.System Comments', 'ViewData.Ticker', 'ViewData.Trade Date',
       'ViewData.Trade Expenses', 'ViewData.Transaction Category',
       'ViewData.Transaction ID', 'ViewData.Transaction Type',
       'ViewData.Underlying Cusip', 'ViewData.Underlying ISIN',
       'ViewData.Underlying Investment ID', 'ViewData.Underlying Sedol',
       'ViewData.Underlying Ticker', 'ViewData.UserTran1',
       'ViewData.UserTran2', 'ViewData.Value Date']]

In [26]:
#all_june_data = all_june_data[final12.columns]

In [27]:
all_june_data

,ViewData.BreakID,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,probability_No_pair,probability_UMR,probability_UMB,probability_UMT,Final_predicted_break,Type,...,ViewData.Transaction ID,ViewData.Transaction Type,ViewData.Underlying Cusip,ViewData.Underlying ISIN,ViewData.Underlying Investment ID,ViewData.Underlying Sedol,ViewData.Underlying Ticker,ViewData.UserTran1,ViewData.UserTran2,ViewData.Value Date
0,1.313893e+09,1285_125880831_CITI,1699_125882599_Advent Geneva,0,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,11887979,Expenses,NaN,NaN,GED,NaN,NaN,06022020BankInterestExpense93,Prime Broker Interest & Stock Loan - May 2020,06-01-2020
1,1.313898e+09,458_125882599_CITI,126_125882599_Advent Geneva,0,0.002000,0.996066,0.004124,0.001550,[1313898136 1313898138],One_to_One/Open,...,11868955,Sell,NaN,0757001858,CES,NaN,NaN,0757001858,NaN,06-01-2020
2,1.314441e+09,513_125881758_US BANK,64_125882870_Advent Geneva,0,0.002000,0.993438,0.004124,0.001550,[1314440621 1314440622],One_to_One/Open,...,11887770,Buy,NaN,NaN,NaN,NaN,NaN,0757685642,amending settledate,NaN
3,NaN,1102_125882975_Goldman Sachs,79_125882975_Advent Geneva,0,0.002000,0.997516,0.004124,0.001550,NaN,One_to_One/Open,...,11732956,Dividend,NaN,NaN,KKR,NaN,NaN,90535_400042,NaN,06-01-2020
4,1.313894e+09,1551_125880831_CITI,450_125882599_Advent Geneva,0,0.000339,0.997612,0.000043,0.002006,[1311423899],One_to_One/Open,...,11887879,Revenue,NaN,NaN,ROM,NaN,NaN,NaN,Securities Lending Income TAX,06-01-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,1.329215e+09,2994_125785315_Morgan Stanley,NaN,86,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,4ZH79H2N,AllocLeg,US78378X1072,US78378X1072,NaN,NaN,SPX.US,015G,NaN,NaN
1263,1.329214e+09,2451_125804334_Morgan Stanley,NaN,71,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,51H6H7X4,AllocLeg,US78378X1072,US78378X1072,NaN,NaN,SPX.US,015G,NaN,NaN
1264,1.329215e+09,2699_125785315_Morgan Stanley,NaN,86,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,4ZGWSDK6,AllocLeg,US78378X1072,US78378X1072,NaN,NaN,SPX.US,015G,NaN,NaN
1265,1.329215e+09,2704_125785315_Morgan Stanley,NaN,86,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,4ZH48ZYH,AllocLeg,US78378X1072,US78378X1072,NaN,NaN,SPX.US,015G,NaN,NaN


In [6552]:
all_june_data = pd.read_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Soros/JuneData/Final_Predictions_239/All_june_data_239.csv')

In [11]:
all_june_data

,ViewData.BreakID,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,probability_No_pair,probability_UMR,probability_UMB,probability_UMT,Final_predicted_break,Type,...,ViewData.Transaction ID,ViewData.Transaction Type,ViewData.Underlying Cusip,ViewData.Underlying ISIN,ViewData.Underlying Investment ID,ViewData.Underlying Sedol,ViewData.Underlying Ticker,ViewData.UserTran1,ViewData.UserTran2,ViewData.Value Date
0,1.311429e+09,2_379882656_BNP Paribas,55_379880844_Advent Geneva,0,0.9922,0.002400,0.004124,0.00155,NaN,One_to_One/Open,...,5733128,Interest,NaN,NaN,NaN,NaN,Q-Park Holding I BV Sr Sec Reg S E+2.00% 03/01...,NaN,NaN,NaN
1,1.311429e+09,4_379882656_BNP Paribas,48_379880844_Advent Geneva,0,0.9922,0.002400,0.004124,0.00155,NaN,One_to_One/Open,...,5733128,Interest,NaN,NaN,NaN,NaN,Q-Park Holding I BV Sr Sec Reg S E+2.00% 03/01...,NaN,NaN,NaN
2,NaN,1_379882658_HSBC,98_379880882_Advent Geneva,0,0.9922,0.002400,0.004124,0.00155,NaN,One_to_One/Open,...,6063595,Buy,NaN,NaN,NaN,NaN,Apache Corp Sr Unsec 4.75% 04/15/2043 Callable...,226994.11785v5,NaN,NaN
3,NaN,167_379883204_State Street,309_379881110_Advent Geneva,0,0.0020,0.959545,0.004124,0.00155,NaN,One_to_One/Open,...,6063602,Buy,NaN,NaN,NaN,NaN,Apache Corp Sr Unsec 4.75% 04/15/2043 Callable...,226994.13005v5,NaN,NaN
4,NaN,78_379883204_State Street,537_379881110_Advent Geneva,0,0.0020,0.975219,0.004124,0.00155,NaN,One_to_One/Open,...,6063465,Buy,NaN,NaN,NaN,NaN,Apache Corp Sr Unsec 5.10% 09/01/2040 Callable...,227004.13196v3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,1.329433e+09,105_379893947_State Street,NaN,0,0.9922,0.002400,0.004124,0.00155,NaN,One_to_One/Open,...,20FFKBBTN4Z,ForwardFX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,1.329433e+09,102_379893947_State Street,NaN,0,0.9922,0.002400,0.004124,0.00155,NaN,One_to_One/Open,...,20FFKBBTN4X,ForwardFX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,1.329433e+09,100_379893947_State Street,NaN,0,0.9922,0.002400,0.004124,0.00155,NaN,One_to_One/Open,...,20FFKBBTN4W,ForwardFX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,1.329433e+09,20_379893947_State Street,NaN,0,0.9922,0.002400,0.004124,0.00155,NaN,One_to_One/Open,...,20FHLBB5JVP,Internal Trans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
print(classification_report(all_june_data[all_june_data['Type']=='One_to_One/Open']['Actual_Status'], all_june_data[all_june_data['Type']=='One_to_One/Open']['Predicted_Status']))

                   precision    recall  f1-score   support

       Open Break       1.00      0.99      1.00     11895
              UMB       1.00      0.15      0.26        34
UMB-Carry-Forward       1.00      1.00      1.00         1
              UMR       0.98      1.00      0.99      3424
              UMT       0.98      0.99      0.98       891

         accuracy                           0.99     16245
        macro avg       0.99      0.83      0.85     16245
     weighted avg       0.99      0.99      0.99     16245



In [29]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
print(classification_report(all_june_data[all_june_data['Type']=='One_to_One/Open']['Actual_Status'], all_june_data[all_june_data['Type']=='One_to_One/Open']['Predicted_Status']))

                   precision    recall  f1-score   support

       Open Break       1.00      0.99      1.00     11895
              UMB       1.00      0.15      0.26        34
UMB-Carry-Forward       1.00      1.00      1.00         1
              UMR       0.98      1.00      0.99      3424
              UMT       0.98      0.99      0.98       891

         accuracy                           0.99     16245
        macro avg       0.99      0.83      0.85     16245
     weighted avg       0.99      0.99      0.99     16245



In [30]:
report_all_june = classification_report(all_june_data[all_june_data['Type']=='One_to_One/Open']['Actual_Status'], all_june_data[all_june_data['Type']=='One_to_One/Open']['Predicted_Status'], output_dict=True)
accuracy_table_all_june = pd.DataFrame(report_all_june).transpose()

In [31]:
accuracy_table_all_june

,precision,recall,f1-score,support
Open Break,0.996628,0.993779,0.995201,11895.00000
UMB,1.000000,0.147059,0.256410,34.00000
UMB-Carry-Forward,1.000000,1.000000,1.000000,1.00000
UMR,0.981604,0.997371,0.989425,3424.00000
UMT,0.979978,0.988777,0.984358,891.00000
accuracy,0.992490,0.992490,0.992490,0.99249
macro avg,0.991642,0.825397,0.845079,16245.00000
weighted avg,0.992555,0.992490,0.991843,16245.00000


In [32]:
from sklearn.metrics import confusion_matrix
crosstab_all_june =  pd.crosstab(all_june_data[all_june_data['Type']=='One_to_One/Open']['Actual_Status'], all_june_data[all_june_data['Type']=='One_to_One/Open']['Predicted_Status'])

In [6545]:
accuracy_table_all_june.to_csv('accuracy_all_j.csv')
crosstab_all_june.to_csv('cross_tab_soros.csv')

In [33]:
crosstab_all_june

Predicted_Status,Open Break,UMB,UMB-Carry-Forward,UMR,UMT
Actual_Status,,,,,
Open Break,11821,0,0,64,10
UMB,21,5,0,0,8
UMB-Carry-Forward,0,0,1,0,0
UMR,9,0,0,3415,0
UMT,10,0,0,0,881


In [35]:
accuracy_table_all_june.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125/accuracy_table_all_june_125.csv')
crosstab_all_june.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_125/crosstab_all_june_125.csv')

In [6517]:
all_june_data = all_june_data.reset_index()
all_june_data = all_june_data.drop('index',1)

In [34]:
all_june_data

,ViewData.BreakID,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,probability_No_pair,probability_UMR,probability_UMB,probability_UMT,Final_predicted_break,Type,...,ViewData.Transaction ID,ViewData.Transaction Type,ViewData.Underlying Cusip,ViewData.Underlying ISIN,ViewData.Underlying Investment ID,ViewData.Underlying Sedol,ViewData.Underlying Ticker,ViewData.UserTran1,ViewData.UserTran2,ViewData.Value Date
0,1.313893e+09,1285_125880831_CITI,1699_125882599_Advent Geneva,0,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,11887979,Expenses,NaN,NaN,GED,NaN,NaN,06022020BankInterestExpense93,Prime Broker Interest & Stock Loan - May 2020,06-01-2020
1,1.313898e+09,458_125882599_CITI,126_125882599_Advent Geneva,0,0.002000,0.996066,0.004124,0.001550,[1313898136 1313898138],One_to_One/Open,...,11868955,Sell,NaN,0757001858,CES,NaN,NaN,0757001858,NaN,06-01-2020
2,1.314441e+09,513_125881758_US BANK,64_125882870_Advent Geneva,0,0.002000,0.993438,0.004124,0.001550,[1314440621 1314440622],One_to_One/Open,...,11887770,Buy,NaN,NaN,NaN,NaN,NaN,0757685642,amending settledate,NaN
3,NaN,1102_125882975_Goldman Sachs,79_125882975_Advent Geneva,0,0.002000,0.997516,0.004124,0.001550,NaN,One_to_One/Open,...,11732956,Dividend,NaN,NaN,KKR,NaN,NaN,90535_400042,NaN,06-01-2020
4,1.313894e+09,1551_125880831_CITI,450_125882599_Advent Geneva,0,0.000339,0.997612,0.000043,0.002006,[1311423899],One_to_One/Open,...,11887879,Revenue,NaN,NaN,ROM,NaN,NaN,NaN,Securities Lending Income TAX,06-01-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,1.329215e+09,2994_125785315_Morgan Stanley,NaN,86,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,4ZH79H2N,AllocLeg,US78378X1072,US78378X1072,NaN,NaN,SPX.US,015G,NaN,NaN
1263,1.329214e+09,2451_125804334_Morgan Stanley,NaN,71,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,51H6H7X4,AllocLeg,US78378X1072,US78378X1072,NaN,NaN,SPX.US,015G,NaN,NaN
1264,1.329215e+09,2699_125785315_Morgan Stanley,NaN,86,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,4ZGWSDK6,AllocLeg,US78378X1072,US78378X1072,NaN,NaN,SPX.US,015G,NaN,NaN
1265,1.329215e+09,2704_125785315_Morgan Stanley,NaN,86,0.992200,0.002400,0.004124,0.001550,NaN,One_to_One/Open,...,4ZH48ZYH,AllocLeg,US78378X1072,US78378X1072,NaN,NaN,SPX.US,015G,NaN,NaN


## Save Results (Entire Month)

In [6187]:
all_june_data.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/Final_Predictions_123/All_june_data_123.csv')

In [19]:
accuracy_table_all_june.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/OakTree/JuneData/Final_Predictions_379/Accuracy_table_all_june_379.csv')

In [18]:
crosstab_all_june.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/OakTree/JuneData/Final_Predictions_379/Crosstab_table_all_june_379.csv')

In [6406]:
comment_file = pd.read_csv('1 month merged file prediction brk_com for weiss123.csv')

#comment_file = pd.read_csv('1 month merged file prediction brk_com for oak tree.csv')

In [6407]:
comment_file = comment_file.drop('Unnamed: 0', 1)

In [6393]:
#all_june_data
import glob
meo_list = []

path = "//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO/*.csv"
for fname in glob.glob(path):
    
    if "RecData_123" in fname:
        print(fname)
        meo_list.append(pd.read_csv(fname))

//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO\MeoCollections.MEO_HST_RecData_123_2020-06-1.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO\MeoCollections.MEO_HST_RecData_123_2020-06-10.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO\MeoCollections.MEO_HST_RecData_123_2020-06-11.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO\MeoCollections.MEO_HST_RecData_123_2020-06-14.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO\MeoCollections.MEO_HST_RecData_123_2020-06-15.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO\MeoCollections.MEO_HST_RecData_123_2020-06-16.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO\MeoCollections.MEO_HST_RecData_123_2020-06-17.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Weiss/JuneData/MEO\MeoCollections.MEO_HST_RecData_123_2020-06-18.csv
//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data

In [6394]:
meo_123 = pd.concat(meo_list)

C:\Users\consultant136\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [6396]:
meo_123= meo_123.reset_index()
meo_123 = meo_123.drop('index', 1)

In [6403]:
meo_123.drop_duplicates()[['ViewData.Side1_UniqueIds']]

,ViewData.Side1_UniqueIds
0,"38_123880759_CITI,36_123880759_CITI"
1,"35_123882606_CITI,36_123882606_CITI"
2,"90_123880759_CITI,91_123880759_CITI,89_1238807..."
3,"32_123882606_CITI,31_123882606_CITI,33_1238826..."
4,"92_123868848_CITI,94_123868848_CITI,95_1238688..."
...,...
2773,NaN
2774,NaN
2775,NaN
2776,NaN


In [6401]:
meo_123[meo_123['ViewData.Side1_UniqueIds'].isin(all_june_data[all_june_data['Actual_Status']=='Open Break']['ViewData.Side1_UniqueIds'].unique())]

,LastPerformedAction,MetaData.0._CombinedID,MetaData.0._IsCombined,MetaData.0._ParentID,MetaData.0._RecordID,MetaData.1._CombinedID,MetaData.1._IsCombined,MetaData.1._ParentID,MetaData.1._RecordID,SourceCombinationCode,TaskInstanceID,ViewData.Account Name,ViewData.Account Type,ViewData.Account Type Difference,ViewData.Accounting Account Type,ViewData.Accounting Activity Code,ViewData.Accounting Asset Type Category,ViewData.Accounting Base Currency,ViewData.Accounting Base Net Amount,ViewData.Accounting Bloomberg_Yellow_Key,ViewData.Accounting Break Tag,ViewData.Accounting CUSIP,ViewData.Accounting Call Put Indicator,ViewData.Accounting Cancel Amount,ViewData.Accounting Cancel Flag,ViewData.Accounting Commission,ViewData.Accounting Currency,ViewData.Accounting Custodian,ViewData.Accounting Custodian Account,ViewData.Accounting Custom 1,ViewData.Accounting Custom 10,ViewData.Accounting Custom 11,ViewData.Accounting Custom 12,ViewData.Accounting Custom 13,ViewData.Accounting Custom 14,ViewData.Accounting Custom 15,ViewData.Accounting Custom 2,ViewData.Accounting Custom 3,ViewData.Accounting Custom 4,ViewData.Accounting Custom 5,ViewData.Accounting Custom 6,ViewData.Accounting Custom 7,ViewData.Accounting Custom 8,ViewData.Accounting Custom 9,ViewData.Accounting Department,ViewData.Accounting Description,ViewData.Accounting ExpiryDate,ViewData.Accounting FX Rate,ViewData.Accounting Fund,ViewData.Accounting Group,...,ViewData.Source Combination,ViewData.Source Combination Code,ViewData.Source Type Code Combination,ViewData.Source Type Combination,ViewData.SourceTypeCode,ViewData.Status,ViewData.Strategy,ViewData.Strategy Difference,ViewData.Strike Price,ViewData.Strike Price Difference,ViewData.System Comments,ViewData.TabID,ViewData.Task Business Date,ViewData.Task ID,ViewData.Task Knowledge Date,ViewData.Ticker,ViewData.Ticker Difference,ViewData.Trade Date,ViewData.Trade Date Difference,ViewData.Trade Expenses,ViewData.Trade Expenses Difference,ViewData.Transaction Category,ViewData.Transaction Category Difference,ViewData.Transaction ID,ViewData.Transaction ID Difference,ViewData.Transaction Type,ViewData.Transaction Type Difference,ViewData.Type,ViewData.UnMapped,ViewData.Underlying Cusip,ViewData.Underlying Cusip Difference,ViewData.Underlying ISIN,ViewData.Underlying ISIN Difference,ViewData.Underlying Investment ID,ViewData.Underlying Investment ID Difference,ViewData.Underlying Sedol,ViewData.Underlying Sedol Difference,ViewData.Underlying Ticker,ViewData.Underlying Ticker Difference,ViewData.UserTran1,ViewData.UserTran2,ViewData.UserTran2 Difference,ViewData.Value Date,ViewData.Value Date Difference,ViewData.Workflow Remark,ViewData.Workflow Status,ViewData._Actions,ViewData._GroupID,ViewData._ID,_id
85,31,NaN,NaN,NaN,92.0,NaN,NaN,NaN,NaN,1909407759,123882606,NaN,NaN,False,NaN,NaN,Cash and Equivalents,NaN,7350.76,NaN,NaN,NaN,NaN,-7350.76,0.0,NaN,USD,CITI,CITIGM_FI_ON,CITI GM PB Fixed Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,To allocate FI ticket charge expense to strate...,NaN,NaN,ON,NaN,...,"Advent Geneva,CITI",1909407759,"Accounting,B-P","Accounting,Prime Broker",Accounting,OB,IR01,False,0.0,0.0,No matching record found,1,2020-06-02T00:00:00.000Z,123882606,6/3/2020,NaN,False,05-18-2020,-139.0,NaN,0.0,NaN,False,11889112,False,Expenses,False,Breaks,NaN,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,To allocate FI ticket charge expense to strate...,False,06-02-2020,-153.0,NaN,None,NaN,5ed72a42155458331c634eab,5ed72a42155458331c634eab,5ed72a43155458331c63500e
86,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909407759,123882606,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Advent Geneva,CITI",1909407759,"Accounting,B-P","Accounting,Prime Broker",B-P,OB,NaN,False,NaN,0.0,No matching record found,1,2020-06-02T00:00:00.000Z,123882606,6/3/2020,NaN,False,06-02-2020,154.0,NaN,0.0,JNL,False,Wir

In [6410]:
comment_file[(comment_file['Actual_Status']=='Open Break') & (comment_file['Predicted_comment'].isnull())]['ViewData.Side1_UniqueIds']

19      70_123882606_CITI
63      88_123895430_CITI
64      58_123895430_CITI
65      38_123895430_CITI
91      50_123897300_CITI
              ...        
453    102_123891808_CITI
454     97_123891808_CITI
466     49_123893657_CITI
467     34_123893657_CITI
468     75_123893657_CITI
Name: ViewData.Side1_UniqueIds, Length: 93, dtype: object

In [6412]:
#meo_123[meo_123['ViewData.Side1_UniqueIds']=='70_123882606_CITI']

In [6399]:
all_june_data[all_june_data['Actual_Status']=='Open Break']['ViewData.Side1_UniqueIds'].unique()

array(['70_123882606_CITI', '88_123895430_CITI', '58_123895430_CITI',
       '38_123895430_CITI', '50_123897300_CITI', '76_123897300_CITI',
       '35_123897300_CITI', '35_123899168_CITI', '48_123899168_CITI',
       '34_123899168_CITI', '47_123899168_CITI', '25_123899168_CITI',
       '70_123899168_CITI', '60_123899168_CITI', '69_123899168_CITI', nan,
       '71_123902864_CITI', '102_123902864_CITI', '70_123902864_CITI',
       '103_123902864_CITI', '101_123902864_CITI', '86_123902864_CITI',
       '69_123902864_CITI', '104_123902864_CITI', '87_123902864_CITI',
       '105_123902864_CITI', '51_123902864_CITI', '50_123902864_CITI',
       '49_123902864_CITI', '67_123902864_CITI', '47_123902864_CITI',
       '64_123902864_CITI', '84_123902864_CITI', '112_123902864_CITI',
       '87_123904625_CITI', '77_123904625_CITI', '78_123904625_CITI',
       '60_123904625_CITI', '61_123904625_CITI', '51_123906593_CITI',
       '44_123906593_CITI', '45_123906593_CITI', '36_123906593_CITI',
       '3

In [6386]:
all_june_data[all_june_data['Predicted_Status']=='Open Break']['ViewData.InternalComment2'].value_counts()

Match                                                                         26
up/down at settle date                                                        12
CITI booked reverse repo on 06/18 , Geneva yet to book                         8
Geneva booked the transfer wire on 06/16, CITI yet to book                     6
CITI booked Mays's billing fees on 06/15 , Geneva yet to book                  5
CITI booked transfer wire on 06/24 , Geneva yet to book                        4
CITI booked the cash repo margin on 06/05, Geneva yet to book                  4
CITI booked transfer wire on 06/11, Geneva yet to book                         3
CITI booked reverse repo on 06/17 , Geneva yet to book                         3
CITI booked the transfer wire on 06/29, Geneva yet to book                     3
CITI booked the cash repo margin on 06/17, Geneva yet to book                  3
CITI booked transfer wire on 06/10 , Geneva yet to book                        3
CITI booked transfer wire on

In [6314]:
#comment_file[comment_file['Actual_Status']=='Open Break']

In [6357]:
all_june_data['Predicted_Status'].value_counts()

No-Prediction        4601
Open Break           3019
UMR                   430
UMB-Carry-Forward     222
UMB                   160
UMT                    80
Unrecognized            1
Name: Predicted_Status, dtype: int64

In [6487]:
all_june_data.shape

(6293, 65)

In [6359]:
#comment_file.loc[comment_file['ViewData.Side0_UniqueIds']=='AA','ViewData.Side0_UniqueIds'] = np.nan

In [6360]:
#comment_file.loc[comment_file['ViewData.Side1_UniqueIds']=='BB','ViewData.Side1_UniqueIds'] = np.nan

In [6519]:
all_june_data['key'] = all_june_data['ViewData.Side0_UniqueIds'].astype(str) + all_june_data['ViewData.Side1_UniqueIds'].astype(str)

#comment_file['key'] = comment_file['ViewData.Side0_UniqueIds'].astype(str) + comment_file['ViewData.Side1_UniqueIds'].astype(str)

In [6520]:
#comment_file = comment_file.drop_duplicates()
#comment_file = comment_file.reset_index()
#comment_file = comment_file.drop('index',1)

In [6521]:
all_june_data = all_june_data.drop_duplicates()
all_june_data = all_june_data.reset_index()
all_june_data = all_june_data.drop('index',1)

In [6522]:
jj = all_june_data[all_june_data['key'].isin(all_june_data['key'].value_counts().reset_index()[all_june_data['key'].value_counts().reset_index()['key']>1]['index'])]

In [6523]:
jj['Actual_Status'].value_counts()

UMB-Carry-Forward    3284
Open Break           1472
UMB                   168
UCB                    46
Name: Actual_Status, dtype: int64

In [6524]:
all_june_data1 = all_june_data[~((all_june_data['key'].isin(jj['key'].unique())) & (all_june_data['Actual_Status']=='Open Break'))]
all_june_data1 = all_june_data1.reset_index()
all_june_data1 = all_june_data1.drop('index',1)

In [6525]:
#comment_file1 = comment_file[~((comment_file['key'].isin(jj['key'].unique())) & (comment_file['Actual_Status']=='Open Break'))]
#comment_file1 = comment_file1.reset_index()
#comment_file1 = comment_file1.drop('index',1)

In [6526]:
all_june_data1['key'].value_counts()

11_239149101_Advent Geneva19_239149101_JP Morgan                                                                                                                                                                                                                                                                                                                                                                                                                                              20
17_239134899_Advent Geneva,16_239134899_Advent Geneva10_239135157_JP Morgan                                                                                                                                                                                                                                                                                                                                                                                                                   20
22_239141943_Advent Geneva37_239141943

In [6527]:
all_june_data2 = all_june_data1[~((all_june_data1['Actual_Status']=='UCB') & (all_june_data1['ViewData.BreakID'].isnull()))]
all_june_data2 = all_june_data2.reset_index()
all_june_data2 = all_june_data2.drop('index',1)

In [6528]:
all_june_data2['key'].value_counts()

11_239149101_Advent Geneva19_239149101_JP Morgan                                                                                                                                                                                                                                                                                                                                                                                                                                              20
17_239134899_Advent Geneva,16_239134899_Advent Geneva10_239135157_JP Morgan                                                                                                                                                                                                                                                                                                                                                                                                                   20
22_239141943_Advent Geneva37_239141943

In [6529]:
all_june_data2[all_june_data2['key']=='11_239149101_Advent Geneva19_239149101_JP Morgan']

,ViewData.BreakID,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,probability_No_pair,probability_UMR,probability_UMB,probability_UMT,Final_predicted_break,Type,ML_flag,Actual_Status,Predicted_Status,ViewData.Account Type,ViewData.Accounting Net Amount,ViewData.Alt ID 1,ViewData.Asset Type Category,ViewData.B-P Net Amount,ViewData.Business Date,ViewData.CUSIP,ViewData.Cancel Amount,ViewData.Cancel Flag,ViewData.Commission,ViewData.Currency,ViewData.Custodian,ViewData.Custodian Account,ViewData.Department,ViewData.Description,ViewData.ExpiryDate,ViewData.ExternalComment2,ViewData.Fund,ViewData.ISIN,ViewData.InternalComment2,ViewData.Investment ID,ViewData.Investment Type,ViewData.Is Combined Data,ViewData.Keys,ViewData.Knowledge Date,ViewData.Mapped Custodian Account,ViewData.Net Amount Difference,ViewData.Non Trade Description,ViewData.PB Account Numeric,ViewData.Portfolio ID,ViewData.Portolio,ViewData.Price,ViewData.Prime Broker,ViewData.Quantity,ViewData.SEDOL,ViewData.Settle Date,ViewData.Status,ViewData.System Comments,ViewData.Ticker,ViewData.Trade Date,ViewData.Trade Expenses,ViewData.Transaction Category,ViewData.Transaction ID,ViewData.Transaction Type,ViewData.Underlying Cusip,ViewData.Underlying ISIN,ViewData.Underlying Investment ID,ViewData.Underlying Sedol,ViewData.Underlying Ticker,ViewData.UserTran1,ViewData.UserTran2,ViewData.Value Date,key
173,NaN,19_239149101_JP Morgan,11_239149101_Advent Geneva,30.0,0.0,0.0001,0.9998,0.0,NaN,One_to_One/Open,ML,UMB-Carry-Forward,UMB-Carry-Forward,NaN,-6.48,QP-CHN,CMDTYFUT,-47.88,05-04-2020,NaN,-6.48,0.0,6.48,EUR,JPM,QP_LP_INT_JPMF,NaN,MILL WHEAT EURO Sep20,09-10-2020,NaN,QP,NaN,05/28- Email sent to PB to provide backup on a...,CAU0,CMDTYFUT,NaN,"Currency,Mapped Custodian Account",05-05-2020,QP_LP_INT_JPMF,-41.4,NaN,14164,QP,QP,183.8389,JPMF,36.0,NaN,05-04-2020,UMB,Mismatch in Net Amount after tolerance. Tolera...,CAU0,05-04-2020,NaN,NaN,4419979,Buy,NaN,NaN,DUMMY,NaN,NaN,JP MORGAN FUTURES,2913535_SOROS,NaN,11_239149101_Advent Geneva19_239149101_JP Morgan
468,NaN,19_239149101_JP Morgan,11_239149101_Advent Geneva,39.0,0.0,0.0001,0.9998,0.0,NaN,One_to_One/Open,ML,UMB-Carry-Forward,UMB-Carry-Forward,NaN,-6.48,QP-CHN,CMDTYFUT,-47.88,05-04-2020,NaN,-6.48,0.0,6.48,EUR,JPM,QP_LP_INT_JPMF,NaN,MILL WHEAT EURO Sep20,09-10-2020,NaN,QP,NaN,06/12- Email sent to PB to provide backup on a...,CAU0,CMDTYFUT,NaN,"Currency,Mapped Custodian Account",05-05-2020,QP_LP_INT_JPMF,-41.4,NaN,14164,QP,QP,183.8389,JPMF,36.0,NaN,05-04-2020,UMB,Mismatch in Net Amount after tolerance. Tolera...,CAU0,05-04-2020,NaN,NaN,4419979,Buy,NaN,NaN,DUMMY,NaN,NaN,JP MORGAN FUTURES,2913535_SOROS,NaN,11_239149101_Advent Geneva19_239149101_JP Morgan
789,NaN,19_239149101_JP Morgan,11_239149101_Advent Geneva,42.0,0.0,0.0001,0.9998,0.0,NaN,One_to_One/Open,ML,UMB-Carry-Forward,UMB-Carry-Forward,NaN,-6.48,QP-CHN,CMDTYFUT,-47.88,05-04-2020,NaN,-6.48,0.0,6.48,EUR,JPM,QP_LP_INT_JPMF,NaN,MILL WHEAT EURO Sep20,09-10-2020,NaN,QP,NaN,06/15- Email sent to PB to provide backup on a...,CAU0,CMDTYFUT,NaN,"Currency,Mapped Custodian Account",05-05-2020,QP_LP_INT_JPMF,-41.4,NaN,14164,QP,QP,183.8389,JPMF,36.0,NaN,05-04-2020,UMB,Mismatch in Net Amount after tolerance. Tolera...,CAU0,05-04-2020,NaN,NaN,4419979,Buy,NaN,NaN,DUMMY,NaN,NaN,JP MORGAN FUTURES,2913535_SOROS,NaN,11_239149101_Advent Geneva19_239149101_JP Morgan
1139,NaN,19_239149101_JP Morgan,11_239149101_Advent Geneva,43.0,0.0,0.0001,0.9998,0.0,NaN,One_to_One/Open,ML,UMB-Carry-Forward,UMB-Carry-Forward,NaN,-6.48,QP-CHN,CMDTYFUT,-47.88,05-04-2020,NaN,-6.48,0.0,6.48,EUR,JPM,QP_LP_INT_JPMF,NaN,MILL WHEAT EURO Sep20,09-10-2020,NaN,QP,NaN,06/15- Email sent to PB to provide backup on a...,CAU0,CMDTYFUT,NaN,"Currency,Mapped Custodian Account",05-05-2020,QP_LP_INT_JPMF,-41.4,NaN,14164,QP,QP,183.8389,JPMF,36.0,NaN,05-04-2020,UMB,Mismatch in Net Amount after tolerance. Tolera...,CAU0,05-04-2020,NaN,NaN,4419979,Buy,NaN,NaN,DUMMY,NaN,NaN,JP MORGAN FUTURES,2913535_SOROS,NaN,11_2391491

In [6530]:
all_june_data2

,ViewData.BreakID,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,probability_No_pair,probability_UMR,probability_UMB,probability_UMT,Final_predicted_break,Type,ML_flag,Actual_Status,Predicted_Status,ViewData.Account Type,ViewData.Accounting Net Amount,ViewData.Alt ID 1,ViewData.Asset Type Category,ViewData.B-P Net Amount,ViewData.Business Date,ViewData.CUSIP,ViewData.Cancel Amount,ViewData.Cancel Flag,ViewData.Commission,ViewData.Currency,ViewData.Custodian,ViewData.Custodian Account,ViewData.Department,ViewData.Description,ViewData.ExpiryDate,ViewData.ExternalComment2,ViewData.Fund,ViewData.ISIN,ViewData.InternalComment2,ViewData.Investment ID,ViewData.Investment Type,ViewData.Is Combined Data,ViewData.Keys,ViewData.Knowledge Date,ViewData.Mapped Custodian Account,ViewData.Net Amount Difference,ViewData.Non Trade Description,ViewData.PB Account Numeric,ViewData.Portfolio ID,ViewData.Portolio,ViewData.Price,ViewData.Prime Broker,ViewData.Quantity,ViewData.SEDOL,ViewData.Settle Date,ViewData.Status,ViewData.System Comments,ViewData.Ticker,ViewData.Trade Date,ViewData.Trade Expenses,ViewData.Transaction Category,ViewData.Transaction ID,ViewData.Transaction Type,ViewData.Underlying Cusip,ViewData.Underlying ISIN,ViewData.Underlying Investment ID,ViewData.Underlying Sedol,ViewData.Underlying Ticker,ViewData.UserTran1,ViewData.UserTran2,ViewData.Value Date,key
0,NaN,"170_239151405_CITI,163_239151405_CITI,167_2391...","9_239151405_Advent Geneva,2_239151405_Advent G...",0.0,NaN,NaN,NaN,NaN,NaN,Many_to_Many,Non-ML,UMB,Open Break,NaN,129501.94,QP-CARS,CMDTYFUT,82713.94,06-02-2020,NaN,32375.485000,0.0,33.58,USD,CITI,QP_LP_INT_CITF,NaN,COPPER FUTURE Jul20,07-29-2020,NaN,QP,NaN,PnL difference due to lot closing. Awaiting fo...,HGN0,CMDTYFUT,True,"Currency,Mapped Custodian Account",06-03-2020,QP_LP_INT_CITF,-46788.00,NaN,34K50002,QP,QP,243.851175,CITF,103.00,NaN,06-02-2020,UMB,Mismatch in Net Amount after tolerance. Tolera...,HGN0,06-02-2020,NaN,NaN,4484286,Sell,NaN,NaN,DUMMY,NaN,NaN,CITIBANK NA FUTURES,2933595_SOROS,NaN,"9_239151405_Advent Geneva,2_239151405_Advent G..."
1,NaN,"69_239153733_CITI,71_239153733_CITI,67_2391537...","1_239153733_Advent Geneva,8_239153733_Advent G...",0.0,NaN,NaN,NaN,NaN,NaN,Many_to_Many,Non-ML,UMB,Open Break,NaN,-445617.11,QP-CHN,CMDTYFUT,6174.09,06-02-2020,NaN,-148539.036667,0.0,1218.00,EUR,CITI,QP_LP_INT_CITF,NaN,EURO STOXX 50 Jun20,06-19-2020,NaN,QP,NaN,PnL difference due to lot closing. Awaiting fo...,VGM0,INDXFUT,True,"Currency,Mapped Custodian Account",06-03-2020,QP_LP_INT_CITF,451791.20,NaN,34K50002,QP,QP,3148.792367,CITF,3045.00,NaN,06-02-2020,UMB,Mismatch in Net Amount after tolerance. Tolera...,VGM0,06-02-2020,NaN,NaN,4483265,Buy,NaN,NaN,SX5E,NaN,NaN,CITIBANK NA FUTURES,2933505_SOROS,NaN,"1_239153733_Advent Geneva,8_239153733_Advent G..."
2,142901225.0,"57_239153733_CITI,59_239153733_CITI,58_2391537...",9_239153733_Advent Geneva,0.0,NaN,NaN,NaN,NaN,NaN,One_to_Many,Non-ML,UMB,Open Break,NaN,-42.00,QP-MAIN,Cash,-30.80,06-02-2020,NaN,-42.000000,0.0,21.00,CHF,CITI,QP_LP_INT_CITF,NaN,SWISS MKT IX FUTR Jun20,06-19-2020,NaN,QP,NaN,"06/02- Booked in Geneva, break will clear in n...",SMM0,INDXFUT,True,"Currency,Mapped Custodian Account",06-03-2020,QP_LP_INT_CITF,11.20,NaN,34K50002,QP,QP,9937.571400,CITF,28.00,NaN,06-02-2020,UMB,Mismatch in Net Amount after tolerance. Tolera...,SMM0,06-02-2020,NaN,NaN,4482794,Buy,NaN,NaN,SMI,NaN,NaN,CITIBANK NA FUTURES,2933033_SOROS,NaN,"9_239153733_Advent Geneva57_239153733_CITI,59_..."
3,142670411.0,"108_239153733_CITI,107_239153733_CITI,110_2391...",20_239153513_Advent Geneva,0.0,NaN,NaN,NaN,NaN,NaN,One_to_Many,Non-ML,UMB,Open Break,NaN,-2892.00,QP-CARS,CMDTYFUT,-6.56,06-01-2020,NaN,-2892.000000,0.0,0.00,USD,CITI,QP_LP_INT_CITF,NaN,LME - NICKEL FORWARD:LMNIDP FUTURE 19-AUG-20 N...,08-17-2020,NaN,QP,NaN,PnL difference due to lot closing. Awaiting fo...,LMNIDS03 Comdty,CMDTYFUT,True,"Currency,Mapped Custodian Account",06-02-2020,QP_LP_INT_CITF,2885.44,NaN,34K5

In [6531]:
#all_june_data2 = all_june_data1[~((all_june_data1['ViewData.BreakID'].isnull()) & (all_june_data1['Actual_Status']=='UCB'))]
#comment_file2 = comment_file1[~((comment_file1['ViewData.BreakID'].isnull()) & (comment_file1['Actual_Status']=='UCB'))]

#all_june_data2 = all_june_data2.reset_index()
#all_june_data2 = all_june_data2.drop('index',1)

#comment_file2 = comment_file2.reset_index()
#comment_file2 = comment_file2.drop('index',1)

In [6501]:
#comment_file2[comment_file2['key']=='nan88_123895430_CITI'].drop_duplicates()

In [6532]:
all_june_data2.to_csv('//vitblrdevcons01/Raman  Strategy ML 2.0/All_Data/Soros/JuneData/Final_Predictions_239/All_june_data_239.csv')

In [6350]:
#new_comment = pd.merge(all_june_data2, comment_file2[['key', 'final_ID', 's/d', 'ViewData.InternalComment2', 'ViewData.Description',
#       'ViewData.Transaction Type', 'new_desc_cat', 'Created_cat_predicted',
#       'Created_cat_actual', 'error', 'Category', 'Predicted_comment']], on='key', how='left')

In [369]:
final12['Type'].value_counts()

One_to_One/Open    1361
Closed Breaks       237
One_to_Many          37
Many_to_Many          3
Name: Type, dtype: int64

In [371]:
final12[final12['Type'] == 'One_to_Many']

,ViewData.BreakID,ViewData.Side1_UniqueIds,ViewData.Side0_UniqueIds,ViewData.Age,probability_No_pair,probability_UMR,probability_UMB,probability_UMT,Final_predicted_break,Type,ML_flag,Actual_Status,Predicted_Status,probability_No_Pair,ViewData.Account Type,ViewData.Accounting Net Amount,ViewData.Alt ID 1,ViewData.Asset Type Category,ViewData.B-P Net Amount,ViewData.Business Date,ViewData.CUSIP,ViewData.Cancel Amount,ViewData.Cancel Flag,ViewData.Commission,ViewData.Currency,ViewData.Custodian,ViewData.Department,ViewData.Description,ViewData.ExpiryDate,ViewData.ExternalComment2,ViewData.Fund,ViewData.ISIN,ViewData.InternalComment2,ViewData.Investment ID,ViewData.Investment Type,ViewData.Is Combined Data,ViewData.Keys,ViewData.Knowledge Date,ViewData.Mapped Custodian Account,ViewData.Net Amount Difference,ViewData.Non Trade Description,ViewData.PB Account Numeric,ViewData.Portfolio ID,ViewData.Portolio,ViewData.Price,ViewData.Prime Broker,ViewData.Quantity,ViewData.SEDOL,ViewData.Settle Date,ViewData.Status,ViewData.System Comments,ViewData.Ticker,ViewData.Trade Date,ViewData.Trade Expenses,ViewData.Transaction Category,ViewData.Transaction ID,ViewData.Transaction Type,ViewData.Underlying Cusip,ViewData.Underlying ISIN,ViewData.Underlying Investment ID,ViewData.Underlying Sedol,ViewData.Underlying Ticker,ViewData.UserTran1,ViewData.UserTran2,ViewData.Value Date
26,NaN,"26_153157591_Goldman Sachs,27_153157591_Goldma...",16_153157591_Advent Geneva,0,NaN,NaN,NaN,NaN,NaN,One_to_Many,Non-ML,UMT,Open Break,NaN,NaN,7.495745e+06,QP-NAYA,Cash,7.495746e+06,06-26-2020,9EQ0HMNO0,7.495745e+06,0.0,0.0,JPY,GS,NaN,SOFTBANK GROUP CORP,NaN,NaN,QP,JP3436100006,Match,9984 JP,EQTY,True,"Currency,Mapped Custodian Account",06-29-2020,NAYA_EXT_GIPB,6.000000e-01,final Kunal Mittal 06102020 01:40 AM,05013831,QP,QP,22.0000,GIPB,0.000,6770620,06-26-2020,UMT,Match in Net Amount after tolerance. Tolerance...,9984 JP,03-30-2020,NaN,WDRL,4214694,Dividend,9EQ0HMNO0,JP3436100006,J9984,6770620,J9984,GIPB,QPNAYAGIPB192676522,NaN
31,146666659,"4_153157591_Goldman Sachs,6_153157591_Goldman ...",30_153157422_Advent Geneva,0,0.428407,0.008105,0.000665,0.562823,[146830497],One_to_Many,Non-ML,UMR,UMT,NaN,NaN,-6.637060e+04,QP-NAYA,Cash,-6.637060e+04,06-25-2020,9EDXYJ1F3,-6.637060e+04,0.0,489.6,EUR,GS,NaN,ATL IM 09/18/20 C17.5,09-18-2020,NaN,QP,IT0016088558,Match,ATL IM 09/18/20 C17.5,EQOPT,True,"Currency,Mapped Custodian Account",06-26-2020,NAYA_EXT_GIPB,0.000000e+00,CALL ATL MILE AMER 18 SEP 2020 17.5EUR20209,05013831,QP,QP,0.3660,GIPB,360.000,NaN,06-29-2020,UMR,NaN,ATL IM 09/18/20 C17.5,06-25-2020,NaN,BUY,4534415,Buy,9EQ0G18K2,IT0003506190,ATL IM,7667163,*AUTOIM,GIPB,2949432_SOROS,NaN
37,146212904,"4_153156963_Morgan Stanley,78_153157570_Morgan...",78_153156543_Advent Geneva,0,0.001440,0.829535,0.158804,0.010222,[146823126],One_to_Many,Non-ML,UMT,UMR,NaN,NaN,-1.032856e+09,QP-ECMR,Cash,-1.032856e+09,06-19-2020,999RYHDN6,-1.032856e+09,0.0,10226300.0,KRW,MS,NaN,SK BIOPHARMACEUTICALS CO LTD,NaN,NaN,QP,KR7326030004,Match,326030 KS,EQTY,True,"Currency,Mapped Custodian Account",06-22-2020,QP_LP_INT_MS,-9.999990e-03,SK BIOPHARMACEUTICALS CO LTD,038850863,QP,QP,49000.0000,MSPB,20870.000,BLF9YV7,06-26-2020,UMT,Match in Net Amount after tolerance. Tolerance...,326030 KS,06-19-2020,NaN,BUY LONG,4517535,Buy,NaN,NaN,NaN,NaN,NaN,MORGAN STANLEY PRIME BROKER,2945859_SOROS,06-23-2020
38,146212918,"75_153157570_Morgan Stanley,115_153156963_Morg...",182_153156543_Advent Geneva,0,0.000756,0.921821,0.069855,0.007568,[146823101],One_to_Many,Non-ML,UMT,UMR,NaN,NaN,-3.602872e+07,PL-ECMR,Cash,-3.602872e+07,06-19-2020,999850031,-3.602872e+07,0.0,356720.0,KRW,MS,NaN,SK BIOPHARMACEUTICALS CO LTD,NaN,NaN,PL,KR7326030004,Match,326030 KS,EQTY,True,"Currency,Mapped Custodian Account",06-22-2020,PLOF_INT_MS,-9.999998e-03,Pay,038CACNC2,PL,PL,49000.0000,MSPB,728.000,BLF9YV7,06-26-2020,UMT,Match in Net Amount after tolerance. Tolerance...,326030 KS,06-19-2020,NaN,FUNDS PAID OR RECEIVED,4517536,Buy,NaN,Na

In [382]:
X_test[(X_test['SideA.ViewData.Side1_UniqueIds']=='1141_153157324_CITI')]['Predicted_action'].value_counts()

No-Pair    342
Name: Predicted_action, dtype: int64